In [1]:
# after we install all the packages, import all of them for the use in today's lecture!
# database access
import pandas_datareader as web
import quandl as quandl
import wrds as wrds
# storage and operations
import pandas as pd
import numpy as np
import datetime
from pathlib import Path
import seaborn as sns

# stats
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from scipy import stats
from scipy.stats.mstats import winsorize
import os
from tqdm import tqdm
from zipfile import ZipFile
import seaborn
import matplotlib.pyplot as plt
import linearmodels as lm
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

#portfolio optimization:
# libraries we might use for testing
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose, STL
import linearmodels as lm
from linearmodels.panel import compare  
import time
import quantstats as qs

from datetime import datetime

path_factors = Path(r'C:\Users\mauri\FSoF\Frankfurt School of Finance - Master Thesis\Master Thesis - Data\Factors')


factor_data = path_factors / 'factor_data.h5'


factor_data_csv = path_factors / 'factor_data.csv'
factor_data_xlsx = path_factors / 'factor_data.xlsx'

monthly_factor_data_csv = path_factors / 'monthly_factor_data.csv'
semi_ann_factor_data_csv =path_factors / 'semi_ann_factor_data.csv'
yearly_factor_data_csv = path_factors / 'yearly_factor_data.csv'

In [2]:
liq_factor_pastor_stambaugh_file = path_factors/ 'Liqudity factors Stambaugh & Yuan/Liquidity factors of Pastor and Stambaugh (2003).CSV'
liq_factor_pastor_stambaugh = pd.read_csv(liq_factor_pastor_stambaugh_file, header=1)
liq_factor_pastor_stambaugh = pd.DataFrame(liq_factor_pastor_stambaugh)
liq_factor_pastor_stambaugh
# liq_factor_pastor_stambaugh.to_excel(path_factors/ 'Liqudity factors/Liquidity factors of Pastor and Stambaugh (2003).xlsx', header=True)

,Month,Agg Liq.,Innov Liq (eq8),Traded Liq (LIQ_V)
0,196208,-0.017537,0.004260,-99.000000
1,196209,-0.004075,0.011476,-99.000000
2,196210,-0.104212,-0.074792,-99.000000
3,196211,-0.019742,0.028435,-99.000000
4,196212,-0.005089,0.014639,-99.000000
...,...,...,...,...
707,202107,0.006380,0.040165,-0.000237
708,202108,0.012418,0.032290,0.003736
709,202109,-0.016866,-0.006602,-0.012899
710,202110,-0.006214,-0.008326,0.015679


In [3]:
ff_factors = pd.DataFrame(pd.read_csv(path_factors/'Fama-French/FF Factor Monthly/F-F_Research_Data_5_Factors_2x3_CSV.zip',header=2,nrows=704))
ff_mom = pd.DataFrame(pd.read_csv(path_factors/'Fama-French/FF Factor Monthly/F-F_Momentum_Factor_CSV.zip',header=11,nrows= 1144))


ff_factors = ff_factors.rename(columns={'Unnamed: 0' : 'date'})
ff_mom = ff_mom.rename(columns={'Unnamed: 0':'date'})

ff_factors['date'] = pd.to_datetime(ff_factors['date'],format='%Y%m')
ff_mom['date'] = pd.to_datetime(ff_mom['date'],format='%Y%m')

ff_factors = ff_factors.merge(ff_mom,how='left', on='date')
ff_factors.columns = [z.lower().strip() for z in ff_factors.columns]

ff_factors['date'] =ff_factors['date'].dt.strftime('%Y-%m')
ff_factors = ff_factors.set_index('date')
ff_factors = ff_factors/100
ff_factors

,mkt-rf,smb,hml,rmw,cma,rf,mom
date,,,,,,,
1963-07,-0.0039,-0.0044,-0.0089,0.0068,-0.0123,0.0027,0.0097
1963-08,0.0507,-0.0075,0.0168,0.0036,-0.0034,0.0025,0.0101
1963-09,-0.0157,-0.0055,0.0008,-0.0071,0.0029,0.0027,0.0019
1963-10,0.0253,-0.0137,-0.0014,0.0280,-0.0202,0.0029,0.0312
1963-11,-0.0085,-0.0089,0.0181,-0.0051,0.0231,0.0027,-0.0074
...,...,...,...,...,...,...,...
2021-10,0.0665,-0.0271,-0.0048,0.0174,-0.0151,0.0000,0.0319
2021-11,-0.0155,-0.0173,-0.0042,0.0738,0.0163,0.0000,0.0089
2021-12,0.0310,-0.0069,0.0322,0.0475,0.0436,0.0001,-0.0260


In [4]:
BAB_factor = pd.DataFrame(pd.read_excel(path_factors/'Betting against Beta (BAB)/bab,qml, hml devil factor extracted.xlsx'))
BAB_factor['date'] = pd.to_datetime(BAB_factor['date'])
BAB_factor['date'] = BAB_factor['date'].dt.strftime('%Y-%m')
BAB_factor.reset_index
BAB_factor = BAB_factor.set_index('date')
BAB_factor.dropna()


,bab,qmj,hml devil
date,,,
1957-08,0.034536,0.011154,0.004665
1957-09,0.037960,0.004877,-0.001842
1957-10,0.011091,0.007009,-0.026697
1957-11,0.015133,0.002708,-0.019449
1957-12,0.065397,-0.008971,-0.040814
...,...,...,...
2021-11,0.005623,0.021314,-0.021198
2021-12,0.035152,0.056344,0.036392
2022-01,-0.004235,0.041855,0.122366


In [5]:
all_factors = pd.merge(ff_factors, BAB_factor, how='left', on='date')
all_factors.dropna()
all_factors

,mkt-rf,smb,hml,rmw,cma,rf,mom,bab,qmj,hml devil
date,,,,,,,,,,
1963-07,-0.0039,-0.0044,-0.0089,0.0068,-0.0123,0.0027,0.0097,0.008123,0.007671,-0.005923
1963-08,0.0507,-0.0075,0.0168,0.0036,-0.0034,0.0025,0.0101,-0.012158,0.006844,0.002805
1963-09,-0.0157,-0.0055,0.0008,-0.0071,0.0029,0.0027,0.0019,0.010687,-0.009915,0.002653
1963-10,0.0253,-0.0137,-0.0014,0.0280,-0.0202,0.0029,0.0312,-0.018756,0.006236,-0.021129
1963-11,-0.0085,-0.0089,0.0181,-0.0051,0.0231,0.0027,-0.0074,-0.005198,0.020016,0.009074
...,...,...,...,...,...,...,...,...,...,...
2021-10,0.0665,-0.0271,-0.0048,0.0174,-0.0151,0.0000,0.0319,0.013240,-0.018356,-0.016657
2021-11,-0.0155,-0.0173,-0.0042,0.0738,0.0163,0.0000,0.0089,0.005623,0.021314,-0.021198
2021-12,0.0310,-0.0069,0.0322,0.0475,0.0436,0.0001,-0.0260,0.035152,0.056344,0.036392


# Profitability factors

factor returns are calculated as: 
* high minus low deciles OR high minus low quintiles 

* ### Asset Turnover monthly returns using quarterly time window (Ato_q)  

In [6]:
profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#Asset Turnover monthly returns using quarterly timeframe: 1 Month
ato_q1_factor = pd.read_csv(profitability_factors_path/'portf_me_atoq_1_monthly_2020.csv')

ato_q1_factor['date'] = ato_q1_factor['year'].astype(str) + '/' + ato_q1_factor['month'].astype(str)
ato_q1_factor['date'] = pd.to_datetime(ato_q1_factor['date'])
ato_q1_factor['date'] = ato_q1_factor['date'].dt.strftime('%Y-%m')
ato_q1_factor = ato_q1_factor.drop(columns=['year','month']).set_index('date')
ato_q1_factor['ret_vw'] =  ato_q1_factor['ret_vw']/100

# ato_q1_return = ato_q1_factor.groupby(['date','rank_ME']).apply(print)

ato_q1_return = ato_q1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ATOq_1']==5),'ret_vw']- x.loc[(x['rank_ATOq_1']==1),'ret_vw'])
ato_q1_return = pd.DataFrame(ato_q1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ato_q1_return = ato_q1_return.rename(columns={'ret_vw' : 'atoq_1'})
display(ato_q1_return)


#Asset Turnober monthly returns using quarterly timeframe: 6 Months 
ato_q6_factor = pd.read_csv(profitability_factors_path/'portf_me_atoq_6_monthly_2020.csv')

ato_q6_factor['date'] = ato_q6_factor['year'].astype(str) + '/' + ato_q6_factor['month'].astype(str)
ato_q6_factor['date'] = pd.to_datetime(ato_q6_factor['date'])
ato_q6_factor['date'] = ato_q6_factor['date'].dt.strftime('%Y-%m')
ato_q6_factor = ato_q6_factor.drop(columns=['year','month']).set_index('date')
ato_q6_factor['ret_vw'] =  ato_q6_factor['ret_vw']/100

ato_q6_return = ato_q6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ATOq_6']==5),'ret_vw']- x.loc[(x['rank_ATOq_6']==1),'ret_vw'])
ato_q6_return = pd.DataFrame(ato_q6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ato_q6_return = ato_q6_return.rename(columns={'ret_vw' : 'atoq_6'})
display(ato_q6_return)


#Asset Turnober monthly returns using quarterly timeframe: 12 Months 
ato_q12_factor = pd.read_csv(profitability_factors_path/'portf_me_atoq_12_monthly_2020.csv')

ato_q12_factor['date'] = ato_q12_factor['year'].astype(str) + '/' + ato_q12_factor['month'].astype(str)
ato_q12_factor['date'] = pd.to_datetime(ato_q12_factor['date'])
ato_q12_factor['date'] = ato_q12_factor['date'].dt.strftime('%Y-%m')
ato_q12_factor = ato_q12_factor.drop(columns=['year','month']).set_index('date')
ato_q12_factor['ret_vw'] =  ato_q12_factor['ret_vw']/100

# ato_q12_factor.groupby(['date','rank_ME']).apply(print)
ato_q12_return = ato_q12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ATOq_12']==5),'ret_vw']- x.loc[(x['rank_ATOq_12']==1),'ret_vw'])
ato_q12_return = pd.DataFrame(ato_q12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ato_q12_return = ato_q12_return.rename(columns={'ret_vw' : 'atoq_12'})
display(ato_q12_return)

all_factors = pd.merge(all_factors, ato_q1_return, how='left', on='date')
all_factors = pd.merge(all_factors, ato_q6_return, how='left', on='date')
all_factors = pd.merge(all_factors, ato_q12_return, how='left', on='date')

,atoq_1
date,
1972-01,-0.057703
1972-02,0.009958
1972-03,-0.001725
1972-04,-0.018721
1972-05,-0.005519
...,...
2020-08,0.030009
2020-09,0.010916
2020-10,0.002211


,atoq_6
date,
1972-01,-0.057703
1972-02,0.010947
1972-03,-0.006382
1972-04,-0.016370
1972-05,0.001254
...,...
2020-08,0.026902
2020-09,0.004547
2020-10,0.023142


,atoq_12
date,
1972-01,-0.057703
1972-02,0.010947
1972-03,-0.006382
1972-04,-0.016370
1972-05,0.001254
...,...
2020-08,0.021677
2020-09,0.009368
2020-10,0.024962


* ### cash-based operating profits-to-lagged assets monthly returns using 1 month and quarterly window (Cla & Cla_q) 

In [7]:
profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#Cash-based operating profits-to-lagged assets returns
cla_factor = pd.read_csv(profitability_factors_path/'portf_me_cla_monthly_2020.csv')

cla_factor['date'] = cla_factor['year'].astype(str) + '/' + cla_factor['month'].astype(str)
cla_factor['date'] = pd.to_datetime(cla_factor['date'])
cla_factor['date'] = cla_factor['date'].dt.strftime('%Y-%m')
cla_factor = cla_factor.drop(columns=['year','month']).set_index('date')
cla_factor['ret_vw'] =  cla_factor['ret_vw']/100

cla_return = cla_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CLA']==5),'ret_vw']- x.loc[(x['rank_CLA']==1),'ret_vw'])
cla_return = pd.DataFrame(cla_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cla_return = cla_return.rename(columns={'ret_vw' : 'cla'})
display(cla_return)


#Cash-based operating profits-to-lagged assets returns using quarterly timeframe: 1 month
cla_q1_factor = pd.read_csv(profitability_factors_path/'portf_me_claq_1_monthly_2020.csv')

cla_q1_factor['date'] = cla_q1_factor['year'].astype(str) + '/' + cla_q1_factor['month'].astype(str)
cla_q1_factor['date'] = pd.to_datetime(cla_q1_factor['date'])
cla_q1_factor['date'] = cla_q1_factor['date'].dt.strftime('%Y-%m')
cla_q1_factor = cla_q1_factor.drop(columns=['year','month']).set_index('date')
cla_q1_factor['ret_vw'] =  cla_q1_factor['ret_vw']/100
# cla_q1_factor.head(20)

cla_q1_return = cla_q1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CLAq_1']==5),'ret_vw']- x.loc[(x['rank_CLAq_1']==1),'ret_vw'])
cla_q1_return = pd.DataFrame(cla_q1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cla_q1_return = cla_q1_return.rename(columns={'ret_vw' : 'claq_1'})
display(cla_q1_return)


#Cash-based operating profits-to-lagged assets returns using quarterly timeframe: 6 months
cla_q6_factor = pd.read_csv(profitability_factors_path/'portf_me_claq_6_monthly_2020.csv')

cla_q6_factor['date'] = cla_q6_factor['year'].astype(str) + '/' + cla_q6_factor['month'].astype(str)
cla_q6_factor['date'] = pd.to_datetime(cla_q6_factor['date'])
cla_q6_factor['date'] = cla_q6_factor['date'].dt.strftime('%Y-%m')
cla_q6_factor = cla_q6_factor.drop(columns=['year','month']).set_index('date')
cla_q6_factor['ret_vw'] =  cla_q6_factor['ret_vw']/100
# cla_q6_factor.head(20)

cla_q6_return = cla_q6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CLAq_6']==5),'ret_vw']- x.loc[(x['rank_CLAq_6']==1),'ret_vw'])
cla_q6_return = pd.DataFrame(cla_q6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cla_q6_return = cla_q6_return.rename(columns={'ret_vw' : 'claq_6'})
display(cla_q6_return)


#Cash-based operating profits-to-lagged assets returns using quarterly timeframe: 12 months
cla_q12_factor = pd.read_csv(profitability_factors_path/'portf_me_claq_12_monthly_2020.csv')

cla_q12_factor['date'] = cla_q12_factor['year'].astype(str) + '/' + cla_q12_factor['month'].astype(str)
cla_q12_factor['date'] = pd.to_datetime(cla_q12_factor['date'])
cla_q12_factor['date'] = cla_q12_factor['date'].dt.strftime('%Y-%m')
cla_q12_factor = cla_q12_factor.drop(columns=['year','month']).set_index('date')
cla_q12_factor['ret_vw'] =  cla_q12_factor['ret_vw']/100
# cla_q6_factor.head(20)

cla_q12_return = cla_q12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CLAq_12']==5),'ret_vw']- x.loc[(x['rank_CLAq_12']==1),'ret_vw'])
cla_q12_return = pd.DataFrame(cla_q12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cla_q12_return = cla_q12_return.rename(columns={'ret_vw' : 'claq_12'})
display(cla_q12_return)

all_factors = pd.merge(all_factors, cla_return, how='left', on='date')
all_factors = pd.merge(all_factors, cla_q1_return, how='left', on='date')
all_factors = pd.merge(all_factors, cla_q6_return, how='left', on='date')
all_factors = pd.merge(all_factors, cla_q12_return, how='left', on='date')


,cla
date,
1967-01,-0.057522
1967-02,0.004463
1967-03,-0.026500
1967-04,-0.005294
1967-05,-0.002536
...,...
2020-08,0.014467
2020-09,-0.017570
2020-10,0.013470


,claq_1
date,
1976-01,-0.009971
1976-02,0.008703
1976-03,0.006150
1976-04,-0.009882
1976-05,0.018613
...,...
2020-08,0.002316
2020-09,-0.032043
2020-10,0.006985


,claq_6
date,
1976-01,-0.009971
1976-02,0.003362
1976-03,0.009714
1976-04,-0.000977
1976-05,0.014903
...,...
2020-08,-0.005318
2020-09,-0.015959
2020-10,0.023700


,claq_12
date,
1976-01,-0.009971
1976-02,0.003362
1976-03,0.009714
1976-04,-0.000977
1976-05,0.014903
...,...
2020-08,-0.025772
2020-09,-0.011384
2020-10,0.022420


* ### Cash-based operating profitability monthly returns (cop)

In [8]:
profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#Cash-based operating profitability returns
cop_factor = pd.read_csv(profitability_factors_path/'portf_me_cop_monthly_2020.csv')

cop_factor['date'] = cop_factor['year'].astype(str) + '/' + cop_factor['month'].astype(str)
cop_factor['date'] = pd.to_datetime(cop_factor['date'])
cop_factor['date'] = cop_factor['date'].dt.strftime('%Y-%m')
cop_factor = cop_factor.drop(columns=['year','month']).set_index('date')
cop_factor['ret_vw'] =  cop_factor['ret_vw']/100
# cop_factor.head(20)

cop_return = cop_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_COP']==5),'ret_vw']- x.loc[(x['rank_COP']==1),'ret_vw'])
cop_return = pd.DataFrame(cop_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cop_return = cop_return.rename(columns={'ret_vw' : 'cop'})
display(cop_return)

all_factors = pd.merge(all_factors, cop_return, how='left', on='date')

,cop
date,
1967-01,-0.062516
1967-02,0.005975
1967-03,-0.025076
1967-04,-0.005491
1967-05,-0.000388
...,...
2020-08,0.016758
2020-09,-0.015408
2020-10,0.014390


* ### Capital Turnover monthly returns using quarterly timewindow (ctoq)

In [9]:
profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#Capital turnover returns using quarterly timeframe: 1 month
cto_q1_factor = pd.read_csv(profitability_factors_path/'portf_me_ctoq_1_monthly_2020.csv')

cto_q1_factor['date'] = cto_q1_factor['year'].astype(str) + '/' + cto_q1_factor['month'].astype(str)
cto_q1_factor['date'] = pd.to_datetime(cto_q1_factor['date'])
cto_q1_factor['date'] = cto_q1_factor['date'].dt.strftime('%Y-%m')
cto_q1_factor = cto_q1_factor.drop(columns=['year','month']).set_index('date')
cto_q1_factor['ret_vw'] =  cto_q1_factor['ret_vw']/100
# cto_q1_factor.head(20)

cto_q1_return = cto_q1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CTOq_1']==5),'ret_vw']- x.loc[(x['rank_CTOq_1']==1),'ret_vw'])
cto_q1_return = pd.DataFrame(cto_q1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cto_q1_return = cto_q1_return.rename(columns={'ret_vw' : 'ctoq_1'})
display(cto_q1_return)

#Capital turnover returns using quarterly timeframe: 6 months
cto_q6_factor = pd.read_csv(profitability_factors_path/'portf_me_ctoq_6_monthly_2020.csv')

cto_q6_factor['date'] = cto_q6_factor['year'].astype(str) + '/' + cto_q6_factor['month'].astype(str)
cto_q6_factor['date'] = pd.to_datetime(cto_q6_factor['date'])
cto_q6_factor['date'] = cto_q6_factor['date'].dt.strftime('%Y-%m')
cto_q6_factor = cto_q6_factor.drop(columns=['year','month']).set_index('date')
cto_q6_factor['ret_vw'] =  cto_q6_factor['ret_vw']/100
# cto_q6_factor.head(20)

cto_q6_return = cto_q6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CTOq_6']==5),'ret_vw']- x.loc[(x['rank_CTOq_6']==1),'ret_vw'])
cto_q6_return = pd.DataFrame(cto_q6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cto_q6_return = cto_q6_return.rename(columns={'ret_vw' : 'ctoq_6'})
display(cto_q6_return)


#Capital turnover returns using quarterly timeframe: 12 months
cto_q12_factor = pd.read_csv(profitability_factors_path/'portf_me_ctoq_12_monthly_2020.csv')

cto_q12_factor['date'] = cto_q12_factor['year'].astype(str) + '/' + cto_q12_factor['month'].astype(str)
cto_q12_factor['date'] = pd.to_datetime(cto_q12_factor['date'])
cto_q12_factor['date'] = cto_q12_factor['date'].dt.strftime('%Y-%m')
cto_q12_factor = cto_q12_factor.drop(columns=['year','month']).set_index('date')
cto_q12_factor['ret_vw'] =  cto_q12_factor['ret_vw']/100
# cto_q6_factor.head(20)

cto_q12_return = cto_q12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CTOq_12']==5),'ret_vw']- x.loc[(x['rank_CTOq_12']==1),'ret_vw'])
cto_q12_return = pd.DataFrame(cto_q12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cto_q12_return = cto_q12_return.rename(columns={'ret_vw' : 'ctoq_12'})
display(cto_q12_return)


all_factors = pd.merge(all_factors, cto_q1_return, how='left', on='date')
all_factors = pd.merge(all_factors, cto_q6_return, how='left', on='date')
all_factors = pd.merge(all_factors, cto_q12_return, how='left', on='date')

,ctoq_1
date,
1972-01,-0.051970
1972-02,0.012612
1972-03,0.008194
1972-04,-0.029956
1972-05,-0.025688
...,...
2020-08,0.024674
2020-09,0.004310
2020-10,0.009221


,ctoq_6
date,
1972-01,-0.051970
1972-02,0.013590
1972-03,0.002626
1972-04,-0.033902
1972-05,-0.004631
...,...
2020-08,0.032272
2020-09,-0.003464
2020-10,0.021017


,ctoq_12
date,
1972-01,-0.051970
1972-02,0.013590
1972-03,0.002626
1972-04,-0.033902
1972-05,-0.004631
...,...
2020-08,0.031443
2020-09,-0.003235
2020-10,0.021672


* ### changes in returns of assets (dRoa)

In [10]:
profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#changes in returns of asset returns: 1 month
droa_1_factor = pd.read_csv(profitability_factors_path/'portf_me_droa_1_monthly_2020.csv')

droa_1_factor['date'] = droa_1_factor['year'].astype(str) + '/' + droa_1_factor['month'].astype(str)
droa_1_factor['date'] = pd.to_datetime(droa_1_factor['date'])
droa_1_factor['date'] = droa_1_factor['date'].dt.strftime('%Y-%m')
droa_1_factor = droa_1_factor.drop(columns=['year','month']).set_index('date')
droa_1_factor['ret_vw'] =  droa_1_factor['ret_vw']/100
# droa_1_factor.head(20)

droa_1_return = droa_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dROA_1']==5),'ret_vw']- x.loc[(x['rank_dROA_1']==1),'ret_vw'])
droa_1_return = pd.DataFrame(droa_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
droa_1_return = droa_1_return.rename(columns={'ret_vw' : 'droa_1'})
display(droa_1_return)


#changes in returns of asset returns: 6 months
droa_6_factor = pd.read_csv(profitability_factors_path/'portf_me_droa_6_monthly_2020.csv')

droa_6_factor['date'] = droa_6_factor['year'].astype(str) + '/' + droa_6_factor['month'].astype(str)
droa_6_factor['date'] = pd.to_datetime(droa_6_factor['date'])
droa_6_factor['date'] = droa_6_factor['date'].dt.strftime('%Y-%m')
droa_6_factor = droa_6_factor.drop(columns=['year','month']).set_index('date')
droa_6_factor['ret_vw'] =  droa_6_factor['ret_vw']/100
# droa_1_factor.head(20)

droa_6_return = droa_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dROA_6']==5),'ret_vw']- x.loc[(x['rank_dROA_6']==1),'ret_vw'])
droa_6_return = pd.DataFrame(droa_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
droa_6_return = droa_6_return.rename(columns={'ret_vw' : 'droa_6'})
display(droa_6_return)

all_factors = pd.merge(all_factors, droa_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, droa_6_return, how='left', on='date')

,droa_1
date,
1973-01,0.011067
1973-02,-0.021857
1973-03,0.057597
1973-04,-0.007892
1973-05,0.018803
...,...
2020-08,-0.019043
2020-09,0.009297
2020-10,0.000861


,droa_6
date,
1973-01,0.011067
1973-02,-0.014617
1973-03,0.045529
1973-04,0.008672
1973-05,0.019716
...,...
2020-08,-0.007254
2020-09,0.009450
2020-10,-0.005989


* ### changes in return on equity (dRoe)

In [11]:
profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#changes in return on equity: 1 month
droe_1_factor = pd.read_csv(profitability_factors_path/'portf_me_droe_1_monthly_2020.csv')

droe_1_factor['date'] = droe_1_factor['year'].astype(str) + '/' + droe_1_factor['month'].astype(str)
droe_1_factor['date'] = pd.to_datetime(droe_1_factor['date'])
droe_1_factor['date'] = droe_1_factor['date'].dt.strftime('%Y-%m')
droe_1_factor = droe_1_factor.drop(columns=['year','month']).set_index('date')
droe_1_factor['ret_vw'] =  droe_1_factor['ret_vw']/100
# droe_1_factor.head(20)

droe_1_return = droe_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dROE_1']==5),'ret_vw']- x.loc[(x['rank_dROE_1']==1),'ret_vw'])
droe_1_return = pd.DataFrame(droe_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
droe_1_return = droe_1_return.rename(columns={'ret_vw' : 'droe_1'})
display(droe_1_return)


#changes in return on equity: 6 months
droe_6_factor = pd.read_csv(profitability_factors_path/'portf_me_droe_6_monthly_2020.csv')

droe_6_factor['date'] = droe_6_factor['year'].astype(str) + '/' + droe_6_factor['month'].astype(str)
droe_6_factor['date'] = pd.to_datetime(droe_6_factor['date'])
droe_6_factor['date'] = droe_6_factor['date'].dt.strftime('%Y-%m')
droe_6_factor = droe_6_factor.drop(columns=['year','month']).set_index('date')
droe_6_factor['ret_vw'] =  droe_6_factor['ret_vw']/100
# droe_6_factor.head(20)

droe_6_return = droe_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dROE_6']==5),'ret_vw']- x.loc[(x['rank_dROE_6']==1),'ret_vw'])
droe_6_return = pd.DataFrame(droe_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
droe_6_return = droe_6_return.rename(columns={'ret_vw' : 'droe_6'})
display(droe_6_return)


#changes in return on equity: 12 months
droe_12_factor = pd.read_csv(profitability_factors_path/'portf_me_droe_12_monthly_2020.csv')

droe_12_factor['date'] = droe_12_factor['year'].astype(str) + '/' + droe_12_factor['month'].astype(str)
droe_12_factor['date'] = pd.to_datetime(droe_12_factor['date'])
droe_12_factor['date'] = droe_12_factor['date'].dt.strftime('%Y-%m')
droe_12_factor = droe_12_factor.drop(columns=['year','month']).set_index('date')
droe_12_factor['ret_vw'] =  droe_12_factor['ret_vw']/100
# droe_6_factor.head(20)

droe_12_return = droe_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dROE_12']==5),'ret_vw']- x.loc[(x['rank_dROE_12']==1),'ret_vw'])
droe_12_return = pd.DataFrame(droe_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
droe_12_return = droe_12_return.rename(columns={'ret_vw' : 'droe_12'})
display(droe_12_return)


all_factors = pd.merge(all_factors, droe_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, droe_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, droe_12_return, how='left', on='date')

,droe_1
date,
1967-01,-0.016221
1967-02,0.017339
1967-03,0.009528
1967-04,0.000384
1967-05,0.005905
...,...
2020-08,-0.023479
2020-09,0.001698
2020-10,0.002349


,droe_6
date,
1967-01,-0.016221
1967-02,0.009671
1967-03,0.020303
1967-04,0.001226
1967-05,0.005696
...,...
2020-08,0.009578
2020-09,0.013830
2020-10,-0.003977


,droe_12
date,
1967-01,-0.016221
1967-02,0.009671
1967-03,0.020303
1967-04,0.001226
1967-05,0.005696
...,...
2020-08,0.009211
2020-09,0.009578
2020-10,-0.003081


* ### failure probability 6 month return (fp) 

In [12]:
profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#Failure probability
fp_6_factor = pd.read_csv(profitability_factors_path/'portf_me_fp_6_monthly_2020.csv')

fp_6_factor['date'] = fp_6_factor['year'].astype(str) + '/' + fp_6_factor['month'].astype(str)
fp_6_factor['date'] = pd.to_datetime(fp_6_factor['date'])
fp_6_factor['date'] = fp_6_factor['date'].dt.strftime('%Y-%m')
fp_6_factor = fp_6_factor.drop(columns=['year','month']).set_index('date')
fp_6_factor['ret_vw'] =  fp_6_factor['ret_vw']/100
# fp_6_factor.head(20)

fp_6_return = fp_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_FP_6']==5),'ret_vw']- x.loc[(x['rank_FP_6']==1),'ret_vw'])
fp_6_return = pd.DataFrame(fp_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
fp_6_return = fp_6_return.rename(columns={'ret_vw' : 'fp_6'})
display(fp_6_return)

all_factors = pd.merge(all_factors, fp_6_return, how='left', on='date')

,fp_6
date,
1976-01,-0.028342
1976-02,0.018727
1976-03,0.022656
1976-04,-0.005341
1976-05,0.004649
...,...
2020-08,0.006379
2020-09,-0.017566
2020-10,0.003301


* ### Quarterly fundamental score (fq)

not included in factor list since late availability of data

In [13]:
# profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

# #Quarterly fundamental score returns: 1 month 
# fq_1_factor = pd.read_csv(profitability_factors_path/'portf_me_fq_1_monthly_2020.csv')

# fq_1_factor['date'] = fq_1_factor['year'].astype(str) + '/' + fq_1_factor['month'].astype(str)
# fq_1_factor['date'] = pd.to_datetime(fq_1_factor['date'])
# fq_1_factor['date'] = fq_1_factor['date'].dt.strftime('%Y-%m')
# fq_1_factor = fq_1_factor.drop(columns=['year','month']).set_index('date')
# fq_1_factor['ret_vw'] =  fq_1_factor['ret_vw']/100
# # fq_1_factor.head(20)

# fq_1_return = fq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_Fq_1']==1),'ret_vw']- x.loc[(x['rank_Fq_1']==5),'ret_vw'])
# fq_1_return = pd.DataFrame(fq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
# fq_1_return = fq_1_return.rename(columns={'ret_vw' : 'fq_1'})
# display(fq_1_return)

# #Quarterly fundamental score returns: 6 months 
# fq_6_factor = pd.read_csv(profitability_factors_path/'portf_me_fq_6_monthly_2020.csv')

# fq_6_factor['date'] = fq_6_factor['year'].astype(str) + '/' + fq_6_factor['month'].astype(str)
# fq_6_factor['date'] = pd.to_datetime(fq_6_factor['date'])
# fq_6_factor['date'] = fq_6_factor['date'].dt.strftime('%Y-%m')
# fq_6_factor = fq_6_factor.drop(columns=['year','month']).set_index('date')
# fq_6_factor['ret_vw'] =  fq_6_factor['ret_vw']/100
# # fq_6_factor.head(20)

# fq_6_return = fq_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_Fq_6']==1),'ret_vw']- x.loc[(x['rank_Fq_6']==5),'ret_vw'])
# fq_6_return = pd.DataFrame(fq_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
# fq_6_return = fq_6_return.rename(columns={'ret_vw' : 'fq_6'})
# display(fq_6_return)

# #Quarterly fundamental score returns: 12 months 
# fq_12_factor = pd.read_csv(profitability_factors_path/'portf_me_fq_12_monthly_2020.csv')

# fq_12_factor['date'] = fq_12_factor['year'].astype(str) + '/' + fq_12_factor['month'].astype(str)
# fq_12_factor['date'] = pd.to_datetime(fq_12_factor['date'])
# fq_12_factor['date'] = fq_12_factor['date'].dt.strftime('%Y-%m')
# fq_12_factor = fq_12_factor.drop(columns=['year','month']).set_index('date')
# fq_12_factor['ret_vw'] =  fq_12_factor['ret_vw']/100
# # fq_12_factor.head(20)

# fq_12_return = fq_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_Fq_12']==1),'ret_vw']- x.loc[(x['rank_Fq_12']==5),'ret_vw'])
# fq_12_return = pd.DataFrame(fq_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
# fq_12_return = fq_12_return.rename(columns={'ret_vw' : 'fq_12'})
# display(fq_12_return)


# all_factors = pd.merge(all_factors, fq_1_return, how='left', on='date')
# all_factors = pd.merge(all_factors, fq_6_return, how='left', on='date')
# all_factors = pd.merge(all_factors, fq_12_return, how='left', on='date')


* ### quarterly gross-profits-to-lagged assets (glaq) 

In [14]:
profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#Quarterly gross-profits-tolagged assets returns: 1 month 
glaq_1_factor = pd.read_csv(profitability_factors_path/'portf_me_glaq_1_monthly_2020.csv')

glaq_1_factor['date'] = glaq_1_factor['year'].astype(str) + '/' + glaq_1_factor['month'].astype(str)
glaq_1_factor['date'] = pd.to_datetime(glaq_1_factor['date'])
glaq_1_factor['date'] = glaq_1_factor['date'].dt.strftime('%Y-%m')
glaq_1_factor = glaq_1_factor.drop(columns=['year','month']).set_index('date')
glaq_1_factor['ret_vw'] =  glaq_1_factor['ret_vw']/100
# glaq_1_factor.head(20)

glaq_1_return = glaq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_GLAq_1']==5),'ret_vw']- x.loc[(x['rank_GLAq_1']==1),'ret_vw'])
glaq_1_return = pd.DataFrame(glaq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
glaq_1_return = glaq_1_return.rename(columns={'ret_vw' : 'glaq_1'})
display(glaq_1_return)

#Quarterly gross-profits-tolagged assets returns: 6 months 
glaq_6_factor = pd.read_csv(profitability_factors_path/'portf_me_glaq_6_monthly_2020.csv')

glaq_6_factor['date'] = glaq_6_factor['year'].astype(str) + '/' + glaq_6_factor['month'].astype(str)
glaq_6_factor['date'] = pd.to_datetime(glaq_6_factor['date'])
glaq_6_factor['date'] = glaq_6_factor['date'].dt.strftime('%Y-%m')
glaq_6_factor = glaq_6_factor.drop(columns=['year','month']).set_index('date')
glaq_6_factor['ret_vw'] =  glaq_6_factor['ret_vw']/100
# glaq_6_factor.head(20)

glaq_6_return = glaq_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_GLAq_6']==5),'ret_vw']- x.loc[(x['rank_GLAq_6']==1),'ret_vw'])
glaq_6_return = pd.DataFrame(glaq_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
glaq_6_return = glaq_6_return.rename(columns={'ret_vw' : 'glaq_6'})
display(glaq_6_return)


#Quarterly gross-profits-tolagged assets returns: 12 months 
glaq_12_factor = pd.read_csv(profitability_factors_path/'portf_me_glaq_12_monthly_2020.csv')

glaq_12_factor['date'] = glaq_12_factor['year'].astype(str) + '/' + glaq_12_factor['month'].astype(str)
glaq_12_factor['date'] = pd.to_datetime(glaq_12_factor['date'])
glaq_12_factor['date'] = glaq_12_factor['date'].dt.strftime('%Y-%m')
glaq_12_factor = glaq_12_factor.drop(columns=['year','month']).set_index('date')
glaq_12_factor['ret_vw'] =  glaq_12_factor['ret_vw']/100
# glaq_12_factor.head(20)

glaq_12_return = glaq_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_GLAq_12']==5),'ret_vw']- x.loc[(x['rank_GLAq_12']==1),'ret_vw'])
glaq_12_return = pd.DataFrame(glaq_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
glaq_12_return = glaq_12_return.rename(columns={'ret_vw' : 'glaq_12'})
display(glaq_12_return)


all_factors = pd.merge(all_factors, glaq_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, glaq_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, glaq_12_return, how='left', on='date')

,glaq_1
date,
1976-01,-0.012039
1976-02,-0.019323
1976-03,0.028168
1976-04,-0.045705
1976-05,-0.007212
...,...
2020-08,0.024053
2020-09,-0.000306
2020-10,0.004576


,glaq_6
date,
1976-01,-0.012039
1976-02,-0.020925
1976-03,0.025752
1976-04,-0.032592
1976-05,-0.017179
...,...
2020-08,0.034348
2020-09,-0.003049
2020-10,0.012944


,glaq_12
date,
1976-01,-0.012039
1976-02,-0.020925
1976-03,0.025752
1976-04,-0.032592
1976-05,-0.017179
...,...
2020-08,0.025079
2020-09,-0.004042
2020-10,0.009812


* ### Gross profits to assets (gpa)

In [15]:
profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#gross-profits to assets returns: 1 month 
gpa_factor = pd.read_csv(profitability_factors_path/'portf_me_gpa_monthly_2020.csv')

gpa_factor['date'] = gpa_factor['year'].astype(str) + '/' + gpa_factor['month'].astype(str)
gpa_factor['date'] = pd.to_datetime(gpa_factor['date'])
gpa_factor['date'] = gpa_factor['date'].dt.strftime('%Y-%m')
gpa_factor = gpa_factor.drop(columns=['year','month']).set_index('date')
gpa_factor['ret_vw'] =  gpa_factor['ret_vw']/100
gpa_factor.head(20)

gpa_return = gpa_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_GPA']==5),'ret_vw']- x.loc[(x['rank_GPA']==1),'ret_vw'])
gpa_return = pd.DataFrame(gpa_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
gpa_return = gpa_return.rename(columns={'ret_vw' : 'gpa'})
display(gpa_return)

all_factors = pd.merge(all_factors, gpa_return, how='left', on='date')

,gpa
date,
1967-01,0.023040
1967-02,0.031121
1967-03,0.006730
1967-04,0.020699
1967-05,0.003936
...,...
2020-08,0.044723
2020-09,-0.016317
2020-10,0.018686


* ### Quarterly operating profits to lagged assets (olaq) 

In [16]:
profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#quarterly operating profits to lagged assets returns: 1 month 
olaq_1_factor = pd.read_csv(profitability_factors_path/'portf_me_olaq_1_monthly_2020.csv')

olaq_1_factor['date'] = olaq_1_factor['year'].astype(str) + '/' + olaq_1_factor['month'].astype(str)
olaq_1_factor['date'] = pd.to_datetime(olaq_1_factor['date'])
olaq_1_factor['date'] = olaq_1_factor['date'].dt.strftime('%Y-%m')
olaq_1_factor = olaq_1_factor.drop(columns=['year','month']).set_index('date')
olaq_1_factor['ret_vw'] =  olaq_1_factor['ret_vw']/100
olaq_1_factor.head(20)

olaq_1_return = olaq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_OLAq_1']==5),'ret_vw']- x.loc[(x['rank_OLAq_1']==1),'ret_vw'])
olaq_1_return = pd.DataFrame(olaq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
olaq_1_return = olaq_1_return.rename(columns={'ret_vw' : 'olaq_1'})
display(olaq_1_return)


#quarterly operating profits to lagged assets returns: 6 months 
olaq_6_factor = pd.read_csv(profitability_factors_path/'portf_me_olaq_6_monthly_2020.csv')

olaq_6_factor['date'] = olaq_6_factor['year'].astype(str) + '/' + olaq_6_factor['month'].astype(str)
olaq_6_factor['date'] = pd.to_datetime(olaq_6_factor['date'])
olaq_6_factor['date'] = olaq_6_factor['date'].dt.strftime('%Y-%m')
olaq_6_factor = olaq_6_factor.drop(columns=['year','month']).set_index('date')
olaq_6_factor['ret_vw'] =  olaq_6_factor['ret_vw']/100
olaq_6_factor.head(20)

olaq_6_return = olaq_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_OLAq_6']==5),'ret_vw']- x.loc[(x['rank_OLAq_6']==1),'ret_vw'])
olaq_6_return = pd.DataFrame(olaq_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
olaq_6_return = olaq_6_return.rename(columns={'ret_vw' : 'olaq_6'})
display(olaq_6_return)


#quarterly operating profits to lagged assets returns: 12 months 
olaq_12_factor = pd.read_csv(profitability_factors_path/'portf_me_olaq_12_monthly_2020.csv')

olaq_12_factor['date'] = olaq_12_factor['year'].astype(str) + '/' + olaq_12_factor['month'].astype(str)
olaq_12_factor['date'] = pd.to_datetime(olaq_12_factor['date'])
olaq_12_factor['date'] = olaq_12_factor['date'].dt.strftime('%Y-%m')
olaq_12_factor = olaq_12_factor.drop(columns=['year','month']).set_index('date')
olaq_12_factor['ret_vw'] =  olaq_12_factor['ret_vw']/100
olaq_12_factor.head(20)

olaq_12_return = olaq_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_OLAq_12']==5),'ret_vw']- x.loc[(x['rank_OLAq_12']==1),'ret_vw'])
olaq_12_return = pd.DataFrame(olaq_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
olaq_12_return = olaq_12_return.rename(columns={'ret_vw' : 'olaq_12'})
display(olaq_12_return)

all_factors = pd.merge(all_factors, olaq_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, olaq_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, olaq_12_return, how='left', on='date')

,olaq_1
date,
1976-01,-0.015682
1976-02,-0.000281
1976-03,-0.004822
1976-04,-0.006719
1976-05,0.012415
...,...
2020-08,-0.016263
2020-09,-0.020865
2020-10,0.005105


,olaq_6
date,
1976-01,-0.015682
1976-02,-0.009009
1976-03,-0.001812
1976-04,0.000833
1976-05,0.016511
...,...
2020-08,-0.021408
2020-09,-0.017520
2020-10,0.014967


,olaq_12
date,
1976-01,-0.015682
1976-02,-0.009009
1976-03,-0.001812
1976-04,0.000833
1976-05,0.016511
...,...
2020-08,-0.037724
2020-09,-0.012047
2020-10,0.011377


* ### Quarterly operating profits to lagged equity (oleq) 

In [17]:
profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#quarterly operating profits to lagged equity returns: 1 month 
oleq_1_factor = pd.read_csv(profitability_factors_path/'portf_me_oleq_1_monthly_2020.csv')

oleq_1_factor['date'] = oleq_1_factor['year'].astype(str) + '/' + oleq_1_factor['month'].astype(str)
oleq_1_factor['date'] = pd.to_datetime(oleq_1_factor['date'])
oleq_1_factor['date'] = oleq_1_factor['date'].dt.strftime('%Y-%m')
oleq_1_factor = oleq_1_factor.drop(columns=['year','month']).set_index('date')
oleq_1_factor['ret_vw'] =  oleq_1_factor['ret_vw']/100
oleq_1_factor.head(20)

oleq_1_return = oleq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_OLEq_1']==5),'ret_vw']- x.loc[(x['rank_OLEq_1']==1),'ret_vw'])
oleq_1_return = pd.DataFrame(oleq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
oleq_1_return = oleq_1_return.rename(columns={'ret_vw' : 'oleq_1'})
display(oleq_1_return)

#quarterly operating profits to lagged equity returns: 6 months 
oleq_6_factor = pd.read_csv(profitability_factors_path/'portf_me_oleq_6_monthly_2020.csv')

oleq_6_factor['date'] = oleq_6_factor['year'].astype(str) + '/' + oleq_6_factor['month'].astype(str)
oleq_6_factor['date'] = pd.to_datetime(oleq_6_factor['date'])
oleq_6_factor['date'] = oleq_6_factor['date'].dt.strftime('%Y-%m')
oleq_6_factor = oleq_6_factor.drop(columns=['year','month']).set_index('date')
oleq_6_factor['ret_vw'] =  oleq_6_factor['ret_vw']/100
oleq_6_factor.head(20)

oleq_6_return = oleq_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_OLEq_6']==5),'ret_vw']- x.loc[(x['rank_OLEq_6']==1),'ret_vw'])
oleq_6_return = pd.DataFrame(oleq_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
oleq_6_return = oleq_6_return.rename(columns={'ret_vw' : 'oleq_6'})
display(oleq_6_return)


all_factors = pd.merge(all_factors, oleq_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, oleq_6_return, how='left', on='date')

,oleq_1
date,
1972-01,0.006586
1972-02,0.028217
1972-03,-0.010657
1972-04,0.042641
1972-05,0.015586
...,...
2020-08,0.021451
2020-09,-0.023329
2020-10,0.007932


,oleq_6
date,
1972-01,0.006586
1972-02,0.014790
1972-03,-0.011052
1972-04,0.031478
1972-05,0.009574
...,...
2020-08,0.032996
2020-09,-0.035328
2020-10,0.016041


* ### Quarterly return on net operating assets (rnaq) 

In [18]:
profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#quarterly return on net operating assets: 1 month 
rnaq_1_factor = pd.read_csv(profitability_factors_path/'portf_me_rnaq_1_monthly_2020.csv')

rnaq_1_factor['date'] = rnaq_1_factor['year'].astype(str) + '/' + rnaq_1_factor['month'].astype(str)
rnaq_1_factor['date'] = pd.to_datetime(rnaq_1_factor['date'])
rnaq_1_factor['date'] = rnaq_1_factor['date'].dt.strftime('%Y-%m')
rnaq_1_factor = rnaq_1_factor.drop(columns=['year','month']).set_index('date')
rnaq_1_factor['ret_vw'] =  rnaq_1_factor['ret_vw']/100
rnaq_1_factor.head(20)

rnaq_1_return = rnaq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_RNAq_1']==5),'ret_vw']- x.loc[(x['rank_RNAq_1']==1),'ret_vw'])
rnaq_1_return = pd.DataFrame(rnaq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
rnaq_1_return = rnaq_1_return.rename(columns={'ret_vw' : 'rnaq_1'})
display(rnaq_1_return)

#quarterly return on net operating assets: 6 months 
rnaq_6_factor = pd.read_csv(profitability_factors_path/'portf_me_rnaq_6_monthly_2020.csv')

rnaq_6_factor['date'] = rnaq_6_factor['year'].astype(str) + '/' + rnaq_6_factor['month'].astype(str)
rnaq_6_factor['date'] = pd.to_datetime(rnaq_6_factor['date'])
rnaq_6_factor['date'] = rnaq_6_factor['date'].dt.strftime('%Y-%m')
rnaq_6_factor = rnaq_6_factor.drop(columns=['year','month']).set_index('date')
rnaq_6_factor['ret_vw'] =  rnaq_6_factor['ret_vw']/100
rnaq_6_factor.head(20)

rnaq_6_return = rnaq_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_RNAq_6']==5),'ret_vw']- x.loc[(x['rank_RNAq_6']==1),'ret_vw'])
rnaq_6_return = pd.DataFrame(rnaq_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
rnaq_6_return = rnaq_6_return.rename(columns={'ret_vw' : 'rnaq_6'})
display(rnaq_6_return)

all_factors = pd.merge(all_factors, rnaq_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, rnaq_6_return, how='left', on='date')

,rnaq_1
date,
1976-01,-0.033160
1976-02,-0.033749
1976-03,-0.015023
1976-04,-0.018231
1976-05,-0.013836
...,...
2020-08,-0.005432
2020-09,-0.015155
2020-10,0.010398


,rnaq_6
date,
1976-01,-0.033160
1976-02,-0.031582
1976-03,-0.010179
1976-04,-0.017041
1976-05,-0.000035
...,...
2020-08,0.000389
2020-09,-0.011939
2020-10,0.020626


* ### return on assets (roa)

In [19]:
 profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#return on assets 1 month return
roa_1_factor = pd.read_csv(profitability_factors_path/'portf_me_roa_1_monthly_2020.csv')

roa_1_factor['date'] = roa_1_factor['year'].astype(str) + '/' + roa_1_factor['month'].astype(str)
roa_1_factor['date'] = pd.to_datetime(roa_1_factor['date'])
roa_1_factor['date'] = roa_1_factor['date'].dt.strftime('%Y-%m')
roa_1_factor = roa_1_factor.drop(columns=['year','month']).set_index('date')
roa_1_factor['ret_vw'] =  roa_1_factor['ret_vw']/100
roa_1_factor.head(20)

roa_1_return = roa_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ROA_1']==5),'ret_vw']- x.loc[(x['rank_ROA_1']==1),'ret_vw'])
roa_1_return = pd.DataFrame(roa_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
roa_1_return = roa_1_return.rename(columns={'ret_vw' : 'roa_1'})
display(roa_1_return)

all_factors = pd.merge(all_factors, roa_1_return, how='left', on='date')

,roa_1
date,
1972-01,-0.110551
1972-02,0.035176
1972-03,0.035547
1972-04,-0.016457
1972-05,0.025499
...,...
2020-08,-0.035943
2020-09,0.000424
2020-10,0.003738


* ### Return on equity (roe) 

In [20]:
 profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#return on equity 1 month return
roe_1_factor = pd.read_csv(profitability_factors_path/'portf_me_roe_1_monthly_2020.csv')

roe_1_factor['date'] = roe_1_factor['year'].astype(str) + '/' + roe_1_factor['month'].astype(str)
roe_1_factor['date'] = pd.to_datetime(roe_1_factor['date'])
roe_1_factor['date'] = roe_1_factor['date'].dt.strftime('%Y-%m')
roe_1_factor = roe_1_factor.drop(columns=['year','month']).set_index('date')
roe_1_factor['ret_vw'] =  roe_1_factor['ret_vw']/100
roe_1_factor.head(20)

roe_1_return = roe_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ROE_1']==5),'ret_vw']- x.loc[(x['rank_ROE_1']==1),'ret_vw'])
roe_1_return = pd.DataFrame(roe_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
roe_1_return = roe_1_return.rename(columns={'ret_vw' : 'roe_1'})
display(roe_1_return)


#return on equity 6 month return
roe_6_factor = pd.read_csv(profitability_factors_path/'portf_me_roe_6_monthly_2020.csv')

roe_6_factor['date'] = roe_6_factor['year'].astype(str) + '/' + roe_6_factor['month'].astype(str)
roe_6_factor['date'] = pd.to_datetime(roe_6_factor['date'])
roe_6_factor['date'] = roe_6_factor['date'].dt.strftime('%Y-%m')
roe_6_factor = roe_6_factor.drop(columns=['year','month']).set_index('date')
roe_6_factor['ret_vw'] =  roe_6_factor['ret_vw']/100
roe_6_factor.head(20)

roe_6_return = roe_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ROE_6']==5),'ret_vw']- x.loc[(x['rank_ROE_6']==1),'ret_vw'])
roe_6_return = pd.DataFrame(roe_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
roe_6_return = roe_6_return.rename(columns={'ret_vw' : 'roe_6'})
display(roe_6_return)



all_factors = pd.merge(all_factors, roe_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, roe_6_return, how='left', on='date')

,roe_1
date,
1967-01,0.026033
1967-02,0.036529
1967-03,0.012667
1967-04,0.013764
1967-05,0.009913
...,...
2020-08,-0.027907
2020-09,-0.008190
2020-10,-0.002547


,roe_6
date,
1967-01,0.026033
1967-02,0.028675
1967-03,0.021222
1967-04,0.018225
1967-05,0.005907
...,...
2020-08,0.005912
2020-09,-0.006804
2020-10,0.005792


# 

* ### Quarterly taxable income to book (tbiq)

In [21]:
 profitability_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Profitability_me_monthly'

#quarterly taxable income to book: 6 month return
tbiq_6_factor = pd.read_csv(profitability_factors_path/'portf_me_tbiq_6_monthly_2020.csv')

tbiq_6_factor['date'] = tbiq_6_factor['year'].astype(str) + '/' + tbiq_6_factor['month'].astype(str)
tbiq_6_factor['date'] = pd.to_datetime(tbiq_6_factor['date'])
tbiq_6_factor['date'] = tbiq_6_factor['date'].dt.strftime('%Y-%m')
tbiq_6_factor = tbiq_6_factor.drop(columns=['year','month']).set_index('date')
tbiq_6_factor['ret_vw'] =  tbiq_6_factor['ret_vw']/100
tbiq_6_factor.head(20)

tbiq_6_return = tbiq_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_TBIq_6']==5),'ret_vw']- x.loc[(x['rank_TBIq_6']==1),'ret_vw'])
tbiq_6_return = pd.DataFrame(tbiq_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
tbiq_6_return = tbiq_6_return.rename(columns={'ret_vw' : 'tbiq_6'})
display(tbiq_6_return)



#quarterly taxable income to book: 12 month return
tbiq_12_factor = pd.read_csv(profitability_factors_path/'portf_me_tbiq_12_monthly_2020.csv')

tbiq_12_factor['date'] = tbiq_12_factor['year'].astype(str) + '/' + tbiq_12_factor['month'].astype(str)
tbiq_12_factor['date'] = pd.to_datetime(tbiq_12_factor['date'])
tbiq_12_factor['date'] = tbiq_12_factor['date'].dt.strftime('%Y-%m')
tbiq_12_factor = tbiq_12_factor.drop(columns=['year','month']).set_index('date')
tbiq_12_factor['ret_vw'] =  tbiq_12_factor['ret_vw']/100
tbiq_12_factor.head(20)

tbiq_12_return = tbiq_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_TBIq_12']==5),'ret_vw']- x.loc[(x['rank_TBIq_12']==1),'ret_vw'])
tbiq_12_return = pd.DataFrame(tbiq_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
tbiq_12_return = tbiq_12_return.rename(columns={'ret_vw' : 'tbiq_12'})
display(tbiq_12_return)


all_factors = pd.merge(all_factors, tbiq_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, tbiq_12_return, how='left', on='date')

,tbiq_6
date,
1967-01,-0.026128
1967-02,0.017681
1967-03,-0.003044
1967-04,0.014340
1967-05,-0.008083
...,...
2020-08,0.023411
2020-09,-0.046113
2020-10,-0.019826


,tbiq_12
date,
1967-01,-0.026128
1967-02,0.017681
1967-03,-0.003044
1967-04,0.014340
1967-05,-0.008083
...,...
2020-08,0.022186
2020-09,-0.030054
2020-10,-0.013322


# Friction Factors 

factor returns are calculated as: 
* high minus low deciles OR high minus low quintiles 

* ### Dollar Trading Volume (Volatility of liquidity) (dtv)

In [22]:
frictions_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Frictions_me_monthly'

#Beta factor 1 month return
dtv_12_factor = pd.read_csv(frictions_factors_path/'portf_me_dtv_12_monthly_2020.csv')

dtv_12_factor['date'] = dtv_12_factor['year'].astype(str) + '/' + dtv_12_factor['month'].astype(str)
dtv_12_factor['date'] = pd.to_datetime(dtv_12_factor['date'])
dtv_12_factor['date'] = dtv_12_factor['date'].dt.strftime('%Y-%m')
dtv_12_factor = dtv_12_factor.drop(columns=['year','month']).set_index('date')
dtv_12_factor['ret_vw'] =  dtv_12_factor['ret_vw']/100
dtv_12_factor.head(20)

dtv_12_return = dtv_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_DTV_12']==5),'ret_vw']- x.loc[(x['rank_DTV_12']==1),'ret_vw'])
dtv_12_return = pd.DataFrame(dtv_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
dtv_12_return = dtv_12_return.rename(columns={'ret_vw' : 'dtv_12'})
display(dtv_12_return)

all_factors = pd.merge(all_factors, dtv_12_return, how='left', on='date')


,dtv_12
date,
1967-01,-0.002024
1967-02,-0.003549
1967-03,-0.019926
1967-04,-0.026774
1967-05,-0.024792
...,...
2020-08,0.035736
2020-09,0.140698
2020-10,-0.035899


* ### Idiosyncratic skewness per the q-factor model (isq)

In [23]:
frictions_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Frictions_me_monthly'

#idiosyncratic skewness factor 1 month return
isq_1_factor = pd.read_csv(frictions_factors_path/'portf_me_isq_1_monthly_2020.csv')

isq_1_factor['date'] = isq_1_factor['year'].astype(str) + '/' + isq_1_factor['month'].astype(str)
isq_1_factor['date'] = pd.to_datetime(isq_1_factor['date'])
isq_1_factor['date'] = isq_1_factor['date'].dt.strftime('%Y-%m')
isq_1_factor = isq_1_factor.drop(columns=['year','month']).set_index('date')
isq_1_factor['ret_vw'] =  isq_1_factor['ret_vw']/100
isq_1_factor.head(20)

isq_1_return = isq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ISq_1']==5),'ret_vw']- x.loc[(x['rank_ISq_1']==1),'ret_vw'])
isq_1_return = pd.DataFrame(isq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
isq_1_return = isq_1_return.rename(columns={'ret_vw' : 'isq_1'})
display(isq_1_return)

all_factors = pd.merge(all_factors, isq_1_return, how='left', on='date')


,isq_1
date,
1967-02,-0.005598
1967-03,0.004531
1967-04,-0.004740
1967-05,-0.012072
1967-06,0.036105
...,...
2020-08,-0.017114
2020-09,-0.007142
2020-10,0.000683


* ### idiosyncratic volatility per the q-factor model (ivq)

In [24]:
frictions_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Frictions_me_monthly'

#idiosyncratic skewness factor 1 month return
ivq_1_factor = pd.read_csv(frictions_factors_path/'portf_me_ivq_1_monthly_2020.csv')

ivq_1_factor['date'] = ivq_1_factor['year'].astype(str) + '/' + ivq_1_factor['month'].astype(str)
ivq_1_factor['date'] = pd.to_datetime(ivq_1_factor['date'])
ivq_1_factor['date'] = ivq_1_factor['date'].dt.strftime('%Y-%m')
ivq_1_factor = ivq_1_factor.drop(columns=['year','month']).set_index('date')
ivq_1_factor['ret_vw'] =  ivq_1_factor['ret_vw']/100
ivq_1_factor.head(20)

ivq_1_return = ivq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IVq_1']==5),'ret_vw']- x.loc[(x['rank_IVq_1']==1),'ret_vw'])
ivq_1_return = pd.DataFrame(ivq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ivq_1_return = ivq_1_return.rename(columns={'ret_vw' : 'ivq_1'})
display(ivq_1_return)

all_factors = pd.merge(all_factors, ivq_1_return, how='left', on='date')


,ivq_1
date,
1967-02,0.032685
1967-03,0.008950
1967-04,-0.004391
1967-05,0.002213
1967-06,0.077509
...,...
2020-08,0.062903
2020-09,-0.015683
2020-10,-0.028704


* ### Systematic Volatitlity risk (sv1)

not included in final factor list due to data availability

In [25]:
# frictions_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Frictions_me_monthly'

# #idiosyncratic skewness factor 1 month return
# sv_1_factor = pd.read_csv(frictions_factors_path/'portf_me_sv_1_monthly_2020.csv')

# sv_1_factor['date'] = sv_1_factor['year'].astype(str) + '/' + sv_1_factor['month'].astype(str)
# sv_1_factor['date'] = pd.to_datetime(sv_1_factor['date'])
# sv_1_factor['date'] = sv_1_factor['date'].dt.strftime('%Y-%m')
# sv_1_factor = sv_1_factor.drop(columns=['year','month']).set_index('date')
# sv_1_factor['ret_vw'] =  sv_1_factor['ret_vw']/100
# sv_1_factor.head(20)

# sv_1_return = sv_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_SV_1']==1),'ret_vw']- x.loc[(x['rank_SV_1']==5),'ret_vw'])
# sv_1_return = pd.DataFrame(sv_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
# sv_1_return = sv_1_return.rename(columns={'ret_vw' : 'sv_1'})
# display(sv_1_return)

# all_factors = pd.merge(all_factors, sv_1_return, how='left', on='date')


# Intangible Factors 
factor returns are calculated as: 
* high minus low deciles OR high minus low quintiles 

* ### Advertising expense to market (adm)

In [26]:
intangible_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Intan_me_monthly'

#idiosyncratic skewness factor 1 month return
adm_factor = pd.read_csv(intangible_factors_path/'portf_me_adm_monthly_2020.csv')

adm_factor['date'] = adm_factor['year'].astype(str) + '/' + adm_factor['month'].astype(str)
adm_factor['date'] = pd.to_datetime(adm_factor['date'])
adm_factor['date'] = adm_factor['date'].dt.strftime('%Y-%m')
adm_factor = adm_factor.drop(columns=['year','month']).set_index('date')
adm_factor['ret_vw'] =  adm_factor['ret_vw']/100
adm_factor.head(20)

adm_return = adm_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ADM']==5),'ret_vw']- x.loc[(x['rank_ADM']==1),'ret_vw'])
adm_return = pd.DataFrame(adm_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
adm_return = adm_return.rename(columns={'ret_vw' : 'adm'})
display(adm_return)

all_factors = pd.merge(all_factors, adm_return, how='left', on='date')



,adm
date,
1973-07,-0.032067
1973-08,-0.004036
1973-09,-0.015242
1973-10,-0.019378
1973-11,0.008195
...,...
2020-08,0.000493
2020-09,-0.009621
2020-10,0.044951


* ### quarterly Asset Liquidity (alm)

In [27]:
intangible_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Intan_me_monthly'

#quarterly asset liquidity: 1 month return
almq_1_factor = pd.read_csv(intangible_factors_path/'portf_me_almq_1_monthly_2020.csv')

almq_1_factor['date'] = almq_1_factor['year'].astype(str) + '/' + almq_1_factor['month'].astype(str)
almq_1_factor['date'] = pd.to_datetime(almq_1_factor['date'])
almq_1_factor['date'] = almq_1_factor['date'].dt.strftime('%Y-%m')
almq_1_factor = almq_1_factor.drop(columns=['year','month']).set_index('date')
almq_1_factor['ret_vw'] =  almq_1_factor['ret_vw']/100
almq_1_factor.head(20)

almq_1_return = almq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ALMq_1']==5),'ret_vw']- x.loc[(x['rank_ALMq_1']==1),'ret_vw'])
almq_1_return = pd.DataFrame(almq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
almq_1_return = almq_1_return.rename(columns={'ret_vw' : 'almq_1'})
display(almq_1_return)


#quarterly asset liquidity: 6 month return
almq_6_factor = pd.read_csv(intangible_factors_path/'portf_me_almq_6_monthly_2020.csv')

almq_6_factor['date'] = almq_6_factor['year'].astype(str) + '/' + almq_6_factor['month'].astype(str)
almq_6_factor['date'] = pd.to_datetime(almq_6_factor['date'])
almq_6_factor['date'] = almq_6_factor['date'].dt.strftime('%Y-%m')
almq_6_factor = almq_6_factor.drop(columns=['year','month']).set_index('date')
almq_6_factor['ret_vw'] =  almq_6_factor['ret_vw']/100
almq_6_factor.head(20)

almq_6_return = almq_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ALMq_6']==5),'ret_vw']- x.loc[(x['rank_ALMq_6']==1),'ret_vw'])
almq_6_return = pd.DataFrame(almq_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
almq_6_return = almq_6_return.rename(columns={'ret_vw' : 'almq_6'})
display(almq_6_return)


#quarterly asset liquidity: 12 month return
almq_12_factor = pd.read_csv(intangible_factors_path/'portf_me_almq_12_monthly_2020.csv')

almq_12_factor['date'] = almq_12_factor['year'].astype(str) + '/' + almq_12_factor['month'].astype(str)
almq_12_factor['date'] = pd.to_datetime(almq_12_factor['date'])
almq_12_factor['date'] = almq_12_factor['date'].dt.strftime('%Y-%m')
almq_12_factor = almq_12_factor.drop(columns=['year','month']).set_index('date')
almq_12_factor['ret_vw'] =  almq_12_factor['ret_vw']/100
almq_12_factor.head(20)

almq_12_return = almq_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ALMq_12']==5),'ret_vw']- x.loc[(x['rank_ALMq_12']==1),'ret_vw'])
almq_12_return = pd.DataFrame(almq_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
almq_12_return = almq_12_return.rename(columns={'ret_vw' : 'almq_12'})
display(almq_12_return)


all_factors = pd.merge(all_factors, almq_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, almq_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, almq_12_return, how='left', on='date')




,almq_1
date,
1976-01,0.098552
1976-02,0.039683
1976-03,0.025318
1976-04,0.001982
1976-05,-0.031804
...,...
2020-08,0.009167
2020-09,-0.004355
2020-10,0.056244


,almq_6
date,
1976-01,0.098552
1976-02,0.044274
1976-03,0.013362
1976-04,0.005503
1976-05,-0.031157
...,...
2020-08,0.004676
2020-09,-0.008132
2020-10,0.050133


,almq_12
date,
1976-01,0.098552
1976-02,0.044274
1976-03,0.013362
1976-04,0.005503
1976-05,-0.031157
...,...
2020-08,0.004975
2020-09,-0.003496
2020-10,0.049368


* ### Earnings predictability (eprd)


In [28]:
#earnings predictability: 1 month return
eprd_factor = pd.read_csv(intangible_factors_path/'portf_me_eprd_monthly_2020.csv')

eprd_factor['date'] = eprd_factor['year'].astype(str) + '/' + eprd_factor['month'].astype(str)
eprd_factor['date'] = pd.to_datetime(eprd_factor['date'])
eprd_factor['date'] = eprd_factor['date'].dt.strftime('%Y-%m')
eprd_factor = eprd_factor.drop(columns=['year','month']).set_index('date')
eprd_factor['ret_vw'] =  eprd_factor['ret_vw']/100
eprd_factor.head(20)

eprd_return = eprd_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_EPRD']==5),'ret_vw']- x.loc[(x['rank_EPRD']==1),'ret_vw'])
eprd_return = pd.DataFrame(eprd_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
eprd_return = eprd_return.rename(columns={'ret_vw' : 'eprd'})
display(eprd_return)

all_factors = pd.merge(all_factors, eprd_return, how='left', on='date')

,eprd
date,
1967-01,0.006694
1967-02,0.023350
1967-03,-0.018446
1967-04,-0.019423
1967-05,-0.014772
...,...
2020-08,0.001435
2020-09,-0.017084
2020-10,0.011396


* ### Earnings timeliness (etl) 

In [29]:
#earnings timeliness: 1 month return
etl_factor = pd.read_csv(intangible_factors_path/'portf_me_etl_monthly_2020.csv')

etl_factor['date'] = etl_factor['year'].astype(str) + '/' + etl_factor['month'].astype(str)
etl_factor['date'] = pd.to_datetime(etl_factor['date'])
etl_factor['date'] = etl_factor['date'].dt.strftime('%Y-%m')
etl_factor = etl_factor.drop(columns=['year','month']).set_index('date')
etl_factor['ret_vw'] =  etl_factor['ret_vw']/100
etl_factor.head(20)

etl_return = etl_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ETL']==5),'ret_vw']- x.loc[(x['rank_ETL']==1),'ret_vw'])
etl_return = pd.DataFrame(etl_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
etl_return = etl_return.rename(columns={'ret_vw' : 'etl'})
display(etl_return)

all_factors = pd.merge(all_factors, etl_return, how='left', on='date')

,etl
date,
1967-01,0.041918
1967-02,0.003329
1967-03,-0.000345
1967-04,-0.011089
1967-05,0.031879
...,...
2020-08,0.029389
2020-09,0.002178
2020-10,0.004922


* ### effective tax rate (etr)

In [30]:
#effective tax rate: 1 month return
etr_factor = pd.read_csv(intangible_factors_path/'portf_me_etr_monthly_2020.csv')

etr_factor['date'] = etr_factor['year'].astype(str) + '/' + etr_factor['month'].astype(str)
etr_factor['date'] = pd.to_datetime(etr_factor['date'])
etr_factor['date'] = etr_factor['date'].dt.strftime('%Y-%m')
etr_factor = etr_factor.drop(columns=['year','month']).set_index('date')
etr_factor['ret_vw'] =  etr_factor['ret_vw']/100
etr_factor.head(20)

etr_return = etr_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ETR']==5),'ret_vw']- x.loc[(x['rank_ETR']==1),'ret_vw'])
etr_return = pd.DataFrame(etr_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
etr_return = etr_return.rename(columns={'ret_vw' : 'etr'})
display(etr_return)

all_factors = pd.merge(all_factors, etr_return, how='left', on='date')

,etr
date,
1967-01,0.003969
1967-02,-0.000921
1967-03,-0.036147
1967-04,-0.026905
1967-05,-0.013893
...,...
2020-08,0.029733
2020-09,0.028852
2020-10,0.025191


* ### Industry concentration (hs) 

In [31]:
#industry concentration: 1 month return
hs_factor = pd.read_csv(intangible_factors_path/'portf_me_hs_monthly_2020.csv')

hs_factor['date'] = hs_factor['year'].astype(str) + '/' + hs_factor['month'].astype(str)
hs_factor['date'] = pd.to_datetime(hs_factor['date'])
hs_factor['date'] = hs_factor['date'].dt.strftime('%Y-%m')
hs_factor = hs_factor.drop(columns=['year','month']).set_index('date')
hs_factor['ret_vw'] =  hs_factor['ret_vw']/100
hs_factor.head(20)

hs_return = hs_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_HS']==5),'ret_vw']- x.loc[(x['rank_HS']==1),'ret_vw'])
hs_return = pd.DataFrame(hs_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
hs_return = hs_return.rename(columns={'ret_vw' : 'hs'})
display(hs_return)

all_factors = pd.merge(all_factors, hs_return, how='left', on='date')

,hs
date,
1967-01,-0.002596
1967-02,0.021711
1967-03,0.016177
1967-04,0.025399
1967-05,-0.017341
...,...
2020-08,0.061308
2020-09,-0.014455
2020-10,0.004586


* ### Industry adjusted organisational capital to assets (ioca) & organisational capital to assets (oca)

In [32]:
#Industry adjusted organisational capital to assets: 1 month return
ioca_factor = pd.read_csv(intangible_factors_path/'portf_me_ioca_monthly_2020.csv')

ioca_factor['date'] = ioca_factor['year'].astype(str) + '/' + ioca_factor['month'].astype(str)
ioca_factor['date'] = pd.to_datetime(ioca_factor['date'])
ioca_factor['date'] = ioca_factor['date'].dt.strftime('%Y-%m')
ioca_factor = ioca_factor.drop(columns=['year','month']).set_index('date')
ioca_factor['ret_vw'] =  ioca_factor['ret_vw']/100
ioca_factor.head(20)

ioca_return = ioca_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IOCA']==5),'ret_vw']- x.loc[(x['rank_IOCA']==1),'ret_vw'])
ioca_return = pd.DataFrame(ioca_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ioca_return = ioca_return.rename(columns={'ret_vw' : 'ioca'})
display(ioca_return)


#organisational capital to assets: 1 month return
oca_factor = pd.read_csv(intangible_factors_path/'portf_me_oca_monthly_2020.csv')

oca_factor['date'] = oca_factor['year'].astype(str) + '/' + oca_factor['month'].astype(str)
oca_factor['date'] = pd.to_datetime(oca_factor['date'])
oca_factor['date'] = oca_factor['date'].dt.strftime('%Y-%m')
oca_factor = oca_factor.drop(columns=['year','month']).set_index('date')
oca_factor['ret_vw'] =  oca_factor['ret_vw']/100
oca_factor.head(20)

oca_return = oca_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_OCA']==5),'ret_vw']- x.loc[(x['rank_OCA']==1),'ret_vw'])
oca_return = pd.DataFrame(oca_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
oca_return = oca_return.rename(columns={'ret_vw' : 'oca'})
display(oca_return)


all_factors = pd.merge(all_factors, ioca_return, how='left', on='date')
all_factors = pd.merge(all_factors, oca_return, how='left', on='date')

,ioca
date,
1967-01,-0.007527
1967-02,0.006995
1967-03,0.010163
1967-04,-0.005599
1967-05,0.014400
...,...
2020-08,-0.026807
2020-09,0.004964
2020-10,0.020505


,oca
date,
1967-01,-0.014776
1967-02,0.030734
1967-03,0.000803
1967-04,0.017833
1967-05,0.001710
...,...
2020-08,-0.000953
2020-09,0.028692
2020-10,0.034674


* ### Operating leverage (ol) & Quarterly operating leverage (olq) 

In [33]:
#Operating leverage: 1 month return
ol_factor = pd.read_csv(intangible_factors_path/'portf_me_ol_monthly_2020.csv')

ol_factor['date'] = ol_factor['year'].astype(str) + '/' + ol_factor['month'].astype(str)
ol_factor['date'] = pd.to_datetime(ol_factor['date'])
ol_factor['date'] = ol_factor['date'].dt.strftime('%Y-%m')
ol_factor = ol_factor.drop(columns=['year','month']).set_index('date')
ol_factor['ret_vw'] =  ol_factor['ret_vw']/100
ol_factor.head(20)

ol_return = ol_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_OL']==5),'ret_vw']- x.loc[(x['rank_OL']==1),'ret_vw'])
ol_return = pd.DataFrame(ol_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ol_return = ol_return.rename(columns={'ret_vw' : 'ol'})
display(ol_return)


#Quarterly operating leverage: 1 month return
olq_1_factor = pd.read_csv(intangible_factors_path/'portf_me_olq_1_monthly_2020.csv')

olq_1_factor['date'] = olq_1_factor['year'].astype(str) + '/' + olq_1_factor['month'].astype(str)
olq_1_factor['date'] = pd.to_datetime(olq_1_factor['date'])
olq_1_factor['date'] = olq_1_factor['date'].dt.strftime('%Y-%m')
olq_1_factor = olq_1_factor.drop(columns=['year','month']).set_index('date')
olq_1_factor['ret_vw'] =  olq_1_factor['ret_vw']/100
olq_1_factor.head(20)

olq_1_return = olq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_OLq_1']==5),'ret_vw']- x.loc[(x['rank_OLq_1']==1),'ret_vw'])
olq_1_return = pd.DataFrame(olq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
olq_1_return = olq_1_return.rename(columns={'ret_vw' : 'olq_1'})
display(olq_1_return)


#Quarterly operating leverage: 6 month return
olq_6_factor = pd.read_csv(intangible_factors_path/'portf_me_olq_6_monthly_2020.csv')

olq_6_factor['date'] = olq_6_factor['year'].astype(str) + '/' + olq_6_factor['month'].astype(str)
olq_6_factor['date'] = pd.to_datetime(olq_6_factor['date'])
olq_6_factor['date'] = olq_6_factor['date'].dt.strftime('%Y-%m')
olq_6_factor = olq_6_factor.drop(columns=['year','month']).set_index('date')
olq_6_factor['ret_vw'] =  olq_6_factor['ret_vw']/100
olq_6_factor.head(20)

olq_6_return = olq_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_OLq_6']==5),'ret_vw']- x.loc[(x['rank_OLq_6']==1),'ret_vw'])
olq_6_return = pd.DataFrame(olq_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
olq_6_return = olq_6_return.rename(columns={'ret_vw' : 'olq_6'})
display(olq_6_return)


#Quarterly operating leverage: 12 month return
olq_12_factor = pd.read_csv(intangible_factors_path/'portf_me_olq_12_monthly_2020.csv')

olq_12_factor['date'] = olq_12_factor['year'].astype(str) + '/' + olq_12_factor['month'].astype(str)
olq_12_factor['date'] = pd.to_datetime(olq_12_factor['date'])
olq_12_factor['date'] = olq_12_factor['date'].dt.strftime('%Y-%m')
olq_12_factor = olq_12_factor.drop(columns=['year','month']).set_index('date')
olq_12_factor['ret_vw'] =  olq_12_factor['ret_vw']/100
olq_12_factor.head(20)

olq_12_return = olq_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_OLq_12']==5),'ret_vw']- x.loc[(x['rank_OLq_12']==1),'ret_vw'])
olq_12_return = pd.DataFrame(olq_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
olq_12_return = olq_12_return.rename(columns={'ret_vw' : 'olq_12'})
display(olq_12_return)


all_factors = pd.merge(all_factors, ol_return, how='left', on='date')
all_factors = pd.merge(all_factors, olq_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, olq_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, olq_12_return, how='left', on='date')


,ol
date,
1967-01,0.024990
1967-02,0.006757
1967-03,-0.029790
1967-04,0.011214
1967-05,0.007906
...,...
2020-08,0.013624
2020-09,0.015926
2020-10,0.042139


,olq_1
date,
1973-01,0.008499
1973-02,-0.000239
1973-03,-0.049735
1973-04,-0.010982
1973-05,-0.003692
...,...
2020-08,0.015576
2020-09,-0.007589
2020-10,0.015695


,olq_6
date,
1973-01,0.008499
1973-02,-0.003637
1973-03,-0.013407
1973-04,-0.003404
1973-05,0.001068
...,...
2020-08,-0.015042
2020-09,0.009400
2020-10,0.037536


,olq_12
date,
1973-01,0.008499
1973-02,-0.003637
1973-03,-0.013407
1973-04,-0.003404
1973-05,0.001068
...,...
2020-08,-0.025647
2020-09,0.023566
2020-10,0.047382


* ### Seasonality (r1a) 

In [34]:
#Seasonality: 1 month return
r1a_factor = pd.read_csv(intangible_factors_path/'portf_me_r1a_monthly_2020.csv')

r1a_factor['date'] = r1a_factor['year'].astype(str) + '/' + r1a_factor['month'].astype(str)
r1a_factor['date'] = pd.to_datetime(r1a_factor['date'])
r1a_factor['date'] = r1a_factor['date'].dt.strftime('%Y-%m')
r1a_factor = r1a_factor.drop(columns=['year','month']).set_index('date')
r1a_factor['ret_vw'] =  r1a_factor['ret_vw']/100
r1a_factor.head(20)

r1a_return = r1a_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_r1a']==5),'ret_vw']- x.loc[(x['rank_r1a']==1),'ret_vw'])
r1a_return = pd.DataFrame(r1a_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
r1a_return = r1a_return.rename(columns={'ret_vw' : 'r1a'})
display(r1a_return)

all_factors = pd.merge(all_factors, r1a_return, how='left', on='date')

,r1a
date,
1967-01,0.026966
1967-02,0.015352
1967-03,-0.001766
1967-04,-0.004467
1967-05,0.003156
...,...
2020-08,0.023931
2020-09,0.007398
2020-10,0.005728


* ### (quarterly) research & development to market (rdm) 

In [35]:
#R&D to market: 1 month return
rdm_factor = pd.read_csv(intangible_factors_path/'portf_me_rdm_monthly_2020.csv')

rdm_factor['date'] = rdm_factor['year'].astype(str) + '/' + rdm_factor['month'].astype(str)
rdm_factor['date'] = pd.to_datetime(rdm_factor['date'])
rdm_factor['date'] = rdm_factor['date'].dt.strftime('%Y-%m')
rdm_factor = rdm_factor.drop(columns=['year','month']).set_index('date')
rdm_factor['ret_vw'] =  rdm_factor['ret_vw']/100
rdm_factor.head(20)

rdm_return = rdm_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_RDM']==5),'ret_vw']- x.loc[(x['rank_RDM']==1),'ret_vw'])
rdm_return = pd.DataFrame(rdm_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
rdm_return = rdm_return.rename(columns={'ret_vw' : 'rdm'})
display(rdm_return)

all_factors = pd.merge(all_factors, rdm_return, how='left', on='date')

,rdm
date,
1976-07,0.015938
1976-08,-0.016570
1976-09,0.002521
1976-10,-0.029946
1976-11,0.006481
...,...
2020-08,-0.036049
2020-09,0.012452
2020-10,-0.031644


* ### Industry adjusted real estate ratio (rer) 

In [36]:
#R&D to market: 1 month return
rer_factor = pd.read_csv(intangible_factors_path/'portf_me_rer_monthly_2020.csv')

rer_factor['date'] = rer_factor['year'].astype(str) + '/' + rer_factor['month'].astype(str)
rer_factor['date'] = pd.to_datetime(rer_factor['date'])
rer_factor['date'] = rer_factor['date'].dt.strftime('%Y-%m')
rer_factor = rer_factor.drop(columns=['year','month']).set_index('date')
rer_factor['ret_vw'] =  rer_factor['ret_vw']/100
rer_factor.head(20)

rer_return = rer_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_RER']==5),'ret_vw']- x.loc[(x['rank_RER']==1),'ret_vw'])
rer_return = pd.DataFrame(rer_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
rer_return = rer_return.rename(columns={'ret_vw' : 'rer'})
display(rer_return)

all_factors = pd.merge(all_factors, rer_return, how='left', on='date')

,rer
date,
1970-07,0.014922
1970-08,-0.026885
1970-09,-0.017252
1970-10,0.011190
1970-11,-0.012989
...,...
2020-08,-0.003773
2020-09,-0.001240
2020-10,-0.004754


# Investment Factors

* ### Abnormal corporate investment (aci) 

In [37]:
investment_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Inv_me_monthly'

#abnormal corporate investment (aci)
aci_factor = pd.read_csv(investment_factors_path/'portf_me_aci_monthly_2020.csv')

aci_factor['date'] = aci_factor['year'].astype(str) + '/' + aci_factor['month'].astype(str)
aci_factor['date'] = pd.to_datetime(aci_factor['date'])
aci_factor['date'] = aci_factor['date'].dt.strftime('%Y-%m')
aci_factor = aci_factor.drop(columns=['year','month']).set_index('date')
aci_factor['ret_vw'] =  aci_factor['ret_vw']/100
aci_factor.head(20)

aci_return = aci_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ACI']==5),'ret_vw']- x.loc[(x['rank_ACI']==1),'ret_vw'])
aci_return = pd.DataFrame(aci_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
aci_return = aci_return.rename(columns={'ret_vw' : 'aci'})
display(aci_return)

all_factors = pd.merge(all_factors, aci_return, how='left', on='date')

,aci
date,
1967-01,-0.005697
1967-02,-0.010051
1967-03,0.002239
1967-04,0.004037
1967-05,-0.016319
...,...
2020-08,-0.010580
2020-09,-0.012887
2020-10,0.007652


* ### composite equity issuance

In [38]:
investment_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Inv_me_monthly'

#composite equity issuance (cei)
cei_factor = pd.read_csv(investment_factors_path/'portf_me_cei_monthly_2020.csv')

cei_factor['date'] = cei_factor['year'].astype(str) + '/' + cei_factor['month'].astype(str)
cei_factor['date'] = pd.to_datetime(cei_factor['date'])
cei_factor['date'] = cei_factor['date'].dt.strftime('%Y-%m')
cei_factor = cei_factor.drop(columns=['year','month']).set_index('date')
cei_factor['ret_vw'] =  cei_factor['ret_vw']/100
cei_factor.head(20)

cei_return = cei_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CEI']==5),'ret_vw']- x.loc[(x['rank_CEI']==1),'ret_vw'])
cei_return = pd.DataFrame(cei_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cei_return = cei_return.rename(columns={'ret_vw' : 'cei'})
display(cei_return)

all_factors = pd.merge(all_factors, cei_return, how='left', on='date')

,cei
date,
1967-01,0.012356
1967-02,0.022170
1967-03,-0.005609
1967-04,-0.013238
1967-05,0.015816
...,...
2020-08,-0.061121
2020-09,-0.001468
2020-10,-0.020390


* ### Discretionary accruals (dac)

In [39]:
# discretionary accruals 
dac_factor = pd.read_csv(investment_factors_path/'portf_me_dac_monthly_2020.csv')

dac_factor['date'] = dac_factor['year'].astype(str) + '/' + dac_factor['month'].astype(str)
dac_factor['date'] = pd.to_datetime(dac_factor['date'])
dac_factor['date'] = dac_factor['date'].dt.strftime('%Y-%m')
dac_factor = dac_factor.drop(columns=['year','month']).set_index('date')
dac_factor['ret_vw'] =  dac_factor['ret_vw']/100
dac_factor.head(20)

dac_return = dac_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_DAC']==5),'ret_vw']- x.loc[(x['rank_DAC']==1),'ret_vw'])
dac_return = pd.DataFrame(dac_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
dac_return = dac_return.rename(columns={'ret_vw' : 'dac'})
display(dac_return)

all_factors = pd.merge(all_factors, dac_return, how='left', on='date')

,dac
date,
1967-01,0.015716
1967-02,0.000746
1967-03,0.011596
1967-04,0.006389
1967-05,0.007507
...,...
2020-08,-0.012834
2020-09,0.028602
2020-10,-0.008173


* ### Change in current operating assets (dcoa)

In [40]:
# Change in current operating assets 
dcoa_factor = pd.read_csv(investment_factors_path/'portf_me_dcoa_monthly_2020.csv')

dcoa_factor['date'] = dcoa_factor['year'].astype(str) + '/' + dcoa_factor['month'].astype(str)
dcoa_factor['date'] = pd.to_datetime(dcoa_factor['date'])
dcoa_factor['date'] = dcoa_factor['date'].dt.strftime('%Y-%m')
dcoa_factor = dcoa_factor.drop(columns=['year','month']).set_index('date')
dcoa_factor['ret_vw'] =  dcoa_factor['ret_vw']/100
dcoa_factor.head(20)

dcoa_return = dcoa_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dCOA']==5),'ret_vw']- x.loc[(x['rank_dCOA']==1),'ret_vw'])
dcoa_return = pd.DataFrame(dcoa_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
dcoa_return = dcoa_return.rename(columns={'ret_vw' : 'dcoa'})
display(dcoa_return)

all_factors = pd.merge(all_factors, dcoa_return, how='left', on='date')

,dcoa
date,
1967-01,0.036494
1967-02,0.005662
1967-03,0.009310
1967-04,0.031144
1967-05,-0.008463
...,...
2020-08,0.040955
2020-09,-0.001988
2020-10,-0.018907


* ### change in net financial assets (dfin)

In [41]:
# Change in current operating assets 
dfin_factor = pd.read_csv(investment_factors_path/'portf_me_dfin_monthly_2020.csv')

dfin_factor['date'] = dfin_factor['year'].astype(str) + '/' + dfin_factor['month'].astype(str)
dfin_factor['date'] = pd.to_datetime(dfin_factor['date'])
dfin_factor['date'] = dfin_factor['date'].dt.strftime('%Y-%m')
dfin_factor = dfin_factor.drop(columns=['year','month']).set_index('date')
dfin_factor['ret_vw'] =  dfin_factor['ret_vw']/100
dfin_factor.head(20)

dfin_return = dfin_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dFIN']==5),'ret_vw']- x.loc[(x['rank_dFIN']==1),'ret_vw'])
dfin_return = pd.DataFrame(dfin_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
dfin_return = dfin_return.rename(columns={'ret_vw' : 'dfin'})
display(dfin_return)

all_factors = pd.merge(all_factors, dfin_return, how='left', on='date')

,dfin
date,
1967-01,-0.002723
1967-02,-0.011355
1967-03,-0.005749
1967-04,-0.017226
1967-05,0.010300
...,...
2020-08,-0.022361
2020-09,0.011039
2020-10,0.000854


* ### Change in financial liabilities (dfnl)

In [42]:
# Change in current operating assets 
dfnl_factor = pd.read_csv(investment_factors_path/'portf_me_dfnl_monthly_2020.csv')

dfnl_factor['date'] = dfnl_factor['year'].astype(str) + '/' + dfnl_factor['month'].astype(str)
dfnl_factor['date'] = pd.to_datetime(dfnl_factor['date'])
dfnl_factor['date'] = dfnl_factor['date'].dt.strftime('%Y-%m')
dfnl_factor = dfnl_factor.drop(columns=['year','month']).set_index('date')
dfnl_factor['ret_vw'] =  dfnl_factor['ret_vw']/100
dfnl_factor.head(20)

dfnl_return = dfnl_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dFNL']==5),'ret_vw']- x.loc[(x['rank_dFNL']==1),'ret_vw'])
dfnl_return = pd.DataFrame(dfnl_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
dfnl_return = dfnl_return.rename(columns={'ret_vw' : 'dfnl'})
display(dfnl_return)

all_factors = pd.merge(all_factors, dfnl_return, how='left', on='date')

,dfnl
date,
1967-01,0.006183
1967-02,0.008046
1967-03,0.001422
1967-04,0.011826
1967-05,-0.008489
...,...
2020-08,-0.000075
2020-09,0.001752
2020-10,0.007093


* ### % change in investment - % change in industry investment (dii)


In [43]:
# Change in current operating assets 
dii_factor = pd.read_csv(investment_factors_path/'portf_me_dii_monthly_2020.csv')

dii_factor['date'] = dii_factor['year'].astype(str) + '/' + dii_factor['month'].astype(str)
dii_factor['date'] = pd.to_datetime(dii_factor['date'])
dii_factor['date'] = dii_factor['date'].dt.strftime('%Y-%m')
dii_factor = dii_factor.drop(columns=['year','month']).set_index('date')
dii_factor['ret_vw'] =  dii_factor['ret_vw']/100
dii_factor.head(20)

dii_return = dii_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dII']==5),'ret_vw']- x.loc[(x['rank_dII']==1),'ret_vw'])
dii_return = pd.DataFrame(dii_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
dii_return = dii_return.rename(columns={'ret_vw' : 'dii'})
display(dii_return)

all_factors = pd.merge(all_factors, dii_return, how='left', on='date')

,dii
date,
1967-01,-0.010081
1967-02,0.008226
1967-03,-0.007409
1967-04,0.016117
1967-05,-0.003187
...,...
2020-08,0.002080
2020-09,-0.017101
2020-10,0.008052


* ### Changes in long-term net operating assets (dlno)

In [44]:
# Change in current operating assets 
dlno_factor = pd.read_csv(investment_factors_path/'portf_me_dlno_monthly_2020.csv')

dlno_factor['date'] = dlno_factor['year'].astype(str) + '/' + dlno_factor['month'].astype(str)
dlno_factor['date'] = pd.to_datetime(dlno_factor['date'])
dlno_factor['date'] = dlno_factor['date'].dt.strftime('%Y-%m')
dlno_factor = dlno_factor.drop(columns=['year','month']).set_index('date')
dlno_factor['ret_vw'] =  dlno_factor['ret_vw']/100
dlno_factor.head(20)

dlno_return = dlno_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dLNO']==5),'ret_vw']- x.loc[(x['rank_dLNO']==1),'ret_vw'])
dlno_return = pd.DataFrame(dlno_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
dlno_return = dlno_return.rename(columns={'ret_vw' : 'dlno'})
display(dlno_return)

all_factors = pd.merge(all_factors, dlno_return, how='left', on='date')

,dlno
date,
1967-01,0.004099
1967-02,0.006500
1967-03,-0.014447
1967-04,-0.000684
1967-05,-0.024359
...,...
2020-08,0.051560
2020-09,-0.019630
2020-10,-0.009809


* ### change in non-current operating assets (dnca)

In [45]:
# Change in non-current operating assets 
dnca_factor = pd.read_csv(investment_factors_path/'portf_me_dnca_monthly_2020.csv')

dnca_factor['date'] = dnca_factor['year'].astype(str) + '/' + dnca_factor['month'].astype(str)
dnca_factor['date'] = pd.to_datetime(dnca_factor['date'])
dnca_factor['date'] = dnca_factor['date'].dt.strftime('%Y-%m')
dnca_factor = dnca_factor.drop(columns=['year','month']).set_index('date')
dnca_factor['ret_vw'] =  dnca_factor['ret_vw']/100
dnca_factor.head(20)

dnca_return = dnca_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dNCA']==5),'ret_vw']- x.loc[(x['rank_dNCA']==1),'ret_vw'])
dnca_return = pd.DataFrame(dnca_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
dnca_return = dnca_return.rename(columns={'ret_vw' : 'dnca'})
display(dnca_return)

all_factors = pd.merge(all_factors, dnca_return, how='left', on='date')

,dnca
date,
1967-01,0.000162
1967-02,0.015688
1967-03,-0.002989
1967-04,0.011019
1967-05,-0.025332
...,...
2020-08,0.049581
2020-09,-0.008890
2020-10,-0.011023


* ### change in net non-current operating assets (dnco)

In [46]:
# Change in net non-current operating assets 
dnco_factor = pd.read_csv(investment_factors_path/'portf_me_dnco_monthly_2020.csv')

dnco_factor['date'] = dnco_factor['year'].astype(str) + '/' + dnco_factor['month'].astype(str)
dnco_factor['date'] = pd.to_datetime(dnco_factor['date'])
dnco_factor['date'] = dnco_factor['date'].dt.strftime('%Y-%m')
dnco_factor = dnco_factor.drop(columns=['year','month']).set_index('date')
dnco_factor['ret_vw'] =  dnco_factor['ret_vw']/100
dnco_factor.head(20)

dnco_return = dnco_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dNCO']==5),'ret_vw']- x.loc[(x['rank_dNCO']==1),'ret_vw'])
dnco_return = pd.DataFrame(dnco_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
dnco_return = dnco_return.rename(columns={'ret_vw' : 'dnco'})
display(dnco_return)

all_factors = pd.merge(all_factors, dnco_return, how='left', on='date')

,dnco
date,
1967-01,-0.003073
1967-02,0.008617
1967-03,-0.004423
1967-04,0.019386
1967-05,-0.024643
...,...
2020-08,0.048873
2020-09,-0.003686
2020-10,-0.017689


* ### Changes in PPE and Inventory to assets (dpia)

In [47]:
# Change in net non-current operating assets 
dpia_factor = pd.read_csv(investment_factors_path/'portf_me_dpia_monthly_2020.csv')

dpia_factor['date'] = dpia_factor['year'].astype(str) + '/' + dpia_factor['month'].astype(str)
dpia_factor['date'] = pd.to_datetime(dpia_factor['date'])
dpia_factor['date'] = dpia_factor['date'].dt.strftime('%Y-%m')
dpia_factor = dpia_factor.drop(columns=['year','month']).set_index('date')
dpia_factor['ret_vw'] =  dpia_factor['ret_vw']/100
dpia_factor.head(20)

dpia_return = dpia_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dPIA']==5),'ret_vw']- x.loc[(x['rank_dPIA']==1),'ret_vw'])
dpia_return = pd.DataFrame(dpia_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
dpia_return = dpia_return.rename(columns={'ret_vw' : 'dpia'})
display(dpia_return)

all_factors = pd.merge(all_factors, dpia_return, how='left', on='date')

,dpia
date,
1967-01,0.018589
1967-02,0.007801
1967-03,0.011787
1967-04,0.018622
1967-05,-0.027603
...,...
2020-08,0.038110
2020-09,-0.039967
2020-10,0.021893


* ### (quarterly)Investment to assets HXZ (iaq & ia) 

In [48]:
# investment to assets
ia_factor = pd.read_csv(investment_factors_path/'portf_me_ia_monthly_2020.csv')

ia_factor['date'] = ia_factor['year'].astype(str) + '/' + ia_factor['month'].astype(str)
ia_factor['date'] = pd.to_datetime(ia_factor['date'])
ia_factor['date'] = ia_factor['date'].dt.strftime('%Y-%m')
ia_factor = ia_factor.drop(columns=['year','month']).set_index('date')
ia_factor['ret_vw'] =  ia_factor['ret_vw']/100
ia_factor.head(20)

ia_return = ia_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IA']==5),'ret_vw']- x.loc[(x['rank_IA']==1),'ret_vw'])
ia_return = pd.DataFrame(ia_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ia_return = ia_return.rename(columns={'ret_vw' : 'ia'})
display(ia_return)

# quarterly investments to assets: 6 month return
iaq_6_factor = pd.read_csv(investment_factors_path/'portf_me_iaq_6_monthly_2020.csv')

iaq_6_factor['date'] = iaq_6_factor['year'].astype(str) + '/' + iaq_6_factor['month'].astype(str)
iaq_6_factor['date'] = pd.to_datetime(iaq_6_factor['date'])
iaq_6_factor['date'] = iaq_6_factor['date'].dt.strftime('%Y-%m')
iaq_6_factor = iaq_6_factor.drop(columns=['year','month']).set_index('date')
iaq_6_factor['ret_vw'] =  iaq_6_factor['ret_vw']/100
iaq_6_factor.head(20)

iaq_6_return = iaq_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IAq_6']==5),'ret_vw']- x.loc[(x['rank_IAq_6']==1),'ret_vw'])
iaq_6_return = pd.DataFrame(iaq_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
iaq_6_return = iaq_6_return.rename(columns={'ret_vw' : 'iaq_6'})
display(iaq_6_return)


# quarterly investments to assets: 12 month return
iaq_12_factor = pd.read_csv(investment_factors_path/'portf_me_iaq_12_monthly_2020.csv')

iaq_12_factor['date'] = iaq_12_factor['year'].astype(str) + '/' + iaq_12_factor['month'].astype(str)
iaq_12_factor['date'] = pd.to_datetime(iaq_12_factor['date'])
iaq_12_factor['date'] = iaq_12_factor['date'].dt.strftime('%Y-%m')
iaq_12_factor = iaq_12_factor.drop(columns=['year','month']).set_index('date')
iaq_12_factor['ret_vw'] =  iaq_12_factor['ret_vw']/100
iaq_12_factor.head(20)

iaq_12_return = iaq_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IAq_12']==5),'ret_vw']- x.loc[(x['rank_IAq_12']==1),'ret_vw'])
iaq_12_return = pd.DataFrame(iaq_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
iaq_12_return = iaq_12_return.rename(columns={'ret_vw' : 'iaq_12'})
display(iaq_12_return)


all_factors = pd.merge(all_factors, ia_return, how='left', on='date')
all_factors = pd.merge(all_factors, iaq_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, iaq_12_return, how='left', on='date')

,ia
date,
1967-01,0.030608
1967-02,0.013548
1967-03,0.016992
1967-04,0.028605
1967-05,-0.016747
...,...
2020-08,0.018656
2020-09,0.002767
2020-10,0.015383


,iaq_6
date,
1973-01,-0.007131
1973-02,-0.033488
1973-03,-0.019927
1973-04,-0.045216
1973-05,-0.036429
...,...
2020-08,0.022733
2020-09,0.020000
2020-10,0.003469


,iaq_12
date,
1973-01,-0.007131
1973-02,-0.033488
1973-03,-0.019927
1973-04,-0.045216
1973-05,-0.036429
...,...
2020-08,0.022639
2020-09,0.011710
2020-10,-0.000237


* ### investment growth one year (ig) & investment growth two year (ig2) 

In [49]:
# investment growth one year
ig_factor = pd.read_csv(investment_factors_path/'portf_me_ig_monthly_2020.csv')

ig_factor['date'] = ig_factor['year'].astype(str) + '/' + ig_factor['month'].astype(str)
ig_factor['date'] = pd.to_datetime(ig_factor['date'])
ig_factor['date'] = ig_factor['date'].dt.strftime('%Y-%m')
ig_factor = ig_factor.drop(columns=['year','month']).set_index('date')
ig_factor['ret_vw'] =  ig_factor['ret_vw']/100
ig_factor.head(20)

ig_return = ig_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IG']==5),'ret_vw']- x.loc[(x['rank_IG']==1),'ret_vw'])
ig_return = pd.DataFrame(ig_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ig_return = ig_return.rename(columns={'ret_vw' : 'ig'})
display(ig_return)


#investment growth two year
ig2_factor = pd.read_csv(investment_factors_path/'portf_me_ig2_monthly_2020.csv')

ig2_factor['date'] = ig2_factor['year'].astype(str) + '/' + ig2_factor['month'].astype(str)
ig2_factor['date'] = pd.to_datetime(ig2_factor['date'])
ig2_factor['date'] = ig2_factor['date'].dt.strftime('%Y-%m')
ig2_factor = ig2_factor.drop(columns=['year','month']).set_index('date')
ig2_factor['ret_vw'] =  ig2_factor['ret_vw']/100
ig2_factor.head(20)

ig2_return = ig2_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IG2']==5),'ret_vw']- x.loc[(x['rank_IG2']==1),'ret_vw'])
ig2_return = pd.DataFrame(ig2_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ig2_return = ig2_return.rename(columns={'ret_vw' : 'ig2'})
display(ig2_return)

all_factors = pd.merge(all_factors, ig_return, how='left', on='date')
all_factors = pd.merge(all_factors, ig2_return, how='left', on='date')

,ig
date,
1967-01,0.004243
1967-02,0.003681
1967-03,-0.004942
1967-04,-0.005348
1967-05,0.002612
...,...
2020-08,-0.016675
2020-09,0.008549
2020-10,0.006899


,ig2
date,
1967-01,-0.012312
1967-02,0.002269
1967-03,0.016683
1967-04,0.007525
1967-05,-0.005066
...,...
2020-08,0.003352
2020-09,-0.025025
2020-10,0.010719


* ### Inventory changes (ivc) & inventory growth (ivg)

In [50]:
#inventory changes 
ivc_factor = pd.read_csv(investment_factors_path/'portf_me_ivc_monthly_2020.csv')

ivc_factor['date'] = ivc_factor['year'].astype(str) + '/' + ivc_factor['month'].astype(str)
ivc_factor['date'] = pd.to_datetime(ivc_factor['date'])
ivc_factor['date'] = ivc_factor['date'].dt.strftime('%Y-%m')
ivc_factor = ivc_factor.drop(columns=['year','month']).set_index('date')
ivc_factor['ret_vw'] =  ivc_factor['ret_vw']/100
ivc_factor.head(20)

ivc_return = ivc_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IVC']==5),'ret_vw']- x.loc[(x['rank_IVC']==1),'ret_vw'])
ivc_return = pd.DataFrame(ivc_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ivc_return = ivc_return.rename(columns={'ret_vw' : 'ivc'})
display(ivc_return)


#inventory growth
ivg_factor = pd.read_csv(investment_factors_path/'portf_me_ivg_monthly_2020.csv')

ivg_factor['date'] = ivg_factor['year'].astype(str) + '/' + ivg_factor['month'].astype(str)
ivg_factor['date'] = pd.to_datetime(ivg_factor['date'])
ivg_factor['date'] = ivg_factor['date'].dt.strftime('%Y-%m')
ivg_factor = ivg_factor.drop(columns=['year','month']).set_index('date')
ivg_factor['ret_vw'] =  ivg_factor['ret_vw']/100
ivg_factor.head(20)

ivg_return = ivg_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IVG']==5),'ret_vw']- x.loc[(x['rank_IVG']==1),'ret_vw'])
ivg_return = pd.DataFrame(ivg_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ivg_return = ivg_return.rename(columns={'ret_vw' : 'ivg'})
display(ivg_return)

all_factors = pd.merge(all_factors, ivc_return, how='left', on='date')
all_factors = pd.merge(all_factors, ivg_return, how='left', on='date')


,ivc
date,
1967-01,0.019774
1967-02,0.002251
1967-03,0.030237
1967-04,0.023848
1967-05,-0.011452
...,...
2020-08,0.035446
2020-09,-0.011229
2020-10,-0.028978


,ivg
date,
1967-01,0.017756
1967-02,0.006584
1967-03,0.014098
1967-04,0.022797
1967-05,-0.018237
...,...
2020-08,0.008044
2020-09,0.006391
2020-10,-0.028516


* ### net debt financing (ndf)

In [51]:
#net debt financing
ndf_factor = pd.read_csv(investment_factors_path/'portf_me_ndf_monthly_2020.csv')

ndf_factor['date'] = ndf_factor['year'].astype(str) + '/' + ndf_factor['month'].astype(str)
ndf_factor['date'] = pd.to_datetime(ndf_factor['date'])
ndf_factor['date'] = ndf_factor['date'].dt.strftime('%Y-%m')
ndf_factor = ndf_factor.drop(columns=['year','month']).set_index('date')
ndf_factor['ret_vw'] =  ndf_factor['ret_vw']/100
ndf_factor.head(20)

ndf_return = ndf_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_NDF']==5),'ret_vw']- x.loc[(x['rank_NDF']==1),'ret_vw'])
ndf_return = pd.DataFrame(ndf_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ndf_return = ndf_return.rename(columns={'ret_vw' : 'ndf'})
display(ndf_return)

all_factors = pd.merge(all_factors, ndf_return, how='left', on='date')

,ndf
date,
1972-07,-0.002352
1972-08,0.006655
1972-09,0.016427
1972-10,-0.005453
1972-11,-0.003800
...,...
2020-08,0.031633
2020-09,0.003596
2020-10,-0.009424


* ### (changes) in net operating assets (dnoa & noa) 

In [52]:
#net operating assets
noa_factor = pd.read_csv(investment_factors_path/'portf_me_noa_monthly_2020.csv')

noa_factor['date'] = noa_factor['year'].astype(str) + '/' + noa_factor['month'].astype(str)
noa_factor['date'] = pd.to_datetime(noa_factor['date'])
noa_factor['date'] = noa_factor['date'].dt.strftime('%Y-%m')
noa_factor = noa_factor.drop(columns=['year','month']).set_index('date')
noa_factor['ret_vw'] =  noa_factor['ret_vw']/100
noa_factor.head(20)

noa_return = noa_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_NOA']==5),'ret_vw']- x.loc[(x['rank_NOA']==1),'ret_vw'])
noa_return = pd.DataFrame(noa_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
noa_return = noa_return.rename(columns={'ret_vw' : 'noa'})
display(noa_return)


#changes in net operating assets
dnoa_factor = pd.read_csv(investment_factors_path/'portf_me_dnoa_monthly_2020.csv')

dnoa_factor['date'] = dnoa_factor['year'].astype(str) + '/' + dnoa_factor['month'].astype(str)
dnoa_factor['date'] = pd.to_datetime(dnoa_factor['date'])
dnoa_factor['date'] = dnoa_factor['date'].dt.strftime('%Y-%m')
dnoa_factor = dnoa_factor.drop(columns=['year','month']).set_index('date')
dnoa_factor['ret_vw'] =  dnoa_factor['ret_vw']/100
dnoa_factor.head(20)

dnoa_return = dnoa_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dNOA']==5),'ret_vw']- x.loc[(x['rank_dNOA']==1),'ret_vw'])
dnoa_return = pd.DataFrame(dnoa_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
dnoa_return = dnoa_return.rename(columns={'ret_vw' : 'dnoa'})
display(dnoa_return)


all_factors = pd.merge(all_factors, noa_return, how='left', on='date')
all_factors = pd.merge(all_factors, dnoa_return, how='left', on='date')




,noa
date,
1967-01,0.014306
1967-02,0.003896
1967-03,-0.025404
1967-04,-0.011362
1967-05,-0.030024
...,...
2020-08,0.025631
2020-09,0.005195
2020-10,-0.012002


,dnoa
date,
1967-01,0.036507
1967-02,0.025635
1967-03,0.001776
1967-04,0.045095
1967-05,-0.013276
...,...
2020-08,0.022506
2020-09,0.008847
2020-10,0.000825


* ### changes in shares outstanding (nsi) 


In [53]:
#changes in shares outstanding
nsi_factor = pd.read_csv(investment_factors_path/'portf_me_nsi_monthly_2020.csv')

nsi_factor['date'] = nsi_factor['year'].astype(str) + '/' + nsi_factor['month'].astype(str)
nsi_factor['date'] = pd.to_datetime(nsi_factor['date'])
nsi_factor['date'] = nsi_factor['date'].dt.strftime('%Y-%m')
nsi_factor = nsi_factor.drop(columns=['year','month']).set_index('date')
nsi_factor['ret_vw'] =  nsi_factor['ret_vw']/100
nsi_factor.head(20)

nsi_return = nsi_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_NSI']==5),'ret_vw']- x.loc[(x['rank_NSI']==1),'ret_vw'])
nsi_return = pd.DataFrame(nsi_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
nsi_return = nsi_return.rename(columns={'ret_vw' : 'nsi'})
display(nsi_return)


all_factors = pd.merge(all_factors, nsi_return, how='left', on='date')

,nsi
date,
1967-01,0.008510
1967-02,-0.006571
1967-03,0.003470
1967-04,-0.010508
1967-05,-0.020660
...,...
2020-08,-0.021126
2020-09,0.006679
2020-10,-0.010471


* ### Operating accruals (oa)

In [54]:
#operating accruals 
oa_factor = pd.read_csv(investment_factors_path/'portf_me_oa_monthly_2020.csv')

oa_factor['date'] = oa_factor['year'].astype(str) + '/' + oa_factor['month'].astype(str)
oa_factor['date'] = pd.to_datetime(oa_factor['date'])
oa_factor['date'] = oa_factor['date'].dt.strftime('%Y-%m')
oa_factor = oa_factor.drop(columns=['year','month']).set_index('date')
oa_factor['ret_vw'] =  oa_factor['ret_vw']/100
oa_factor.head(20)

oa_return = oa_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_OA']==5),'ret_vw']- x.loc[(x['rank_OA']==1),'ret_vw'])
oa_return = pd.DataFrame(oa_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
oa_return = oa_return.rename(columns={'ret_vw' : 'oa'})
display(oa_return)

all_factors = pd.merge(all_factors, oa_return, how='left', on='date')

,oa
date,
1967-01,0.027734
1967-02,-0.007963
1967-03,0.019462
1967-04,0.025077
1967-05,-0.013049
...,...
2020-08,-0.005117
2020-09,0.044161
2020-10,-0.007274


* ### percent discretionary accruals (pda) 

In [55]:
#percent discretionary accruals 
pda_factor = pd.read_csv(investment_factors_path/'portf_me_pda_monthly_2020.csv')

pda_factor['date'] = pda_factor['year'].astype(str) + '/' + pda_factor['month'].astype(str)
pda_factor['date'] = pd.to_datetime(pda_factor['date'])
pda_factor['date'] = pda_factor['date'].dt.strftime('%Y-%m')
pda_factor = pda_factor.drop(columns=['year','month']).set_index('date')
pda_factor['ret_vw'] =  pda_factor['ret_vw']/100
pda_factor.head(20)

pda_return = pda_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_PDA']==5),'ret_vw']- x.loc[(x['rank_PDA']==1),'ret_vw'])
pda_return = pd.DataFrame(pda_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
pda_return = pda_return.rename(columns={'ret_vw' : 'pda'})
display(pda_return)

all_factors = pd.merge(all_factors, pda_return, how='left', on='date')

,pda
date,
1967-01,0.018099
1967-02,-0.014737
1967-03,0.016557
1967-04,0.006637
1967-05,-0.006737
...,...
2020-08,-0.038015
2020-09,0.020307
2020-10,-0.008187


* ### percent operating accruals (poa) 

In [56]:
#percent operating accruals 
poa_factor = pd.read_csv(investment_factors_path/'portf_me_poa_monthly_2020.csv')

poa_factor['date'] = poa_factor['year'].astype(str) + '/' + poa_factor['month'].astype(str)
poa_factor['date'] = pd.to_datetime(poa_factor['date'])
poa_factor['date'] = poa_factor['date'].dt.strftime('%Y-%m')
poa_factor = poa_factor.drop(columns=['year','month']).set_index('date')
poa_factor['ret_vw'] =  poa_factor['ret_vw']/100
poa_factor.head(20)

poa_return = poa_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_POA']==5),'ret_vw']- x.loc[(x['rank_POA']==1),'ret_vw'])
poa_return = pd.DataFrame(poa_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
poa_return = poa_return.rename(columns={'ret_vw' : 'poa'})
display(poa_return)

all_factors = pd.merge(all_factors, poa_return, how='left', on='date')

,poa
date,
1967-01,0.010608
1967-02,-0.011641
1967-03,0.017521
1967-04,0.029612
1967-05,-0.018026
...,...
2020-08,-0.042876
2020-09,0.040919
2020-10,-0.006829


* ### percent total accruals (pta) 


In [57]:
#percent operating accruals 
pta_factor = pd.read_csv(investment_factors_path/'portf_me_pta_monthly_2020.csv')

pta_factor['date'] = pta_factor['year'].astype(str) + '/' + pta_factor['month'].astype(str)
pta_factor['date'] = pd.to_datetime(pta_factor['date'])
pta_factor['date'] = pta_factor['date'].dt.strftime('%Y-%m')
pta_factor = pta_factor.drop(columns=['year','month']).set_index('date')
pta_factor['ret_vw'] =  pta_factor['ret_vw']/100
pta_factor.head(20)

pta_return = pta_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_PTA']==5),'ret_vw']- x.loc[(x['rank_PTA']==1),'ret_vw'])
pta_return = pd.DataFrame(pta_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
pta_return = poa_return.rename(columns={'ret_vw' : 'pta'})
display(pta_return)

all_factors = pd.merge(all_factors, pta_return, how='left', on='date')

,poa
date,
1967-01,0.010608
1967-02,-0.011641
1967-03,0.017521
1967-04,0.029612
1967-05,-0.018026
...,...
2020-08,-0.042876
2020-09,0.040919
2020-10,-0.006829


* ### total accruals (ta)

In [58]:
#percent operating accruals 
ta_factor = pd.read_csv(investment_factors_path/'portf_me_ta_monthly_2020.csv')

ta_factor['date'] = ta_factor['year'].astype(str) + '/' + ta_factor['month'].astype(str)
ta_factor['date'] = pd.to_datetime(ta_factor['date'])
ta_factor['date'] = ta_factor['date'].dt.strftime('%Y-%m')
ta_factor = ta_factor.drop(columns=['year','month']).set_index('date')
ta_factor['ret_vw'] =  ta_factor['ret_vw']/100
ta_factor.head(20)

ta_return = ta_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_TA']==5),'ret_vw']- x.loc[(x['rank_TA']==1),'ret_vw'])
ta_return = pd.DataFrame(ta_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ta_return = ta_return.rename(columns={'ret_vw' : 'ta'})
display(ta_return)

all_factors = pd.merge(all_factors, ta_return, how='left', on='date')

,ta
date,
1967-01,-0.001245
1967-02,-0.007144
1967-03,0.008555
1967-04,0.015368
1967-05,-0.018618
...,...
2020-08,-0.001280
2020-09,0.020368
2020-10,-0.000454


# Momentum factors

* ### abnormal returns around earnings announcement (abr) 

In [59]:
momentum_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Mom_me_monthly'

#abnormal returns around earnings announcement: 1 month return
abr_1_factor = pd.read_csv(momentum_factors_path/'portf_me_abr_1_monthly_2020.csv')

abr_1_factor['date'] = abr_1_factor['year'].astype(str) + '/' + abr_1_factor['month'].astype(str)
abr_1_factor['date'] = pd.to_datetime(abr_1_factor['date'])
abr_1_factor['date'] = abr_1_factor['date'].dt.strftime('%Y-%m')
abr_1_factor = abr_1_factor.drop(columns=['year','month']).set_index('date')
abr_1_factor['ret_vw'] =  abr_1_factor['ret_vw']/100
abr_1_factor.head(20)

abr_1_return = abr_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_Abr_1']==5),'ret_vw']- x.loc[(x['rank_Abr_1']==1),'ret_vw'])
abr_1_return = pd.DataFrame(abr_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
abr_1_return = abr_1_return.rename(columns={'ret_vw' : 'abr_1'})
display(abr_1_return)


#abnormal returns around earnings announcement: 6 month return
abr_6_factor = pd.read_csv(momentum_factors_path/'portf_me_abr_6_monthly_2020.csv')

abr_6_factor['date'] = abr_6_factor['year'].astype(str) + '/' + abr_6_factor['month'].astype(str)
abr_6_factor['date'] = pd.to_datetime(abr_6_factor['date'])
abr_6_factor['date'] = abr_6_factor['date'].dt.strftime('%Y-%m')
abr_6_factor = abr_6_factor.drop(columns=['year','month']).set_index('date')
abr_6_factor['ret_vw'] =  abr_6_factor['ret_vw']/100
abr_6_factor.head(20)

abr_6_return = abr_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_Abr_6']==5),'ret_vw']- x.loc[(x['rank_Abr_6']==1),'ret_vw'])
abr_6_return = pd.DataFrame(abr_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
abr_6_return = abr_6_return.rename(columns={'ret_vw' : 'abr_6'})
display(abr_6_return)


#abnormal returns around earnings announcement: 12 month return
abr_12_factor = pd.read_csv(momentum_factors_path/'portf_me_abr_12_monthly_2020.csv')

abr_12_factor['date'] = abr_12_factor['year'].astype(str) + '/' + abr_12_factor['month'].astype(str)
abr_12_factor['date'] = pd.to_datetime(abr_12_factor['date'])
abr_12_factor['date'] = abr_12_factor['date'].dt.strftime('%Y-%m')
abr_12_factor = abr_12_factor.drop(columns=['year','month']).set_index('date')
abr_12_factor['ret_vw'] =  abr_12_factor['ret_vw']/100
abr_12_factor.head(20)

abr_12_return = abr_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_Abr_12']==5),'ret_vw']- x.loc[(x['rank_Abr_12']==1),'ret_vw'])
abr_12_return = pd.DataFrame(abr_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
abr_12_return = abr_12_return.rename(columns={'ret_vw' : 'abr_12'})
display(abr_12_return)


all_factors = pd.merge(all_factors, abr_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, abr_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, abr_12_return, how='left', on='date')

,abr_1
date,
1972-01,0.028829
1972-02,0.002608
1972-03,0.009985
1972-04,0.017279
1972-05,0.017980
...,...
2020-08,0.018227
2020-09,0.006131
2020-10,-0.003378


,abr_6
date,
1972-01,0.028829
1972-02,0.000163
1972-03,0.012506
1972-04,0.019416
1972-05,0.013071
...,...
2020-08,0.005280
2020-09,-0.011446
2020-10,-0.007752


,abr_12
date,
1972-01,0.028829
1972-02,0.000163
1972-03,0.012506
1972-04,0.019416
1972-05,0.013071
...,...
2020-08,0.013549
2020-09,-0.014598
2020-10,-0.004676


* ### customer industries momentum (cim) 

In [60]:
momentum_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/Mom_me_monthly'

#customer industries momentum: 1 month return
cim_1_factor = pd.read_csv(momentum_factors_path/'portf_me_cim_1_monthly_2020.csv')

cim_1_factor['date'] = cim_1_factor['year'].astype(str) + '/' + cim_1_factor['month'].astype(str)
cim_1_factor['date'] = pd.to_datetime(cim_1_factor['date'])
cim_1_factor['date'] = cim_1_factor['date'].dt.strftime('%Y-%m')
cim_1_factor = cim_1_factor.drop(columns=['year','month']).set_index('date')
cim_1_factor['ret_vw'] =  cim_1_factor['ret_vw']/100
cim_1_factor.head(20)

cim_1_return = cim_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CIM_1']==5),'ret_vw']- x.loc[(x['rank_CIM_1']==1),'ret_vw'])
cim_1_return = pd.DataFrame(cim_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cim_1_return = cim_1_return.rename(columns={'ret_vw' : 'cim_1'})
display(cim_1_return)

#customer industries momentum: 6 month return
cim_6_factor = pd.read_csv(momentum_factors_path/'portf_me_cim_6_monthly_2020.csv')

cim_6_factor['date'] = cim_6_factor['year'].astype(str) + '/' + cim_6_factor['month'].astype(str)
cim_6_factor['date'] = pd.to_datetime(cim_6_factor['date'])
cim_6_factor['date'] = cim_6_factor['date'].dt.strftime('%Y-%m')
cim_6_factor = cim_6_factor.drop(columns=['year','month']).set_index('date')
cim_6_factor['ret_vw'] =  cim_6_factor['ret_vw']/100
cim_6_factor.head(20)

cim_6_return = cim_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CIM_6']==5),'ret_vw']- x.loc[(x['rank_CIM_6']==1),'ret_vw'])
cim_6_return = pd.DataFrame(cim_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cim_6_return = cim_6_return.rename(columns={'ret_vw' : 'cim_6'})
display(cim_6_return)


#customer industries momentum: 12 month return
cim_12_factor = pd.read_csv(momentum_factors_path/'portf_me_cim_12_monthly_2020.csv')

cim_12_factor['date'] = cim_12_factor['year'].astype(str) + '/' + cim_12_factor['month'].astype(str)
cim_12_factor['date'] = pd.to_datetime(cim_12_factor['date'])
cim_12_factor['date'] = cim_12_factor['date'].dt.strftime('%Y-%m')
cim_12_factor = cim_12_factor.drop(columns=['year','month']).set_index('date')
cim_12_factor['ret_vw'] =  cim_12_factor['ret_vw']/100
cim_12_factor.head(20)

cim_12_return = cim_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CIM_12']==5),'ret_vw']- x.loc[(x['rank_CIM_12']==1),'ret_vw'])
cim_12_return = pd.DataFrame(cim_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cim_12_return = cim_12_return.rename(columns={'ret_vw' : 'cim_12'})
display(cim_12_return)


all_factors = pd.merge(all_factors, cim_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, cim_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, cim_12_return, how='left', on='date')

,cim_1
date,
1967-01,0.009557
1967-02,0.032716
1967-03,0.010608
1967-04,0.012126
1967-05,-0.018013
...,...
2020-08,0.030368
2020-09,-0.003720
2020-10,0.006640


,cim_6
date,
1967-01,0.009557
1967-02,0.027474
1967-03,0.010998
1967-04,0.012927
1967-05,-0.009724
...,...
2020-08,0.022998
2020-09,0.017347
2020-10,0.023735


,cim_12
date,
1967-01,0.009557
1967-02,0.027474
1967-03,0.010998
1967-04,0.012927
1967-05,-0.009724
...,...
2020-08,0.014351
2020-09,0.014844
2020-10,0.014132


* ### Customer Momentum (cm) 

In [61]:
# #customer momentum: 1 month return
# cm_1_factor = pd.read_csv(momentum_factors_path/'portf_me_cm_1_monthly_2020.csv')

# cm_1_factor['date'] = cm_1_factor['year'].astype(str) + '/' + cm_1_factor['month'].astype(str)
# cm_1_factor['date'] = pd.to_datetime(cm_1_factor['date'])
# cm_1_factor['date'] = cm_1_factor['date'].dt.strftime('%Y-%m')
# cm_1_factor = cm_1_factor.drop(columns=['year','month']).set_index('date')
# cm_1_factor['ret_vw'] =  cm_1_factor['ret_vw']/100
# cm_1_factor.head(20)

# cm_1_return = cm_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CM_1']==5),'ret_vw']- x.loc[(x['rank_CM_1']==1),'ret_vw'])
# cm_1_return = pd.DataFrame(cm_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
# cm_1_return = cm_1_return.rename(columns={'ret_vw' : 'cm_1'})
# display(cm_1_return)


# #customer momentum: 12 month return
# cm_12_factor = pd.read_csv(momentum_factors_path/'portf_me_cm_12_monthly_2020.csv')

# cm_12_factor['date'] = cm_12_factor['year'].astype(str) + '/' + cm_12_factor['month'].astype(str)
# cm_12_factor['date'] = pd.to_datetime(cm_12_factor['date'])
# cm_12_factor['date'] = cm_12_factor['date'].dt.strftime('%Y-%m')
# cm_12_factor = cm_12_factor.drop(columns=['year','month']).set_index('date')
# cm_12_factor['ret_vw'] =  cm_12_factor['ret_vw']/100
# cm_12_factor.head(20)

# cm_12_return = cm_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CM_12']==5),'ret_vw']- x.loc[(x['rank_CM_12']==1),'ret_vw'])
# cm_12_return = pd.DataFrame(cm_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
# cm_12_return = cm_1_return.rename(columns={'ret_vw' : 'cm_12'})
# display(cm_12_return)


# all_factors = pd.merge(all_factors, cm_1_return, how='left', on='date')
# all_factors = pd.merge(all_factors, cm_12_return, how='left', on='date')

* ### changes in analyst earnings forecast (def)

In [62]:
#changes in analyst earnings forecast: 1 month return
def_1_factor = pd.read_csv(momentum_factors_path/'portf_me_def_1_monthly_2020.csv')

def_1_factor['date'] = def_1_factor['year'].astype(str) + '/' + def_1_factor['month'].astype(str)
def_1_factor['date'] = pd.to_datetime(def_1_factor['date'])
def_1_factor['date'] = def_1_factor['date'].dt.strftime('%Y-%m')
def_1_factor = def_1_factor.drop(columns=['year','month']).set_index('date')
def_1_factor['ret_vw'] =  def_1_factor['ret_vw']/100
def_1_factor.head(20)

def_1_return = def_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dEF_1']==5),'ret_vw']- x.loc[(x['rank_dEF_1']==1),'ret_vw'])
def_1_return = pd.DataFrame(def_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
def_1_return = def_1_return.rename(columns={'ret_vw' : 'def_1'})
display(def_1_return)


#changes in analyst earnings forecast: 6 month return
def_6_factor = pd.read_csv(momentum_factors_path/'portf_me_def_6_monthly_2020.csv')

def_6_factor['date'] = def_6_factor['year'].astype(str) + '/' + def_6_factor['month'].astype(str)
def_6_factor['date'] = pd.to_datetime(def_6_factor['date'])
def_6_factor['date'] = def_6_factor['date'].dt.strftime('%Y-%m')
def_6_factor = def_6_factor.drop(columns=['year','month']).set_index('date')
def_6_factor['ret_vw'] =  def_6_factor['ret_vw']/100
def_6_factor.head(20)

def_6_return = def_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dEF_6']==5),'ret_vw']- x.loc[(x['rank_dEF_6']==1),'ret_vw'])
def_6_return = pd.DataFrame(def_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
def_6_return = def_6_return.rename(columns={'ret_vw' : 'def_6'})
display(def_6_return)


#changes in analyst earnings forecast: 12 month return
def_12_factor = pd.read_csv(momentum_factors_path/'portf_me_def_12_monthly_2020.csv')

def_12_factor['date'] = def_12_factor['year'].astype(str) + '/' + def_12_factor['month'].astype(str)
def_12_factor['date'] = pd.to_datetime(def_12_factor['date'])
def_12_factor['date'] = def_12_factor['date'].dt.strftime('%Y-%m')
def_12_factor = def_12_factor.drop(columns=['year','month']).set_index('date')
def_12_factor['ret_vw'] =  def_12_factor['ret_vw']/100
def_12_factor.head(20)

def_12_return = def_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_dEF_12']==5),'ret_vw']- x.loc[(x['rank_dEF_12']==1),'ret_vw'])
def_12_return = pd.DataFrame(def_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
def_12_return = def_12_return.rename(columns={'ret_vw' : 'def_12'})
display(def_12_return)


all_factors = pd.merge(all_factors, def_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, def_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, def_12_return, how='left', on='date')

,def_1
date,
1976-03,0.029399
1976-04,-0.000243
1976-05,-0.017329
1976-06,0.003480
1976-07,0.022206
...,...
2020-08,0.025110
2020-09,0.005520
2020-10,0.019709


,def_6
date,
1976-03,0.029399
1976-04,-0.001167
1976-05,-0.025827
1976-06,-0.014689
1976-07,0.003555
...,...
2020-08,-0.027549
2020-09,0.018382
2020-10,-0.012303


,def_12
date,
1976-03,0.029399
1976-04,-0.001167
1976-05,-0.025827
1976-06,-0.014689
1976-07,0.003555
...,...
2020-08,-0.026171
2020-09,0.012039
2020-10,-0.014891


* ### industry lead-lag effect in prior returns  (earnings surprise) (ilr & ile) 

In [63]:
#industry lead-lag effect in prior returns: 1 month return
ilr_1_factor = pd.read_csv(momentum_factors_path/'portf_me_ilr_1_monthly_2020.csv')

ilr_1_factor['date'] = ilr_1_factor['year'].astype(str) + '/' + ilr_1_factor['month'].astype(str)
ilr_1_factor['date'] = pd.to_datetime(ilr_1_factor['date'])
ilr_1_factor['date'] = ilr_1_factor['date'].dt.strftime('%Y-%m')
ilr_1_factor = ilr_1_factor.drop(columns=['year','month']).set_index('date')
ilr_1_factor['ret_vw'] =  ilr_1_factor['ret_vw']/100
ilr_1_factor.head(20)

ilr_1_return = ilr_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ILR_1']==5),'ret_vw']- x.loc[(x['rank_ILR_1']==1),'ret_vw'])
ilr_1_return = pd.DataFrame(ilr_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ilr_1_return = ilr_1_return.rename(columns={'ret_vw' : 'ilr_1'})
display(ilr_1_return)


#industry lead-lag effect in prior returns: 6 month return
ilr_6_factor = pd.read_csv(momentum_factors_path/'portf_me_ilr_6_monthly_2020.csv')

ilr_6_factor['date'] = ilr_6_factor['year'].astype(str) + '/' + ilr_6_factor['month'].astype(str)
ilr_6_factor['date'] = pd.to_datetime(ilr_6_factor['date'])
ilr_6_factor['date'] = ilr_6_factor['date'].dt.strftime('%Y-%m')
ilr_6_factor = ilr_6_factor.drop(columns=['year','month']).set_index('date')
ilr_6_factor['ret_vw'] =  ilr_6_factor['ret_vw']/100
ilr_6_factor.head(20)

ilr_6_return = ilr_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ILR_6']==5),'ret_vw']- x.loc[(x['rank_ILR_6']==1),'ret_vw'])
ilr_6_return = pd.DataFrame(ilr_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ilr_6_return = ilr_6_return.rename(columns={'ret_vw' : 'ilr_6'})
display(ilr_6_return)


#industry lead-lag effect in prior returns: 12 month return
ilr_12_factor = pd.read_csv(momentum_factors_path/'portf_me_ilr_12_monthly_2020.csv')

ilr_12_factor['date'] = ilr_12_factor['year'].astype(str) + '/' + ilr_12_factor['month'].astype(str)
ilr_12_factor['date'] = pd.to_datetime(ilr_12_factor['date'])
ilr_12_factor['date'] = ilr_12_factor['date'].dt.strftime('%Y-%m')
ilr_12_factor = ilr_12_factor.drop(columns=['year','month']).set_index('date')
ilr_12_factor['ret_vw'] =  ilr_12_factor['ret_vw']/100
ilr_12_factor.head(20)

ilr_12_return = ilr_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ILR_12']==5),'ret_vw']- x.loc[(x['rank_ILR_12']==1),'ret_vw'])
ilr_12_return = pd.DataFrame(ilr_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ilr_12_return = ilr_12_return.rename(columns={'ret_vw' : 'ilr_12'})
display(ilr_12_return)


#industry lead-lag effect in earnings surprise: 1 month return
ile_1_factor = pd.read_csv(momentum_factors_path/'portf_me_ile_1_monthly_2020.csv')

ile_1_factor['date'] = ile_1_factor['year'].astype(str) + '/' + ile_1_factor['month'].astype(str)
ile_1_factor['date'] = pd.to_datetime(ile_1_factor['date'])
ile_1_factor['date'] = ile_1_factor['date'].dt.strftime('%Y-%m')
ile_1_factor = ile_1_factor.drop(columns=['year','month']).set_index('date')
ile_1_factor['ret_vw'] =  ile_1_factor['ret_vw']/100
ile_1_factor.head(20)

ile_1_return = ile_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_ILE_1']==5),'ret_vw']- x.loc[(x['rank_ILE_1']==1),'ret_vw'])
ile_1_return = pd.DataFrame(ile_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ile_1_return = ile_1_return.rename(columns={'ret_vw' : 'ile_1'})
display(ile_1_return)

all_factors = pd.merge(all_factors, ilr_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, ilr_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, ilr_12_return, how='left', on='date')
all_factors = pd.merge(all_factors, ile_1_return, how='left', on='date')

,ilr_1
date,
1967-01,-0.016656
1967-02,-0.005693
1967-03,0.037497
1967-04,0.017584
1967-05,-0.027020
...,...
2020-08,-0.010460
2020-09,0.013154
2020-10,-0.003662


,ilr_6
date,
1967-01,-0.016656
1967-02,0.002271
1967-03,0.021319
1967-04,0.027462
1967-05,-0.011548
...,...
2020-08,0.007512
2020-09,0.010534
2020-10,0.001699


,ilr_12
date,
1967-01,-0.016656
1967-02,0.002271
1967-03,0.021319
1967-04,0.027462
1967-05,-0.011548
...,...
2020-08,0.013998
2020-09,0.006793
2020-10,-0.006919


,ile_1
date,
1967-01,-0.006804
1967-02,0.023860
1967-03,-0.012055
1967-04,-0.000151
1967-05,-0.023869
...,...
2020-08,-0.104245
2020-09,-0.006503
2020-10,0.010594


* ### Industry Momentum (im) 

In [64]:
#industry momentum: 1 month return
im_1_factor = pd.read_csv(momentum_factors_path/'portf_me_im_1_monthly_2020.csv')

im_1_factor['date'] = im_1_factor['year'].astype(str) + '/' + im_1_factor['month'].astype(str)
im_1_factor['date'] = pd.to_datetime(im_1_factor['date'])
im_1_factor['date'] = im_1_factor['date'].dt.strftime('%Y-%m')
im_1_factor = im_1_factor.drop(columns=['year','month']).set_index('date')
im_1_factor['ret_vw'] =  im_1_factor['ret_vw']/100
im_1_factor.head(20)

im_1_return = im_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IM_1']==5),'ret_vw']- x.loc[(x['rank_IM_1']==1),'ret_vw'])
im_1_return = pd.DataFrame(im_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
im_1_return = im_1_return.rename(columns={'ret_vw' : 'im_1'})
display(im_1_return)


#industry momentum: 6 month return
im_6_factor = pd.read_csv(momentum_factors_path/'portf_me_im_6_monthly_2020.csv')

im_6_factor['date'] = im_6_factor['year'].astype(str) + '/' + im_6_factor['month'].astype(str)
im_6_factor['date'] = pd.to_datetime(im_6_factor['date'])
im_6_factor['date'] = im_6_factor['date'].dt.strftime('%Y-%m')
im_6_factor = im_6_factor.drop(columns=['year','month']).set_index('date')
im_6_factor['ret_vw'] =  im_6_factor['ret_vw']/100
im_6_factor.head(20)

im_6_return = im_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IM_6']==5),'ret_vw']- x.loc[(x['rank_IM_6']==1),'ret_vw'])
im_6_return = pd.DataFrame(im_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
im_6_return = im_6_return.rename(columns={'ret_vw' : 'im_6'})
display(im_6_return)


#industry momentum: 12 month return
im_12_factor = pd.read_csv(momentum_factors_path/'portf_me_im_12_monthly_2020.csv')

im_12_factor['date'] = im_12_factor['year'].astype(str) + '/' + im_12_factor['month'].astype(str)
im_12_factor['date'] = pd.to_datetime(im_12_factor['date'])
im_12_factor['date'] = im_12_factor['date'].dt.strftime('%Y-%m')
im_12_factor = im_12_factor.drop(columns=['year','month']).set_index('date')
im_12_factor['ret_vw'] =  im_12_factor['ret_vw']/100
im_12_factor.head(20)

im_12_return = im_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IM_12']==5),'ret_vw']- x.loc[(x['rank_IM_12']==1),'ret_vw'])
im_12_return = pd.DataFrame(im_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
im_12_return = im_12_return.rename(columns={'ret_vw' : 'im_12'})
display(im_12_return)


all_factors = pd.merge(all_factors, im_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, im_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, im_12_return, how='left', on='date')

,im_1
date,
1967-01,-0.025657
1967-02,0.023912
1967-03,0.038243
1967-04,0.036133
1967-05,0.006616
...,...
2020-08,0.007620
2020-09,0.012308
2020-10,0.015528


,im_6
date,
1967-01,-0.025657
1967-02,0.021154
1967-03,0.038527
1967-04,0.019863
1967-05,-0.008540
...,...
2020-08,0.016893
2020-09,0.001586
2020-10,-0.003238


,im_12
date,
1967-01,-0.025657
1967-02,0.021154
1967-03,0.038527
1967-04,0.019863
1967-05,-0.008540
...,...
2020-08,0.020542
2020-09,0.007319
2020-10,-0.004061


* ### Number of consecutive quarters with earnings higher than same quarter a year ago (nei) 

In [65]:
#Number of consecutive quarters with earnings higher than same quarter a year ago: 1 month return
nei_1_factor = pd.read_csv(momentum_factors_path/'portf_me_nei_1_monthly_2020.csv')

nei_1_factor['date'] = nei_1_factor['year'].astype(str) + '/' + nei_1_factor['month'].astype(str)
nei_1_factor['date'] = pd.to_datetime(nei_1_factor['date'])
nei_1_factor['date'] = nei_1_factor['date'].dt.strftime('%Y-%m')
nei_1_factor = nei_1_factor.drop(columns=['year','month']).set_index('date')
nei_1_factor['ret_vw'] =  nei_1_factor['ret_vw']/100
nei_1_factor.head(20)

nei_1_return = nei_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_NEI_1']==5),'ret_vw']- x.loc[(x['rank_NEI_1']==1),'ret_vw'])
nei_1_return = pd.DataFrame(nei_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
nei_1_return = nei_1_return.rename(columns={'ret_vw' : 'nei_1'})
display(nei_1_return)


all_factors = pd.merge(all_factors, nei_1_return, how='left', on='date')

,nei_1
date,
1969-01,0.006171
1969-02,0.018148
1969-03,0.013769
1969-04,-0.013682
1969-05,-0.002582
...,...
2020-08,-0.042667
2020-09,0.035820
2020-10,0.005424


* ### 52 week high (52w6) 

In [66]:
#52 week high: 6 month return
p52w_6_factor = pd.read_csv(momentum_factors_path/'portf_me_p52w_6_monthly_2020.csv')

p52w_6_factor['date'] = p52w_6_factor['year'].astype(str) + '/' + p52w_6_factor['month'].astype(str)
p52w_6_factor['date'] = pd.to_datetime(p52w_6_factor['date'])
p52w_6_factor['date'] = p52w_6_factor['date'].dt.strftime('%Y-%m')
p52w_6_factor = p52w_6_factor.drop(columns=['year','month']).set_index('date')
p52w_6_factor['ret_vw'] =  p52w_6_factor['ret_vw']/100
p52w_6_factor.head(20)

p52w_6_return = p52w_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_P52W_6']==5),'ret_vw']- x.loc[(x['rank_P52W_6']==1),'ret_vw'])
p52w_6_return = pd.DataFrame(p52w_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
p52w_6_return = p52w_6_return.rename(columns={'ret_vw' : 'p52w_6'})
display(p52w_6_return)


all_factors = pd.merge(all_factors, p52w_6_return, how='left', on='date')

,p52w_6
date,
1967-01,-0.124108
1967-02,0.038167
1967-03,0.010205
1967-04,0.003832
1967-05,0.005077
...,...
2020-08,-0.011467
2020-09,0.048654
2020-10,-0.026979


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1319183257.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, p52w_6_return, how='left', on='date')


* ### Prior six month return (r6) 

In [67]:
#prior six month return: 1 month return
r6_1_factor = pd.read_csv(momentum_factors_path/'portf_me_r6_1_monthly_2020.csv')

r6_1_factor['date'] = r6_1_factor['year'].astype(str) + '/' + r6_1_factor['month'].astype(str)
r6_1_factor['date'] = pd.to_datetime(r6_1_factor['date'])
r6_1_factor['date'] = r6_1_factor['date'].dt.strftime('%Y-%m')
r6_1_factor = r6_1_factor.drop(columns=['year','month']).set_index('date')
r6_1_factor['ret_vw'] =  r6_1_factor['ret_vw']/100
r6_1_factor.head(20)

r6_1_return = r6_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_R6_1']==5),'ret_vw']- x.loc[(x['rank_R6_1']==1),'ret_vw'])
r6_1_return = pd.DataFrame(r6_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
r6_1_return = r6_1_return.rename(columns={'ret_vw' : 'r6_1'})
display(r6_1_return)


all_factors = pd.merge(all_factors, r6_1_return, how='left', on='date')

,r6_1
date,
1967-01,-0.090723
1967-02,0.024068
1967-03,0.035126
1967-04,0.025236
1967-05,0.000012
...,...
2020-08,-0.033558
2020-09,0.043098
2020-10,-0.013827


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1290630703.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, r6_1_return, how='left', on='date')


* ### revisions in analysts' earnings forecasts (re)

In [68]:
#revisions in analysts' earnings forecasts returns: 1 month return
re_1_factor = pd.read_csv(momentum_factors_path/'portf_me_re_1_monthly_2020.csv')

re_1_factor['date'] = re_1_factor['year'].astype(str) + '/' + re_1_factor['month'].astype(str)
re_1_factor['date'] = pd.to_datetime(re_1_factor['date'])
re_1_factor['date'] = re_1_factor['date'].dt.strftime('%Y-%m')
re_1_factor = re_1_factor.drop(columns=['year','month']).set_index('date')
re_1_factor['ret_vw'] =  re_1_factor['ret_vw']/100
re_1_factor.head(20)

re_1_return = re_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_RE_1']==5),'ret_vw']- x.loc[(x['rank_RE_1']==1),'ret_vw'])
re_1_return = pd.DataFrame(re_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
re_1_return = re_1_return.rename(columns={'ret_vw' : 're_1'})
display(re_1_return)


#revisions in analysts' earnings forecasts returns: 6 month return
re_6_factor = pd.read_csv(momentum_factors_path/'portf_me_re_6_monthly_2020.csv')

re_6_factor['date'] = re_6_factor['year'].astype(str) + '/' + re_6_factor['month'].astype(str)
re_6_factor['date'] = pd.to_datetime(re_6_factor['date'])
re_6_factor['date'] = re_6_factor['date'].dt.strftime('%Y-%m')
re_6_factor = re_6_factor.drop(columns=['year','month']).set_index('date')
re_6_factor['ret_vw'] =  re_6_factor['ret_vw']/100
re_6_factor.head(20)

re_6_return = re_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_RE_6']==5),'ret_vw']- x.loc[(x['rank_RE_6']==1),'ret_vw'])
re_6_return = pd.DataFrame(re_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
re_6_return = re_6_return.rename(columns={'ret_vw' : 're_6'})
display(re_6_return)

all_factors = pd.merge(all_factors, re_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, re_6_return, how='left', on='date')

,re_1
date,
1976-07,0.001918
1976-08,0.024221
1976-09,0.008736
1976-10,0.005079
1976-11,0.013187
...,...
2020-08,-0.074930
2020-09,0.046926
2020-10,-0.025220


,re_6
date,
1976-07,0.001918
1976-08,0.016755
1976-09,0.007926
1976-10,0.001731
1976-11,0.016198
...,...
2020-08,-0.047352
2020-09,0.042424
2020-10,-0.022988


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1222305775.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, re_1_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1222305775.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, re_6_return, how='left', on='date')


* ### Six month residual momentum (resid6) 


In [69]:
#six month residual momentum: 6 month return
resid6_factor = pd.read_csv(momentum_factors_path/'portf_me_resid6_6_monthly_2020.csv')

resid6_factor['date'] = resid6_factor['year'].astype(str) + '/' + resid6_factor['month'].astype(str)
resid6_factor['date'] = pd.to_datetime(resid6_factor['date'])
resid6_factor['date'] = resid6_factor['date'].dt.strftime('%Y-%m')
resid6_factor = resid6_factor.drop(columns=['year','month']).set_index('date')
resid6_factor['ret_vw'] =  resid6_factor['ret_vw']/100
resid6_factor.head(20)

resid6_return = resid6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_Resid6_6']==5),'ret_vw']- x.loc[(x['rank_Resid6_6']==1),'ret_vw'])
resid6_return = pd.DataFrame(resid6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
resid6_return = resid6_return.rename(columns={'ret_vw' : 'resid6'})
display(resid6_return)


all_factors = pd.merge(all_factors, resid6_return, how='left', on='date')

,resid6
date,
1967-01,-0.014364
1967-02,0.027364
1967-03,0.018905
1967-04,0.006867
1967-05,0.000500
...,...
2020-08,-0.025118
2020-09,-0.013987
2020-10,-0.005904


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\2755816430.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, resid6_return, how='left', on='date')


* ### Revenue Surprises (rs) 

In [70]:
#revenue surprises: 1 month return
rs_factor = pd.read_csv(momentum_factors_path/'portf_me_rs_1_monthly_2020.csv')

rs_factor['date'] = rs_factor['year'].astype(str) + '/' + rs_factor['month'].astype(str)
rs_factor['date'] = pd.to_datetime(rs_factor['date'])
rs_factor['date'] = rs_factor['date'].dt.strftime('%Y-%m')
rs_factor = rs_factor.drop(columns=['year','month']).set_index('date')
rs_factor['ret_vw'] =  rs_factor['ret_vw']/100
rs_factor.head(20)

rs_return = rs_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_RS_1']==5),'ret_vw']- x.loc[(x['rank_RS_1']==1),'ret_vw'])
rs_return = pd.DataFrame(rs_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
rs_return = rs_return.rename(columns={'ret_vw' : 'rs'})
display(rs_return)


all_factors = pd.merge(all_factors, rs_return, how='left', on='date')

,rs
date,
1967-01,0.011319
1967-02,0.023364
1967-03,-0.012754
1967-04,-0.005144
1967-05,0.034507
...,...
2020-08,-0.044317
2020-09,0.010247
2020-10,-0.033724


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\2562065805.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, rs_return, how='left', on='date')


* ### supplier industries momentum (sim)

In [71]:
#supplier industries momentum: 1 month return
sim_1_factor = pd.read_csv(momentum_factors_path/'portf_me_sim_1_monthly_2020.csv')

sim_1_factor['date'] = sim_1_factor['year'].astype(str) + '/' + sim_1_factor['month'].astype(str)
sim_1_factor['date'] = pd.to_datetime(sim_1_factor['date'])
sim_1_factor['date'] = sim_1_factor['date'].dt.strftime('%Y-%m')
sim_1_factor = sim_1_factor.drop(columns=['year','month']).set_index('date')
sim_1_factor['ret_vw'] =  sim_1_factor['ret_vw']/100
sim_1_factor.head(20)

sim_1_return = sim_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_SIM_1']==5),'ret_vw']- x.loc[(x['rank_SIM_1']==1),'ret_vw'])
sim_1_return = pd.DataFrame(sim_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
sim_1_return = sim_1_return.rename(columns={'ret_vw' : 'sim_1'})
display(sim_1_return)

#supplier industries momentum: 12 month return
sim_12_factor = pd.read_csv(momentum_factors_path/'portf_me_sim_12_monthly_2020.csv')

sim_12_factor['date'] = sim_12_factor['year'].astype(str) + '/' + sim_12_factor['month'].astype(str)
sim_12_factor['date'] = pd.to_datetime(sim_12_factor['date'])
sim_12_factor['date'] = sim_12_factor['date'].dt.strftime('%Y-%m')
sim_12_factor = sim_12_factor.drop(columns=['year','month']).set_index('date')
sim_12_factor['ret_vw'] =  sim_12_factor['ret_vw']/100
sim_12_factor.head(20)

sim_12_return = sim_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_SIM_12']==5),'ret_vw']- x.loc[(x['rank_SIM_12']==1),'ret_vw'])
sim_12_return = pd.DataFrame(sim_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
sim_12_return = sim_12_return.rename(columns={'ret_vw' : 'sim_12'})
display(sim_12_return)


all_factors = pd.merge(all_factors, sim_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, sim_12_return, how='left', on='date')

,sim_1
date,
1967-01,-0.020529
1967-02,-0.006903
1967-03,0.003371
1967-04,0.024256
1967-05,-0.007447
...,...
2020-08,0.041166
2020-09,0.001301
2020-10,0.000313


,sim_12
date,
1967-01,-0.020529
1967-02,-0.012487
1967-03,0.009401
1967-04,0.011484
1967-05,-0.005350
...,...
2020-08,0.008649
2020-09,0.004208
2020-10,0.006864


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1786092599.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, sim_1_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1786092599.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, sim_12_return, how='left', on='date')


* ### Segment Momentum (sm)

In [72]:
#supplier industries momentum: 1 month return
sm_1_factor = pd.read_csv(momentum_factors_path/'portf_me_sm_1_monthly_2020.csv')

sm_1_factor['date'] = sm_1_factor['year'].astype(str) + '/' + sm_1_factor['month'].astype(str)
sm_1_factor['date'] = pd.to_datetime(sm_1_factor['date'])
sm_1_factor['date'] = sm_1_factor['date'].dt.strftime('%Y-%m')
sm_1_factor = sm_1_factor.drop(columns=['year','month']).set_index('date')
sm_1_factor['ret_vw'] =  sm_1_factor['ret_vw']/100
sm_1_factor.head(20)

sm_1_return = sm_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_SM_1']==5),'ret_vw']- x.loc[(x['rank_SM_1']==1),'ret_vw'])
sm_1_return = pd.DataFrame(sm_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
sm_1_return = sm_1_return.rename(columns={'ret_vw' : 'sm_1'})
display(sm_1_return)

#supplier industries momentum: 12 month return
sm_12_factor = pd.read_csv(momentum_factors_path/'portf_me_sm_12_monthly_2020.csv')

sm_12_factor['date'] = sm_12_factor['year'].astype(str) + '/' + sm_12_factor['month'].astype(str)
sm_12_factor['date'] = pd.to_datetime(sm_12_factor['date'])
sm_12_factor['date'] = sm_12_factor['date'].dt.strftime('%Y-%m')
sm_12_factor = sm_12_factor.drop(columns=['year','month']).set_index('date')
sm_12_factor['ret_vw'] =  sm_12_factor['ret_vw']/100
sm_12_factor.head(20)

sm_12_return = sm_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_SM_12']==5),'ret_vw']- x.loc[(x['rank_SM_12']==1),'ret_vw'])
sm_12_return = pd.DataFrame(sm_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
sm_12_return = sm_12_return.rename(columns={'ret_vw' : 'sm_12'})
display(sm_12_return)


all_factors = pd.merge(all_factors, sm_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, sm_12_return, how='left', on='date')

,sm_1
date,
1977-07,-0.012329
1977-08,0.004139
1977-09,-0.011220
1977-10,-0.010087
1977-11,0.017129
...,...
2020-08,-0.014914
2020-09,0.031568
2020-10,0.037887


,sm_12
date,
1977-07,-0.012329
1977-08,-0.001942
1977-09,0.012855
1977-10,-0.007198
1977-11,-0.002527
...,...
2020-08,-0.008040
2020-09,0.019793
2020-10,0.000303


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1763515955.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, sm_1_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1763515955.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, sm_12_return, how='left', on='date')


* ### Standardized unexpected earnings (sue) 

In [73]:
#standardized unexpected earnings: 1 month return
sue_1_factor = pd.read_csv(momentum_factors_path/'portf_me_sue_1_monthly_2020.csv')

sue_1_factor['date'] = sue_1_factor['year'].astype(str) + '/' + sue_1_factor['month'].astype(str)
sue_1_factor['date'] = pd.to_datetime(sue_1_factor['date'])
sue_1_factor['date'] = sue_1_factor['date'].dt.strftime('%Y-%m')
sue_1_factor = sue_1_factor.drop(columns=['year','month']).set_index('date')
sue_1_factor['ret_vw'] =  sue_1_factor['ret_vw']/100
sue_1_factor.head(20)

sue_1_return = sue_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_SUE_1']==5),'ret_vw']- x.loc[(x['rank_SUE_1']==1),'ret_vw'])
sue_1_return = pd.DataFrame(sue_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
sue_1_return = sue_1_return.rename(columns={'ret_vw' : 'sue_1'})
display(sue_1_return)


#standardized unexpected earnings: 6 month return
sue_6_factor = pd.read_csv(momentum_factors_path/'portf_me_sue_6_monthly_2020.csv')

sue_6_factor['date'] = sue_6_factor['year'].astype(str) + '/' + sue_6_factor['month'].astype(str)
sue_6_factor['date'] = pd.to_datetime(sue_6_factor['date'])
sue_6_factor['date'] = sue_6_factor['date'].dt.strftime('%Y-%m')
sue_6_factor = sue_6_factor.drop(columns=['year','month']).set_index('date')
sue_6_factor['ret_vw'] =  sue_6_factor['ret_vw']/100
sue_6_factor.head(20)

sue_6_return = sue_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_SUE_6']==5),'ret_vw']- x.loc[(x['rank_SUE_6']==1),'ret_vw'])
sue_6_return = pd.DataFrame(sue_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
sue_6_return = sue_6_return.rename(columns={'ret_vw' : 'sue_6'})
display(sue_6_return)



all_factors = pd.merge(all_factors, sue_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, sue_6_return, how='left', on='date')

,sue_1
date,
1967-01,0.072958
1967-02,0.012157
1967-03,-0.041543
1967-04,0.004440
1967-05,0.015416
...,...
2020-08,-0.055008
2020-09,0.021054
2020-10,-0.007067


,sue_6
date,
1967-01,0.072958
1967-02,0.013862
1967-03,0.002917
1967-04,0.006332
1967-05,0.023797
...,...
2020-08,-0.035323
2020-09,0.031783
2020-10,-0.009132


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1102006592.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, sue_1_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1102006592.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, sue_6_return, how='left', on='date')


# Value versus Growth factors

* ### Book to market and book to june-end market equity (bm & bmj)

In [74]:
VvG_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/VvG_me_monthly'

#book to market: 1 month return
bm_factor = pd.read_csv(VvG_factors_path/'portf_me_bm_monthly_2020.csv')

bm_factor['date'] = bm_factor['year'].astype(str) + '/' + bm_factor['month'].astype(str)
bm_factor['date'] = pd.to_datetime(bm_factor['date'])
bm_factor['date'] = bm_factor['date'].dt.strftime('%Y-%m')
bm_factor = bm_factor.drop(columns=['year','month']).set_index('date')
bm_factor['ret_vw'] =  bm_factor['ret_vw']/100
bm_factor.head(20)

bm_return = bm_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_BM']==5),'ret_vw']- x.loc[(x['rank_BM']==1),'ret_vw'])
bm_return = pd.DataFrame(bm_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
bm_return = bm_return.rename(columns={'ret_vw' : 'bm'})
display(bm_return)


#book to june-end market equity: 1 month return
bmj_factor = pd.read_csv(VvG_factors_path/'portf_me_bmj_monthly_2020.csv')

bmj_factor['date'] = bmj_factor['year'].astype(str) + '/' + bmj_factor['month'].astype(str)
bmj_factor['date'] = pd.to_datetime(bmj_factor['date'])
bmj_factor['date'] = bmj_factor['date'].dt.strftime('%Y-%m')
bmj_factor = bmj_factor.drop(columns=['year','month']).set_index('date')
bmj_factor['ret_vw'] =  bmj_factor['ret_vw']/100
bmj_factor.head(20)

bmj_return = bmj_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_BMJ']==5),'ret_vw']- x.loc[(x['rank_BMJ']==1),'ret_vw'])
bmj_return = pd.DataFrame(bmj_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
bmj_return = bmj_return.rename(columns={'ret_vw' : 'bmj'})
display(bmj_return)

all_factors = pd.merge(all_factors, bm_return, how='left', on='date')
all_factors = pd.merge(all_factors, bmj_return, how='left', on='date')

,bm
date,
1967-01,0.001801
1967-02,-0.025644
1967-03,-0.005929
1967-04,-0.027556
1967-05,0.011904
...,...
2020-08,-0.010299
2020-09,-0.028398
2020-10,0.024671


,bmj
date,
1967-01,0.004707
1967-02,-0.034307
1967-03,-0.017377
1967-04,-0.023204
1967-05,0.015096
...,...
2020-08,0.031608
2020-09,-0.063046
2020-10,0.042567


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\793634696.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, bm_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\793634696.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, bmj_return, how='left', on='date')


* ###  (quarterly) cash-flow-to price (cp & cpq)

In [75]:
VvG_factors_path = path_factors / 'Replicating anomalies (187 Factors)/3 by 5 sorts with size/VvG_me_monthly'

#cash-flow-to-price: 1 month return
cp_factor = pd.read_csv(VvG_factors_path/'portf_me_cp_monthly_2020.csv')

cp_factor['date'] = cp_factor['year'].astype(str) + '/' + cp_factor['month'].astype(str)
cp_factor['date'] = pd.to_datetime(cp_factor['date'])
cp_factor['date'] = cp_factor['date'].dt.strftime('%Y-%m')
cp_factor = cp_factor.drop(columns=['year','month']).set_index('date')
cp_factor['ret_vw'] =  cp_factor['ret_vw']/100
cp_factor.head(20)

cp_return = cp_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CP']==5),'ret_vw']- x.loc[(x['rank_CP']==1),'ret_vw'])
cp_return = pd.DataFrame(cp_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cp_return = cp_return.rename(columns={'ret_vw' : 'cp'})
display(cp_return)


#quarterly cash-flow-to-price: 1 month return
cpq_1_factor = pd.read_csv(VvG_factors_path/'portf_me_cpq_1_monthly_2020.csv')

cpq_1_factor['date'] = cpq_1_factor['year'].astype(str) + '/' + cpq_1_factor['month'].astype(str)
cpq_1_factor['date'] = pd.to_datetime(cpq_1_factor['date'])
cpq_1_factor['date'] = cpq_1_factor['date'].dt.strftime('%Y-%m')
cpq_1_factor = cpq_1_factor.drop(columns=['year','month']).set_index('date')
cpq_1_factor['ret_vw'] =  cpq_1_factor['ret_vw']/100
cpq_1_factor.head(20)

cpq_1_return = cpq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_CPq_1']==5),'ret_vw']- x.loc[(x['rank_CPq_1']==1),'ret_vw'])
cpq_1_return = pd.DataFrame(cpq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
cpq_1_return = cpq_1_return.rename(columns={'ret_vw' : 'cpq_1'})
display(cpq_1_return)

all_factors = pd.merge(all_factors, cp_return, how='left', on='date')
all_factors = pd.merge(all_factors, cpq_1_return, how='left', on='date')

,cp
date,
1967-01,0.003172
1967-02,-0.032935
1967-03,-0.006517
1967-04,-0.027463
1967-05,0.007866
...,...
2020-08,0.031743
2020-09,-0.034803
2020-10,0.021830


,cpq_1
date,
1967-01,0.070879
1967-02,-0.022687
1967-03,-0.016156
1967-04,-0.003047
1967-05,-0.011151
...,...
2020-08,-0.002335
2020-09,-0.044326
2020-10,0.022203


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\2433741642.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, cp_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\2433741642.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, cpq_1_return, how='left', on='date')


* ### equity duration (dur)

In [76]:
#equity duration: 1 month return
dur_factor = pd.read_csv(VvG_factors_path/'portf_me_dur_monthly_2020.csv')

dur_factor['date'] = dur_factor['year'].astype(str) + '/' + dur_factor['month'].astype(str)
dur_factor['date'] = pd.to_datetime(dur_factor['date'])
dur_factor['date'] = dur_factor['date'].dt.strftime('%Y-%m')
dur_factor = dur_factor.drop(columns=['year','month']).set_index('date')
dur_factor['ret_vw'] =  dur_factor['ret_vw']/100
dur_factor.head(20)

dur_return = dur_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_Dur']==5),'ret_vw']- x.loc[(x['rank_Dur']==1),'ret_vw'])
dur_return = pd.DataFrame(dur_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
dur_return = dur_return.rename(columns={'ret_vw' : 'dur'})
display(dur_return)

all_factors = pd.merge(all_factors, dur_return, how='left', on='date')

,dur
date,
1967-01,-0.033596
1967-02,0.064564
1967-03,0.006749
1967-04,-0.020337
1967-05,0.013937
...,...
2020-08,-0.000583
2020-09,-0.003437
2020-10,-0.010139


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\3238893108.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, dur_return, how='left', on='date')


* ### enterprise book to price (ebp) 

In [77]:
#enterprise book to price: 1 month return
ebp_factor = pd.read_csv(VvG_factors_path/'portf_me_ebp_monthly_2020.csv')

ebp_factor['date'] = ebp_factor['year'].astype(str) + '/' + ebp_factor['month'].astype(str)
ebp_factor['date'] = pd.to_datetime(ebp_factor['date'])
ebp_factor['date'] = ebp_factor['date'].dt.strftime('%Y-%m')
ebp_factor = ebp_factor.drop(columns=['year','month']).set_index('date')
ebp_factor['ret_vw'] =  ebp_factor['ret_vw']/100
ebp_factor.head(20)

ebp_return = ebp_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_EBP']==5),'ret_vw']- x.loc[(x['rank_EBP']==1),'ret_vw'])
ebp_return = pd.DataFrame(ebp_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ebp_return = ebp_return.rename(columns={'ret_vw' : 'ebp'})
display(ebp_return)

all_factors = pd.merge(all_factors, ebp_return, how='left', on='date')

,ebp
date,
1967-01,0.019969
1967-02,-0.024595
1967-03,0.000887
1967-04,-0.022341
1967-05,-0.017007
...,...
2020-08,-0.002167
2020-09,-0.020265
2020-10,0.006779


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\2775569107.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, ebp_return, how='left', on='date')


* ### (quarterly) enterprise multiple (em & emq)

In [78]:
#enterprise multiple: 1 month return
em_factor = pd.read_csv(VvG_factors_path/'portf_me_em_monthly_2020.csv')

em_factor['date'] = em_factor['year'].astype(str) + '/' + em_factor['month'].astype(str)
em_factor['date'] = pd.to_datetime(em_factor['date'])
em_factor['date'] = em_factor['date'].dt.strftime('%Y-%m')
em_factor = em_factor.drop(columns=['year','month']).set_index('date')
em_factor['ret_vw'] =  em_factor['ret_vw']/100
em_factor.head(20)

em_return = em_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_EM']==5),'ret_vw']- x.loc[(x['rank_EM']==1),'ret_vw'])
em_return = pd.DataFrame(em_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
em_return = em_return.rename(columns={'ret_vw' : 'em'})
display(em_return)


#quarterly enterprise multiple: 1 month return
emq_1_factor = pd.read_csv(VvG_factors_path/'portf_me_emq_1_monthly_2020.csv')

emq_1_factor['date'] = emq_1_factor['year'].astype(str) + '/' + emq_1_factor['month'].astype(str)
emq_1_factor['date'] = pd.to_datetime(emq_1_factor['date'])
emq_1_factor['date'] = emq_1_factor['date'].dt.strftime('%Y-%m')
emq_1_factor = emq_1_factor.drop(columns=['year','month']).set_index('date')
emq_1_factor['ret_vw'] =  emq_1_factor['ret_vw']/100
emq_1_factor.head(20)

emq_1_return = emq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_EMq_1']==5),'ret_vw']- x.loc[(x['rank_EMq_1']==1),'ret_vw'])
emq_1_return = pd.DataFrame(emq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
emq_1_return = emq_1_return.rename(columns={'ret_vw' : 'emq_1'})
display(emq_1_return)


#quarterly enterprise multiple: 6 month return
emq_6_factor = pd.read_csv(VvG_factors_path/'portf_me_emq_6_monthly_2020.csv')

emq_6_factor['date'] = emq_6_factor['year'].astype(str) + '/' + emq_6_factor['month'].astype(str)
emq_6_factor['date'] = pd.to_datetime(emq_6_factor['date'])
emq_6_factor['date'] = emq_6_factor['date'].dt.strftime('%Y-%m')
emq_6_factor = emq_6_factor.drop(columns=['year','month']).set_index('date')
emq_6_factor['ret_vw'] =  emq_6_factor['ret_vw']/100
emq_6_factor.head(20)

emq_6_return = emq_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_EMq_6']==5),'ret_vw']- x.loc[(x['rank_EMq_6']==1),'ret_vw'])
emq_6_return = pd.DataFrame(emq_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
emq_6_return = emq_6_return.rename(columns={'ret_vw' : 'emq_6'})
display(emq_6_return)


#quarterly enterprise multiple: 12 month return
emq_12_factor = pd.read_csv(VvG_factors_path/'portf_me_emq_12_monthly_2020.csv')

emq_12_factor['date'] = emq_12_factor['year'].astype(str) + '/' + emq_12_factor['month'].astype(str)
emq_12_factor['date'] = pd.to_datetime(emq_12_factor['date'])
emq_12_factor['date'] = emq_12_factor['date'].dt.strftime('%Y-%m')
emq_12_factor = emq_12_factor.drop(columns=['year','month']).set_index('date')
emq_12_factor['ret_vw'] =  emq_12_factor['ret_vw']/100
emq_12_factor.head(20)

emq_12_return = emq_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_EMq_12']==5),'ret_vw']- x.loc[(x['rank_EMq_12']==1),'ret_vw'])
emq_12_return = pd.DataFrame(emq_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
emq_12_return = emq_12_return.rename(columns={'ret_vw' : 'emq_12'})
display(emq_12_return)


all_factors = pd.merge(all_factors, em_return, how='left', on='date')
all_factors = pd.merge(all_factors, emq_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, emq_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, emq_12_return, how='left', on='date')

,em
date,
1967-01,0.003216
1967-02,0.027136
1967-03,-0.004933
1967-04,0.012688
1967-05,-0.005044
...,...
2020-08,-0.028965
2020-09,0.037415
2020-10,-0.002188


,emq_1
date,
1976-01,-0.010046
1976-02,-0.004089
1976-03,0.001588
1976-04,-0.011266
1976-05,-0.010087
...,...
2020-08,0.022747
2020-09,0.022975
2020-10,-0.001761


,emq_6
date,
1976-01,-0.010046
1976-02,-0.001379
1976-03,0.004007
1976-04,-0.005882
1976-05,-0.013431
...,...
2020-08,-0.001262
2020-09,0.043184
2020-10,-0.001424


,emq_12
date,
1976-01,-0.010046
1976-02,-0.001379
1976-03,0.004007
1976-04,-0.005882
1976-05,-0.013431
...,...
2020-08,-0.002519
2020-09,0.037301
2020-10,-0.000774


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\2315601466.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, em_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\2315601466.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, emq_1_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\2315601466.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.

* ### (quarterly) earnings to price (ep & epq) 

In [79]:
#earnings to price: 1 month return
ep_factor = pd.read_csv(VvG_factors_path/'portf_me_ep_monthly_2020.csv')

ep_factor['date'] = ep_factor['year'].astype(str) + '/' + ep_factor['month'].astype(str)
ep_factor['date'] = pd.to_datetime(ep_factor['date'])
ep_factor['date'] = ep_factor['date'].dt.strftime('%Y-%m')
ep_factor = ep_factor.drop(columns=['year','month']).set_index('date')
ep_factor['ret_vw'] =  ep_factor['ret_vw']/100
ep_factor.head(20)

ep_return = ep_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_EP']==5),'ret_vw']- x.loc[(x['rank_EP']==1),'ret_vw'])
ep_return = pd.DataFrame(ep_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ep_return = ep_return.rename(columns={'ret_vw' : 'ep'})
display(ep_return)

#quarterly earnings to price: 1 month return
epq_1_factor = pd.read_csv(VvG_factors_path/'portf_me_epq_1_monthly_2020.csv')

epq_1_factor['date'] = epq_1_factor['year'].astype(str) + '/' + epq_1_factor['month'].astype(str)
epq_1_factor['date'] = pd.to_datetime(epq_1_factor['date'])
epq_1_factor['date'] = epq_1_factor['date'].dt.strftime('%Y-%m')
epq_1_factor = epq_1_factor.drop(columns=['year','month']).set_index('date')
epq_1_factor['ret_vw'] =  epq_1_factor['ret_vw']/100
epq_1_factor.head(20)

epq_1_return = epq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_EPq_1']==5),'ret_vw']- x.loc[(x['rank_EPq_1']==1),'ret_vw'])
epq_1_return = pd.DataFrame(epq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
epq_1_return = epq_1_return.rename(columns={'ret_vw' : 'epq_1'})
display(epq_1_return)


#quarterly earnings to price: 6 month return
epq_6_factor = pd.read_csv(VvG_factors_path/'portf_me_epq_6_monthly_2020.csv')

epq_6_factor['date'] = epq_6_factor['year'].astype(str) + '/' + epq_6_factor['month'].astype(str)
epq_6_factor['date'] = pd.to_datetime(epq_6_factor['date'])
epq_6_factor['date'] = epq_6_factor['date'].dt.strftime('%Y-%m')
epq_6_factor = epq_6_factor.drop(columns=['year','month']).set_index('date')
epq_6_factor['ret_vw'] =  epq_6_factor['ret_vw']/100
epq_6_factor.head(20)

epq_6_return = epq_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_EPq_6']==5),'ret_vw']- x.loc[(x['rank_EPq_6']==1),'ret_vw'])
epq_6_return = pd.DataFrame(epq_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
epq_6_return = epq_6_return.rename(columns={'ret_vw' : 'epq_6'})
display(epq_6_return)


#quarterly earnings to price: 12 month return
epq_12_factor = pd.read_csv(VvG_factors_path/'portf_me_epq_12_monthly_2020.csv')

epq_12_factor['date'] = epq_12_factor['year'].astype(str) + '/' + epq_12_factor['month'].astype(str)
epq_12_factor['date'] = pd.to_datetime(epq_12_factor['date'])
epq_12_factor['date'] = epq_12_factor['date'].dt.strftime('%Y-%m')
epq_12_factor = epq_12_factor.drop(columns=['year','month']).set_index('date')
epq_12_factor['ret_vw'] =  epq_12_factor['ret_vw']/100
epq_12_factor.head(20)

epq_12_return = epq_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_EPq_12']==5),'ret_vw']- x.loc[(x['rank_EPq_12']==1),'ret_vw'])
epq_12_return = pd.DataFrame(epq_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
epq_12_return = epq_12_return.rename(columns={'ret_vw' : 'epq_12'})
display(epq_12_return)

all_factors = pd.merge(all_factors, ep_return, how='left', on='date')
all_factors = pd.merge(all_factors, epq_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, epq_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, epq_12_return, how='left', on='date')

,ep
date,
1967-01,0.014362
1967-02,-0.033859
1967-03,-0.014588
1967-04,-0.009917
1967-05,0.007123
...,...
2020-08,0.032425
2020-09,-0.000158
2020-10,0.006205


,epq_1
date,
1967-01,0.065210
1967-02,-0.011962
1967-03,0.004548
1967-04,-0.006964
1967-05,0.011454
...,...
2020-08,-0.038062
2020-09,-0.002573
2020-10,-0.009690


,epq_6
date,
1967-01,0.065210
1967-02,-0.022212
1967-03,0.003451
1967-04,-0.004102
1967-05,0.012694
...,...
2020-08,-0.019592
2020-09,0.005520
2020-10,0.020210


,epq_12
date,
1967-01,0.065210
1967-02,-0.022212
1967-03,0.003451
1967-04,-0.004102
1967-05,0.012694
...,...
2020-08,-0.019374
2020-09,0.009763
2020-10,0.028488


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\4105467636.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, ep_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\4105467636.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, epq_1_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\4105467636.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.

* ### intangible return (ir) 

In [80]:
#quarterly earnings to price: 12 month return
ir_factor = pd.read_csv(VvG_factors_path/'portf_me_ir_monthly_2020.csv')

ir_factor['date'] = ir_factor['year'].astype(str) + '/' + ir_factor['month'].astype(str)
ir_factor['date'] = pd.to_datetime(ir_factor['date'])
ir_factor['date'] = ir_factor['date'].dt.strftime('%Y-%m')
ir_factor = ir_factor.drop(columns=['year','month']).set_index('date')
ir_factor['ret_vw'] =  ir_factor['ret_vw']/100
ir_factor.head(20)

ir_return = ir_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_IR']==5),'ret_vw']- x.loc[(x['rank_IR']==1),'ret_vw'])
ir_return = pd.DataFrame(ir_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ir_return = ir_return.rename(columns={'ret_vw' : 'ir'})
display(ir_return)

all_factors = pd.merge(all_factors, ir_return, how='left', on='date')

,ir
date,
1967-01,-0.002821
1967-02,0.029218
1967-03,0.006632
1967-04,-0.006708
1967-05,-0.039761
...,...
2020-08,0.028299
2020-09,0.032621
2020-10,-0.018434


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1093797506.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, ir_return, how='left', on='date')


* ### (net) payout yield (nop)

In [81]:
#quarterly earnings to price: 12 month return
nop_factor = pd.read_csv(VvG_factors_path/'portf_me_nop_monthly_2020.csv')

nop_factor['date'] = nop_factor['year'].astype(str) + '/' + nop_factor['month'].astype(str)
nop_factor['date'] = pd.to_datetime(nop_factor['date'])
nop_factor['date'] = nop_factor['date'].dt.strftime('%Y-%m')
nop_factor = nop_factor.drop(columns=['year','month']).set_index('date')
nop_factor['ret_vw'] =  nop_factor['ret_vw']/100
nop_factor.head(20)

nop_return = nop_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_NOP']==5),'ret_vw']- x.loc[(x['rank_NOP']==1),'ret_vw'])
nop_return = pd.DataFrame(nop_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
nop_return = nop_return.rename(columns={'ret_vw' : 'nop'})
display(nop_return)

all_factors = pd.merge(all_factors, nop_return, how='left', on='date')

,nop
date,
1972-07,0.038684
1972-08,0.034631
1972-09,0.024979
1972-10,0.013123
1972-11,0.022142
...,...
2020-08,0.030626
2020-09,-0.024689
2020-10,-0.015538


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\2455281591.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, nop_return, how='left', on='date')


* ### operating cash flow to price (ocp)

In [82]:
#quarterly earnings to price: 12 month return
ocp_factor = pd.read_csv(VvG_factors_path/'portf_me_ocp_monthly_2020.csv')

ocp_factor['date'] = ocp_factor['year'].astype(str) + '/' + ocp_factor['month'].astype(str)
ocp_factor['date'] = pd.to_datetime(ocp_factor['date'])
ocp_factor['date'] = ocp_factor['date'].dt.strftime('%Y-%m')
ocp_factor = ocp_factor.drop(columns=['year','month']).set_index('date')
ocp_factor['ret_vw'] =  ocp_factor['ret_vw']/100
ocp_factor.head(20)

ocp_return = ocp_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_OCP']==5),'ret_vw']- x.loc[(x['rank_OCP']==1),'ret_vw'])
ocp_return = pd.DataFrame(ocp_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
ocp_return = ocp_return.rename(columns={'ret_vw' : 'ocp'})
display(ocp_return)

all_factors = pd.merge(all_factors, ocp_return, how='left', on='date')

,ocp
date,
1972-07,0.003210
1972-08,0.049250
1972-09,-0.005205
1972-10,0.007775
1972-11,0.039320
...,...
2020-08,0.020987
2020-09,-0.029842
2020-10,0.032590


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\44145956.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, ocp_return, how='left', on='date')


* ### Reversal (rev) 


In [83]:
#reversal returns: 1 month
rev_1_factor = pd.read_csv(VvG_factors_path/'portf_me_rev_1_monthly_2020.csv')

rev_1_factor['date'] = rev_1_factor['year'].astype(str) + '/' + rev_1_factor['month'].astype(str)
rev_1_factor['date'] = pd.to_datetime(rev_1_factor['date'])
rev_1_factor['date'] = rev_1_factor['date'].dt.strftime('%Y-%m')
rev_1_factor = rev_1_factor.drop(columns=['year','month']).set_index('date')
rev_1_factor['ret_vw'] =  rev_1_factor['ret_vw']/100
rev_1_factor.head(20)

rev_1_return = rev_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_Rev_1']==5),'ret_vw']- x.loc[(x['rank_Rev_1']==1),'ret_vw'])
rev_1_return = pd.DataFrame(rev_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
rev_1_return = rev_1_return.rename(columns={'ret_vw' : 'rev_1'})
display(rev_1_return)


#reversal returns: 6 month
rev_6_factor = pd.read_csv(VvG_factors_path/'portf_me_rev_6_monthly_2020.csv')

rev_6_factor['date'] = rev_6_factor['year'].astype(str) + '/' + rev_6_factor['month'].astype(str)
rev_6_factor['date'] = pd.to_datetime(rev_6_factor['date'])
rev_6_factor['date'] = rev_6_factor['date'].dt.strftime('%Y-%m')
rev_6_factor = rev_6_factor.drop(columns=['year','month']).set_index('date')
rev_6_factor['ret_vw'] =  rev_6_factor['ret_vw']/100
rev_6_factor.head(20)

rev_6_return = rev_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_Rev_6']==5),'ret_vw']- x.loc[(x['rank_Rev_6']==1),'ret_vw'])
rev_6_return = pd.DataFrame(rev_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
rev_6_return = rev_6_return.rename(columns={'ret_vw' : 'rev_6'})
display(rev_6_return)



#reversal returns: 12 month
rev_12_factor = pd.read_csv(VvG_factors_path/'portf_me_rev_12_monthly_2020.csv')

rev_12_factor['date'] = rev_12_factor['year'].astype(str) + '/' + rev_12_factor['month'].astype(str)
rev_12_factor['date'] = pd.to_datetime(rev_12_factor['date'])
rev_12_factor['date'] = rev_12_factor['date'].dt.strftime('%Y-%m')
rev_12_factor = rev_12_factor.drop(columns=['year','month']).set_index('date')
rev_12_factor['ret_vw'] =  rev_12_factor['ret_vw']/100
rev_12_factor.head(20)

rev_12_return = rev_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_Rev_12']==5),'ret_vw']- x.loc[(x['rank_Rev_12']==1),'ret_vw'])
rev_12_return = pd.DataFrame(rev_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
rev_12_return = rev_12_return.rename(columns={'ret_vw' : 'rev_12'})
display(rev_12_return)


all_factors = pd.merge(all_factors, rev_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, rev_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, rev_12_return, how='left', on='date')

,rev_1
date,
1967-01,0.000439
1967-02,-0.023228
1967-03,0.005225
1967-04,0.021042
1967-05,-0.002789
...,...
2020-08,0.054950
2020-09,0.019183
2020-10,0.013602


,rev_6
date,
1967-01,0.000439
1967-02,-0.028387
1967-03,0.004959
1967-04,0.020570
1967-05,-0.002849
...,...
2020-08,0.045736
2020-09,0.019941
2020-10,0.017013


,rev_12
date,
1967-01,0.000439
1967-02,-0.028387
1967-03,0.004959
1967-04,0.020570
1967-05,-0.002849
...,...
2020-08,0.036032
2020-09,0.019689
2020-10,0.007273


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\4140374434.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, rev_1_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\4140374434.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, rev_6_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\4140374434.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fra

* ### (quarterly) Sales to price (sp & spq)

In [84]:
#sales to price 
sp_factor = pd.read_csv(VvG_factors_path/'portf_me_sp_monthly_2020.csv')

sp_factor['date'] = sp_factor['year'].astype(str) + '/' + sp_factor['month'].astype(str)
sp_factor['date'] = pd.to_datetime(sp_factor['date'])
sp_factor['date'] = sp_factor['date'].dt.strftime('%Y-%m')
sp_factor = sp_factor.drop(columns=['year','month']).set_index('date')
sp_factor['ret_vw'] =  sp_factor['ret_vw']/100
sp_factor.head(20)

sp_return = sp_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_SP']==5),'ret_vw']- x.loc[(x['rank_SP']==1),'ret_vw'])
sp_return = pd.DataFrame(sp_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
sp_return = sp_return.rename(columns={'ret_vw' : 'sp'})
display(sp_return)


#quarterly sales to price: 1 month return
spq_1_factor = pd.read_csv(VvG_factors_path/'portf_me_spq_1_monthly_2020.csv')

spq_1_factor['date'] = spq_1_factor['year'].astype(str) + '/' + spq_1_factor['month'].astype(str)
spq_1_factor['date'] = pd.to_datetime(spq_1_factor['date'])
spq_1_factor['date'] = spq_1_factor['date'].dt.strftime('%Y-%m')
spq_1_factor = spq_1_factor.drop(columns=['year','month']).set_index('date')
spq_1_factor['ret_vw'] =  spq_1_factor['ret_vw']/100
spq_1_factor.head(20)

spq_1_return = spq_1_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_SPq_1']==5),'ret_vw']- x.loc[(x['rank_SPq_1']==1),'ret_vw'])
spq_1_return = pd.DataFrame(spq_1_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
spq_1_return = spq_1_return.rename(columns={'ret_vw' : 'spq_1'})
display(spq_1_return)


#quarterly sales to price: 6 month return
spq_6_factor = pd.read_csv(VvG_factors_path/'portf_me_spq_6_monthly_2020.csv')

spq_6_factor['date'] = spq_6_factor['year'].astype(str) + '/' + spq_6_factor['month'].astype(str)
spq_6_factor['date'] = pd.to_datetime(spq_6_factor['date'])
spq_6_factor['date'] = spq_6_factor['date'].dt.strftime('%Y-%m')
spq_6_factor = spq_6_factor.drop(columns=['year','month']).set_index('date')
spq_6_factor['ret_vw'] =  spq_6_factor['ret_vw']/100
spq_6_factor.head(20)

spq_6_return = spq_6_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_SPq_6']==5),'ret_vw']- x.loc[(x['rank_SPq_6']==1),'ret_vw'])
spq_6_return = pd.DataFrame(spq_6_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
spq_6_return = spq_6_return.rename(columns={'ret_vw' : 'spq_6'})
display(spq_6_return)


#quarterly sales to price: 12 month return
spq_12_factor = pd.read_csv(VvG_factors_path/'portf_me_spq_12_monthly_2020.csv')

spq_12_factor['date'] = spq_12_factor['year'].astype(str) + '/' + spq_12_factor['month'].astype(str)
spq_12_factor['date'] = pd.to_datetime(spq_12_factor['date'])
spq_12_factor['date'] = spq_12_factor['date'].dt.strftime('%Y-%m')
spq_12_factor = spq_12_factor.drop(columns=['year','month']).set_index('date')
spq_12_factor['ret_vw'] =  spq_12_factor['ret_vw']/100
spq_12_factor.head(20)

spq_12_return = spq_12_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_SPq_12']==5),'ret_vw']- x.loc[(x['rank_SPq_12']==1),'ret_vw'])
spq_12_return = pd.DataFrame(spq_12_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
spq_12_return = spq_12_return.rename(columns={'ret_vw' : 'spq_12'})
display(spq_12_return)


all_factors = pd.merge(all_factors, sp_return, how='left', on='date')
all_factors = pd.merge(all_factors, spq_1_return, how='left', on='date')
all_factors = pd.merge(all_factors, spq_6_return, how='left', on='date')
all_factors = pd.merge(all_factors, spq_12_return, how='left', on='date')

,sp
date,
1967-01,0.050971
1967-02,-0.002031
1967-03,-0.009786
1967-04,-0.010316
1967-05,0.015194
...,...
2020-08,0.030571
2020-09,-0.019765
2020-10,0.025982


,spq_1
date,
1967-01,0.078996
1967-02,-0.001010
1967-03,-0.014697
1967-04,-0.006040
1967-05,0.010124
...,...
2020-08,0.034553
2020-09,-0.017473
2020-10,0.032141


,spq_6
date,
1967-01,0.078996
1967-02,-0.003749
1967-03,-0.013385
1967-04,0.000983
1967-05,0.003929
...,...
2020-08,0.037381
2020-09,-0.029365
2020-10,0.032177


,spq_12
date,
1967-01,0.078996
1967-02,-0.003749
1967-03,-0.013385
1967-04,0.000983
1967-05,0.003929
...,...
2020-08,0.032909
2020-09,-0.025945
2020-10,0.030765


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\3385659651.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, sp_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\3385659651.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, spq_1_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\3385659651.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.

* ### Intrinsic value to market (vhp & vfp)

In [85]:
#intrinisc value to market (vfp)
vfp_factor = pd.read_csv(VvG_factors_path/'portf_me_vfp_monthly_2020.csv')

vfp_factor['date'] = vfp_factor['year'].astype(str) + '/' + vfp_factor['month'].astype(str)
vfp_factor['date'] = pd.to_datetime(vfp_factor['date'])
vfp_factor['date'] = vfp_factor['date'].dt.strftime('%Y-%m')
vfp_factor = vfp_factor.drop(columns=['year','month']).set_index('date')
vfp_factor['ret_vw'] =  vfp_factor['ret_vw']/100
vfp_factor.head(20)

vfp_return = vfp_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_VFP']==5),'ret_vw']- x.loc[(x['rank_VFP']==1),'ret_vw'])
vfp_return = pd.DataFrame(vfp_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
vfp_return = vfp_return.rename(columns={'ret_vw' : 'vfp'})
display(vfp_return)

#intrinisc value to market (vfp)
vhp_factor = pd.read_csv(VvG_factors_path/'portf_me_vhp_monthly_2020.csv')

vhp_factor['date'] = vhp_factor['year'].astype(str) + '/' + vhp_factor['month'].astype(str)
vhp_factor['date'] = pd.to_datetime(vhp_factor['date'])
vhp_factor['date'] = vhp_factor['date'].dt.strftime('%Y-%m')
vhp_factor = vhp_factor.drop(columns=['year','month']).set_index('date')
vhp_factor['ret_vw'] =  vhp_factor['ret_vw']/100
vhp_factor.head(20)

vhp_return = vhp_factor.groupby(['date','rank_ME']).apply(lambda x: x.loc[(x['rank_VHP']==5),'ret_vw']- x.loc[(x['rank_VHP']==1),'ret_vw'])
vhp_return = pd.DataFrame(vhp_return.groupby('date').apply(lambda x: np.nansum(x.values)/len(x.values)))
vhp_return = vhp_return.rename(columns={'ret_vw' : 'vhp'})
display(vhp_return)



all_factors = pd.merge(all_factors, vfp_return, how='left', on='date')
all_factors = pd.merge(all_factors, vhp_return, how='left', on='date')

,vfp
date,
1976-07,0.033068
1976-08,0.038688
1976-09,0.122934
1976-10,-0.034214
1976-11,0.085167
...,...
2020-08,-0.006249
2020-09,-0.005784
2020-10,0.004541


,vhp
date,
1967-01,0.031276
1967-02,-0.031611
1967-03,0.014467
1967-04,0.001062
1967-05,-0.016319
...,...
2020-08,0.038324
2020-09,0.006464
2020-10,0.002010


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\979203955.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, vfp_return, how='left', on='date')
C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\979203955.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, vhp_return, how='left', on='date')


# Factors Brian Kelly

* ### Amihud liqudity measure, %change in sales - % change in receivables, return volatility, labor force efficiency, 

In [86]:
path_bkelly = path_factors / 'Is there a replication crisis in Finance (B. Kelly)/Factors monthly'

amihud = pd.DataFrame(pd.read_csv(path_bkelly/'[usa]_[ami_126d]_[monthly]_[vw].csv'))
amihud['date'] = pd.to_datetime(amihud['date'])
amihud['date'] = amihud['date'].dt.strftime('%Y-%m')
amihud = amihud.drop(columns=['location', 'name', 'freq', 'weighting', 'direction', 'n_stocks', 'n_stocks_min']).set_index('date')
amihud = amihud.rename(columns={'ret':'amihud'})
display(amihud)
all_factors = pd.merge(all_factors, amihud, how='left', on='date')


#% change in sales - % change in acc. receivables
dsre = pd.DataFrame(pd.read_csv(path_bkelly/'[usa]_[dsale_drec]_[monthly]_[vw].csv'))
dsre['date'] = pd.to_datetime(dsre['date'])
dsre['date'] = dsre['date'].dt.strftime('%Y-%m')
dsre = dsre.drop(columns=['location', 'name', 'freq', 'weighting', 'direction', 'n_stocks', 'n_stocks_min']).set_index('date')
dsre = dsre.rename(columns={'ret':'dsre'})
display(dsre)
all_factors = pd.merge(all_factors, dsre, how='left', on='date')

# return volatility
rvol = pd.DataFrame(pd.read_csv(path_bkelly/'[usa]_[rvol_21d]_[monthly]_[vw].csv'))
rvol['date'] = pd.to_datetime(rvol['date'])
rvol['date'] = rvol['date'].dt.strftime('%Y-%m')
rvol = rvol.drop(columns=['location', 'name', 'freq', 'weighting', 'direction', 'n_stocks', 'n_stocks_min']).set_index('date')
rvol = rvol.rename(columns={'ret':'rvol'})
display(rvol)
all_factors = pd.merge(all_factors, rvol, how='left', on='date')

# labor force efficiency
lfe = pd.DataFrame(pd.read_csv(path_bkelly/'[usa]_[sale_emp_gr1]_[monthly]_[vw].csv'))
lfe['date'] = pd.to_datetime(lfe['date'])
lfe['date'] = lfe['date'].dt.strftime('%Y-%m')
lfe = lfe.drop(columns=['location', 'name', 'freq', 'weighting', 'direction', 'n_stocks', 'n_stocks_min']).set_index('date')
lfe = lfe.rename(columns={'ret':'lfe'})
display(lfe)
all_factors = pd.merge(all_factors, lfe, how='left', on='date')

#share turnover
sturn = pd.DataFrame(pd.read_csv(path_bkelly/'[usa]_[turnover_126d]_[monthly]_[vw].csv'))
sturn['date'] = pd.to_datetime(sturn['date'])
sturn['date'] = sturn['date'].dt.strftime('%Y-%m')
sturn = sturn.drop(columns=['location', 'name', 'freq', 'weighting', 'direction', 'n_stocks', 'n_stocks_min']).set_index('date')
sturn = sturn.rename(columns={'ret':'sturn'})
display(sturn)
all_factors = pd.merge(all_factors, sturn, how='left', on='date')


#number of zero trades 
zerotrade = pd.DataFrame(pd.read_csv(path_bkelly/'[usa]_[zero_trades_252d]_[monthly]_[vw].csv'))
zerotrade['date'] = pd.to_datetime(zerotrade['date'])
zerotrade['date'] = zerotrade['date'].dt.strftime('%Y-%m')
zerotrade = zerotrade.drop(columns=['location', 'name', 'freq', 'weighting', 'direction', 'n_stocks', 'n_stocks_min']).set_index('date')
zerotrade = zerotrade.rename(columns={'ret':'0trade'})
display(zerotrade)
all_factors = pd.merge(all_factors, zerotrade, how='left', on='date')

,amihud
date,
1926-04,-0.003724
1926-05,-0.005195
1926-06,-0.023081
1926-07,-0.017221
1926-08,-0.008983
...,...
2021-08,-0.004034
2021-09,0.015235
2021-10,-0.043463


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1126383071.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, amihud, how='left', on='date')


,dsre
date,
1952-11,-0.022706
1952-12,-0.014293
1953-01,0.039101
1953-02,0.019824
1953-03,-0.017690
...,...
2021-08,-0.008832
2021-09,0.029762
2021-10,0.034580


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1126383071.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, dsre, how='left', on='date')


,rvol
date,
1926-02,0.034089
1926-03,0.055450
1926-04,0.004096
1926-05,0.009348
1926-06,-0.017478
...,...
2021-08,-0.000758
2021-09,-0.009554
2021-10,0.022217


...

C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1126383071.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, lfe, how='left', on='date')


,sturn
date,
1926-04,-0.009026
1926-05,0.010668
1926-06,-0.063498
1926-07,-0.039358
1926-08,0.008771
...,...
2021-08,-0.008165
2021-09,-0.023154
2021-10,-0.025484


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1126383071.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, sturn, how='left', on='date')


,0trade
date,
1926-06,-0.022175
1926-07,-0.011167
1926-08,-0.009243
1926-09,-0.002692
1926-10,0.014763
...,...
2021-08,-0.008452
2021-09,-0.025665
2021-10,-0.024304


C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\1126383071.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, zerotrade, how='left', on='date')


In [87]:
liq = pd.read_csv(path_factors /'Liqudity factors Stambaugh & Yuan/Liquidity factors of Pastor and Stambaugh (2003).CSV', header=1)
liq = liq.drop(liq.index[0:65])
liq = liq.rename(columns={'Month':'date', 'Traded Liq (LIQ_V)':'liq_v'})
liq['date'] = pd.to_datetime(liq['date'], format='%Y%m')
liq['date'] = liq['date'].dt.strftime('%Y-%m')
liq = liq.drop(columns=['Agg Liq.','Innov Liq (eq8)'])
liq = liq.set_index('date')
liq
all_factors = pd.merge(all_factors, liq, how='left', on='date')

C:\Users\mauri\AppData\Local\Temp\ipykernel_4980\3358402730.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_factors = pd.merge(all_factors, liq, how='left', on='date')


In [88]:
all_factors = all_factors.sort_index(axis=1)
all_factors = all_factors.loc['1977-07':'2020-12']
# all_factors

In [89]:
################################################## saving ###################################################
# all_factors.to_hdf(factor_data, 'all_factors_monthly', mode='a', data_columns=True, complib='zlib')
###############################################################################################################

In [90]:
################################################# saving  ##################################################
all_factors.to_csv(factor_data_csv) #, compression='zip', encoding='unicode_escape-8'
############################################################################################################

In [91]:
################################################# saving  ##################################################
all_factors.to_excel(factor_data_xlsx) #, compression='zip', encoding='unicode_escape-8'
############################################################################################################

In [92]:
all_factors.describe()

,0trade,abr_1,abr_12,abr_6,aci,adm,almq_1,almq_12,almq_6,amihud,...,spq_12,spq_6,sturn,sue_1,sue_6,ta,tbiq_12,tbiq_6,vfp,vhp
count,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,...,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000
mean,-0.001324,0.007178,0.002769,0.004033,-0.001065,0.002491,0.003464,0.003207,0.004320,0.001029,...,0.002956,0.003736,-0.001340,0.008304,0.004700,-0.001009,0.001158,0.001173,0.000708,0.002345
std,0.041670,0.017187,0.009298,0.012238,0.014198,0.035328,0.031383,0.027971,0.029125,0.028872,...,0.040005,0.041205,0.042625,0.023040,0.019652,0.017050,0.014910,0.015257,0.032473,0.028082
min,-0.287191,-0.096646,-0.051947,-0.066013,-0.086380,-0.188163,-0.135605,-0.109481,-0.121134,-0.109204,...,-0.306745,-0.291147,-0.294490,-0.197626,-0.119896,-0.080611,-0.127842,-0.108817,-0.262746,-0.164404
25%,-0.021481,-0.002088,-0.002581,-0.002542,-0.008630,-0.016026,-0.013517,-0.012193,-0.011981,-0.016383,...,-0.016568,-0.016826,-0.022364,-0.002726,-0.004414,-0.010139,-0.006867,-0.006228,-0.015655,-0.011993
50%,-0.002732,0.007642,0.003283,0.004620,-0.001561,0.002683,0.003852,0.003069,0.003968,0.000903,...,0.004010,0.003991,-0.001696,0.008260,0.005051,-0.000463,0.001612,0.001559,-0.000027,0.001714
75%,0.018814,0.017170,0.008258,0.010918,0.006509,0.022865,0.021370,0.020692,0.022004,0.018879,...,0.022367,0.023119,0.019477,0.021998,0.015640,0.009963,0.010080,0.010708,0.018179,0.015418
max,0.226885,0.107107,0.041808,0.051721,0.070530,0.139692,0.167916,0.136139,0.156792,0.149636,...,0.215728,0.290870,0.215168,0.070844,0.091266,0.057861,0.056692,0.052083,0.124970,0.124190


In [4]:
all_factors = pd.read_csv(factor_data_csv)
all_factors

,date,0trade,abr_1,abr_12,abr_6,aci,adm,almq_1,almq_12,almq_6,...,spq_12,spq_6,sturn,sue_1,sue_6,ta,tbiq_12,tbiq_6,vfp,vhp
0,1977-07,0.018692,-0.004539,-0.000145,-0.000543,-0.021161,0.012778,-0.016623,-0.011760,-0.012382,...,-0.002878,-0.003610,0.025480,0.025039,0.020080,-0.006119,0.014968,0.012904,0.008684,-0.007701
1,1977-08,-0.005823,0.016975,-0.001645,-0.003775,-0.003913,-0.003893,-0.016834,-0.028656,-0.031445,...,-0.009160,-0.010745,-0.000193,0.015148,0.015928,0.016202,-0.003095,-0.002748,-0.005812,-0.029833
2,1977-09,0.009677,0.017575,0.006374,0.008475,-0.002640,-0.007515,-0.005426,-0.014506,-0.009094,...,-0.019648,-0.019498,0.004160,0.020957,0.018645,0.010797,0.013106,0.012017,-0.009841,0.018992
3,1977-10,0.011402,-0.006925,-0.007474,-0.004100,-0.005689,0.012399,0.015994,0.010833,0.013832,...,-0.000928,-0.000866,0.010722,0.000512,0.005392,0.007045,0.008343,0.006349,-0.107805,0.011240
4,1977-11,-0.030581,0.031692,0.008866,0.012289,-0.009848,0.001452,0.023535,0.004836,0.010960,...,0.010892,0.007995,-0.035501,0.044304,0.026799,0.003959,0.001326,0.008671,0.028768,0.004361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,2020-08,-0.049368,0.018227,0.013549,0.005280,-0.010580,0.000493,0.009167,0.004975,0.004676,...,0.032909,0.037381,-0.045621,-0.055008,-0.035323,-0.001280,0.022186,0.023411,-0.006249,0.038324
518,2020-09,-0.006052,0.006131,-0.014598,-0.011446,-0.012887,-0.009621,-0.004355,-0.003496,-0.008132,...,-0.025945,-0.029365,0.001516,0.021054,0.031783,0.020368,-0.030054,-0.046113,-0.005784,0.006464
519,2020-10,-0.011785,-0.003378,-0.004676,-0.007752,0.007652,0.044951,0.056244,0.049368,0.050133,...,0.030765,0.032177,-0.004541,-0.007067,-0.009132,-0.000454,-0.013322,-0.019826,0.004541,0.002010
520,2020-11,-0.100174,-0.014360,-0.011801,0.002971,-0.031970,0.021466,0.087609,0.057446,0.055007,...,0.052702,0.063769,-0.111326,-0.114737,-0.080291,-0.026441,0.018709,0.039207,0.012657,0.035152


In [5]:
selected_rows = all_factors[all_factors.isna().any(axis=1)]
selected_rows

for c in selected_rows.columns:
    if selected_rows[c].isna().any():
        print(c)
        


## Creation of monthly, semi-annually and yearly factors

In [6]:
column_names = all_factors.columns.to_list()
column_names

['date',
 '0trade',
 'abr_1',
 'abr_12',
 'abr_6',
 'aci',
 'adm',
 'almq_1',
 'almq_12',
 'almq_6',
 'amihud',
 'atoq_1',
 'atoq_12',
 'atoq_6',
 'bab',
 'bm',
 'bmj',
 'cei',
 'cim_1',
 'cim_12',
 'cim_6',
 'cla',
 'claq_1',
 'claq_12',
 'claq_6',
 'cma',
 'cop',
 'cp',
 'cpq_1',
 'ctoq_1',
 'ctoq_12',
 'ctoq_6',
 'dac',
 'dcoa',
 'def_1',
 'def_12',
 'def_6',
 'dfin',
 'dfnl',
 'dii',
 'dlno',
 'dnca',
 'dnco',
 'dnoa',
 'dpia',
 'droa_1',
 'droa_6',
 'droe_1',
 'droe_12',
 'droe_6',
 'dsre',
 'dtv_12',
 'dur',
 'ebp',
 'em',
 'emq_1',
 'emq_12',
 'emq_6',
 'ep',
 'epq_1',
 'epq_12',
 'epq_6',
 'eprd',
 'etl',
 'etr',
 'fp_6',
 'glaq_1',
 'glaq_12',
 'glaq_6',
 'gpa',
 'hml',
 'hml devil',
 'hs',
 'ia',
 'iaq_12',
 'iaq_6',
 'ig',
 'ig2',
 'ile_1',
 'ilr_1',
 'ilr_12',
 'ilr_6',
 'im_1',
 'im_12',
 'im_6',
 'ioca',
 'ir',
 'isq_1',
 'ivc',
 'ivg',
 'ivq_1',
 'lfe',
 'liq_v',
 'mkt-rf',
 'mom',
 'ndf',
 'nei_1',
 'noa',
 'nop',
 'nsi',
 'oa',
 'oca',
 'ocp',
 'ol',
 'olaq_1',
 'olaq_

In [7]:
# factors that are only available at a specific time interval
# [cpq_1, dtv_12, fp_6, iaq_12, ile_1, isq_1, ivq_1, nei_1, oleq_1, p52w_6, r6_1, r1a,resid6, tbiq_12]

unique_factors_for_monthly = ['dtv_12', 'fp_6', 'iaq_12', 'p52w_6','r6_1', 'resid6', 'tbiq_12']

In [10]:
monthly_factors =[] 
for c in column_names:
    if ('6' not in c) and ('12' not in c):
        monthly_factors.append(c)

monthly_factors = all_factors[monthly_factors+unique_factors_for_monthly]
monthly_factors

,date,0trade,abr_1,aci,adm,almq_1,amihud,atoq_1,bab,bm,...,ta,vfp,vhp,dtv_12,fp_6,iaq_12,p52w_6,r6_1,resid6,tbiq_12
0,1977-07,0.018692,-0.004539,-0.021161,0.012778,-0.016623,0.013096,0.010988,0.028603,-0.010927,...,-0.006119,0.008684,-0.007701,-0.011470,-0.008559,-0.006932,0.008370,-0.016347,0.000741,0.014968
1,1977-08,-0.005823,0.016975,-0.003913,-0.003893,-0.016834,0.001388,0.017685,0.003437,-0.033594,...,0.016202,-0.005812,-0.029833,-0.066274,-0.024926,0.017061,-0.018493,-0.011630,-0.016691,-0.003095
2,1977-09,0.009677,0.017575,-0.002640,-0.007515,-0.005426,0.011608,-0.007131,0.004118,-0.001737,...,0.010797,-0.009841,0.018992,-0.070893,-0.022246,0.005390,0.022098,0.024979,0.007343,0.013106
3,1977-10,0.011402,-0.006925,-0.005689,0.012399,0.015994,0.016949,-0.000285,0.001830,0.014104,...,0.007045,-0.107805,0.011240,0.054972,-0.008334,0.000042,0.010250,-0.006560,0.002978,0.008343
4,1977-11,-0.030581,0.031692,-0.009848,0.001452,0.023535,0.041280,0.006559,0.002795,0.011124,...,0.003959,0.028768,0.004361,-0.005183,0.012565,-0.013679,-0.005397,0.025724,0.006848,0.001326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,2020-08,-0.049368,0.018227,-0.010580,0.000493,0.009167,-0.013972,0.030009,-0.039535,-0.010299,...,-0.001280,-0.006249,0.038324,0.035736,0.006379,0.022639,-0.011467,-0.033558,-0.025118,0.022186
518,2020-09,-0.006052,0.006131,-0.012887,-0.009621,-0.004355,-0.004987,0.010916,0.012960,-0.028398,...,0.020368,-0.005784,0.006464,0.140698,-0.017566,0.011710,0.048654,0.043098,-0.013987,-0.030054
519,2020-10,-0.011785,-0.003378,0.007652,0.044951,0.056244,0.041393,0.002211,-0.019986,0.024671,...,-0.000454,0.004541,0.002010,-0.035899,0.003301,-0.000237,-0.026979,-0.013827,-0.005904,-0.013322
520,2020-11,-0.100174,-0.014360,-0.031970,0.021466,0.087609,0.077867,-0.057850,-0.050780,0.055987,...,-0.026441,0.012657,0.035152,0.082814,0.218547,-0.017548,-0.197040,0.016636,0.004878,0.018709


In [11]:
################################################# saving  ##################################################
monthly_factors.to_csv(monthly_factor_data_csv) #, compression='zip', encoding='unicode_escape-8'
############################################################################################################

In [68]:
semi_ann_factors =[] 
for c in column_names:
    if ('1' not in c) and ('12' not in c):
        semi_ann_factors.append(c)

semi_ann_factors = all_factors[semi_ann_factors]
semi_ann_factors

,date,0trade,abr_6,aci,adm,almq_6,amihud,atoq_6,bab,bm,...,rvol,smb,sp,spq_6,sturn,sue_6,ta,tbiq_6,vfp,vhp
0,1977-07,0.018692,-0.000543,-0.021161,0.012778,-0.012382,0.013096,0.014043,0.028603,-0.010927,...,0.006749,0.0188,-0.005382,-0.003610,0.025480,0.020080,-0.006119,0.012904,0.008684,-0.007701
1,1977-08,-0.005823,-0.003775,-0.003913,-0.003893,-0.031445,0.001388,0.023454,0.003437,-0.033594,...,-0.007967,0.0086,-0.027291,-0.010745,-0.000193,0.015928,0.016202,-0.002748,-0.005812,-0.029833
2,1977-09,0.009677,0.008475,-0.002640,-0.007515,-0.009094,0.011608,-0.005028,0.004118,-0.001737,...,0.006982,0.0143,-0.006873,-0.019498,0.004160,0.018645,0.010797,0.012017,-0.009841,0.018992
3,1977-10,0.011402,-0.004100,-0.005689,0.012399,0.013832,0.016949,0.004488,0.001830,0.014104,...,-0.001238,0.0148,0.006467,-0.000866,0.010722,0.005392,0.007045,0.006349,-0.107805,0.011240
4,1977-11,-0.030581,0.012289,-0.009848,0.001452,0.010960,0.041280,-0.003374,0.002795,0.011124,...,-0.033044,0.0364,0.012190,0.007995,-0.035501,0.026799,0.003959,0.008671,0.028768,0.004361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,2020-08,-0.049368,0.005280,-0.010580,0.000493,0.004676,-0.013972,0.026902,-0.039535,-0.010299,...,-0.056292,-0.0092,0.030571,0.037381,-0.045621,-0.035323,-0.001280,0.023411,-0.006249,0.038324
518,2020-09,-0.006052,-0.011446,-0.012887,-0.009621,-0.008132,-0.004987,0.004547,0.012960,-0.028398,...,0.007960,0.0001,-0.019765,-0.029365,0.001516,0.031783,0.020368,-0.046113,-0.005784,0.006464
519,2020-10,-0.011785,-0.007752,0.007652,0.044951,0.050133,0.041393,0.023142,-0.019986,0.024671,...,0.008721,0.0466,0.025982,0.032177,-0.004541,-0.009132,-0.000454,-0.019826,0.004541,0.002010
520,2020-11,-0.100174,0.002971,-0.031970,0.021466,0.055007,0.077867,-0.041372,-0.050780,0.055987,...,-0.115229,0.0699,0.056120,0.063769,-0.111326,-0.080291,-0.026441,0.039207,0.012657,0.035152


In [69]:

################################################# saving  ##################################################
semi_ann_factors.to_csv(semi_ann_factor_data_csv) #, compression='zip', encoding='unicode_escape-8'
############################################################################################################

In [72]:
yearly_factors =[] 
for c in column_names:
    if ('1' not in c) and ('6' not in c):
        yearly_factors.append(c)

yearly_factors = all_factors[yearly_factors]
yearly_factors

,date,0trade,aci,adm,amihud,bab,bm,bmj,cei,cla,...,rf,rmw,rs,rvol,smb,sp,sturn,ta,vfp,vhp
0,1977-07,0.018692,-0.021161,0.012778,0.013096,0.028603,-0.010927,-0.004305,-0.009322,0.024058,...,0.0042,0.0074,0.022433,0.006749,0.0188,-0.005382,0.025480,-0.006119,0.008684,-0.007701
1,1977-08,-0.005823,-0.003913,-0.003893,0.001388,0.003437,-0.033594,-0.032617,0.008553,0.022274,...,0.0044,0.0099,0.030851,-0.007967,0.0086,-0.027291,-0.000193,0.016202,-0.005812,-0.029833
2,1977-09,0.009677,-0.002640,-0.007515,0.011608,0.004118,-0.001737,-0.013842,-0.006576,0.022327,...,0.0043,0.0145,0.018186,0.006982,0.0143,-0.006873,0.004160,0.010797,-0.009841,0.018992
3,1977-10,0.011402,-0.005689,0.012399,0.016949,0.001830,0.014104,0.018709,-0.005088,0.002777,...,0.0049,-0.0019,0.001486,-0.001238,0.0148,0.006467,0.010722,0.007045,-0.107805,0.011240
4,1977-11,-0.030581,-0.009848,0.001452,0.041280,0.002795,0.011124,0.003643,0.030013,0.006653,...,0.0050,-0.0013,0.033850,-0.033044,0.0364,0.012190,-0.035501,0.003959,0.028768,0.004361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,2020-08,-0.049368,-0.010580,0.000493,-0.013972,-0.039535,-0.010299,0.031608,-0.061121,0.014467,...,0.0001,0.0436,-0.044317,-0.056292,-0.0092,0.030571,-0.045621,-0.001280,-0.006249,0.038324
518,2020-09,-0.006052,-0.012887,-0.009621,-0.004987,0.012960,-0.028398,-0.063046,-0.001468,-0.017570,...,0.0001,-0.0138,0.010247,0.007960,0.0001,-0.019765,0.001516,0.020368,-0.005784,0.006464
519,2020-10,-0.011785,0.007652,0.044951,0.041393,-0.019986,0.024671,0.042567,-0.020390,0.013470,...,0.0001,-0.0093,-0.033724,0.008721,0.0466,0.025982,-0.004541,-0.000454,0.004541,0.002010
520,2020-11,-0.100174,-0.031970,0.021466,0.077867,-0.050780,0.055987,0.113277,0.021557,-0.064741,...,0.0001,-0.0211,-0.141899,-0.115229,0.0699,0.056120,-0.111326,-0.026441,0.012657,0.035152


In [73]:
################################################# saving  ##################################################
yearly_factors.to_csv(yearly_factor_data_csv) #, compression='zip', encoding='unicode_escape-8'
############################################################################################################